In [ ]:
# %pip install -U kaleido
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import json
import copy
import significantdigits as sd

In [ ]:
# Define the file path to your JSON file
Results_tableNoConf=pd.read_pickle('/home/ubuntu/Desktop/Thesis/overlap/allbatches/Result_tableNoConf.pkl')
Results_tableWConf=pd.read_pickle('/home/ubuntu/Desktop/Thesis/overlap/allbatches/Result_tableWConf.pkl')
subjects_to_remove = ['sub-172260', 'sub-43046', 'sub-43045', 'sub-116870', 'sub-160040', 'sub-168195',  'sub-162941', 'sub-156734', 'sub-140041']#they have RLSplit so has been removed 
Results_tableWConf = Results_tableWConf[~Results_tableWConf['subject'].isin(subjects_to_remove)]

Results_tableNoConf = Results_tableNoConf[~Results_tableNoConf['subject'].isin(subjects_to_remove)]

# filtered_results = Results_table[(Results_table['subject'] == 'sub-133486') &  (Results_table['session'] == 1) &  (Results_table['acquisition'] == 'acq-AP') &    (Results_table['repetition'] == 1)]
# Define the column names
Anatcolumns = [
    'repetition', 
    'session',
    'acquisition', 
    'stacked_matrices',
    'std_matrix',
    'avr_matrix',
    'significantdigits_matrix'
]
# Define the column names
Numcolumns = [
    'subject', 
    'session',
    'acquisition', 
    'stacked_matrices',
    'std_matrix',
    'avr_matrix',
    'significantdigits_matrix',
    'std',
    'mean'
]


In [ ]:
y = pd.DataFrame(columns=Anatcolumns)
all_iteration_data = []
r=Results_tableNoConf.groupby(["repetition","session","acquisition"]).apply(lambda x : x)
for i in r.repetition.unique():
    for j in r.session.unique():
        for k in r.acquisition.unique():
            x = Results_tableNoConf[(Results_tableNoConf['repetition'] == i) & (Results_tableNoConf['session']==j) & (Results_tableNoConf['acquisition']==k)]
            if len(x)<2:
                continue
            # mean = np.array(x[['correlation_matrix']]).squeeze().mean(axis=0)
            # std = np.array(x[['correlation_matrix']]).squeeze().std(axis=0)
            # print(type(np.array(x[['correlation_matrix']]).squeeze()))
            # print(np.array(x[['correlation_matrix']]).squeeze().shape)
            # print(type(mean), mean.shape)
            correlation_matrices = np.stack(x['correlation_matrix'].values)
            mean=np.mean(correlation_matrices,axis=0)
            std=np.std(correlation_matrices,axis=0)
            # display(correlation_matrices.dtype, correlation_matrices.shape)
            sigdig_matrix = sd.significant_digits(
                    correlation_matrices, 
                    reference=mean, 
                    basis=10,
                    dtype=np.float32
                )
            sigdig_matrix[sigdig_matrix<0] = 0   
                            # Compute the standard deviation along axis 0 (across the 10 matrices)

            std_matrixx = np.std(sigdig_matrix)
            avr_matrixx = np.mean(sigdig_matrix)  
            sigdig_matrix[sigdig_matrix>=5] = np.nan

                            # Create a dictionary to store data for this subject, session, and acquisition
            data = {
                'repetition': i,
                'session': j,
                'acquisition': k,
                'stacked_matrices': correlation_matrices,
                'std_matrix':std_matrixx,
                'avr_matrix':avr_matrixx,
                'significant_digits_matrix': sigdig_matrix,
                'std' : std,
                'mean' : mean
            }
            
            # Append the dictionary to the list
            all_iteration_data.append(data)

# Convert the list of dictionaries to a DataFrame
y = pd.DataFrame(all_iteration_data)
np.mean(y['avr_matrix'])

In [ ]:
import numpy as np
import pandas as pd

# Create an empty DataFrame with the defined columns
Significantdigits_AnattableNoConf = pd.DataFrame(columns=Anatcolumns)
# Assuming you know the size of your correlation matrices (100, 100)
matrix_size = (100, 100)
# Columns to exclude
columns_to_exclude = [
    'degree_centralities', 'eigenvector_centralities', 
    'betweenness_centralities', 'clustering_coefficients', 
    'avg_shortest_path_length', 'small_worldness'
]

# Create an empty list to store dictionaries for each iteration
all_iteration_data = []

# Loop over each subject
for rep in range (1,11):

    
    # Filter the DataFrame for each subject and drop unnecessary columns once
    filtered_df = Results_tableNoConf[Results_tableNoConf['repetition'] == f'rep-{rep}'].drop(columns=columns_to_exclude)
    
    # Loop over each unique session and acquisition combination
    for session in np.unique(filtered_df['session']):
        for acquisition in np.unique(filtered_df['acquisition']):
            if acquisition not in ['acq-RLsplit1', 'acq-LRsplit1']:
                try:
                    # Filter for each specific repetition
                    filtered_rows = filtered_df[
                        (filtered_df['session'] == session) &
                        (filtered_df['acquisition'] == acquisition)
                    ]
                        
                    # If no rows match the filter, skip to the next acquisition
                    if len(filtered_rows)< 2:
                        # print(f"No data for {subj}, session {session}, acquisition {acquisition}, rep {rep}. Skipping acquisition.")
                        continue
                    
                    stacked_matrices = np.zeros((len(filtered_rows), *matrix_size))
                    initial_stacked_matrices = stacked_matrices.copy()
                    for idx, subj in  enumerate(filtered_rows['subject'].unique()): 
                        # Assign the matrix to the corresponding slice of the 3D array
                        corr_matrix = filtered_rows[filtered_rows['subject']==subj]['correlation_matrix'].values[0]
                        stacked_matrices[idx] = corr_matrix  
                    # if len(stacked_matrices) < 2:
                    #     continue
                except Exception as e:
                    print(f"Error with {subj}, session {session}, acquisition {acquisition}, rep {rep}: {e}")
                    continue
                # Check if stacked_matrices has changed
                if np.array_equal(stacked_matrices, initial_stacked_matrices):
                    # print(f"No changes in stacked_matrices for {subj}, session {session}, acquisition {acquisition}.")
                    continue  # Skip this session/acquisition if no changes occurred
                # print(stacked_matrices.shape)
                # Compute the significant digits matrix
                std = np.std(stacked_matrices, axis=0)
                mean = np.mean(stacked_matrices, axis=0)
                sigdig_matrix = sd.significant_digits(
                    stacked_matrices, 
                    reference=mean, 
                    basis=10
                )
                sigdig_matrix[sigdig_matrix<0] = 0   
                                # Compute the standard deviation along axis 0 (across the 10 matrices)
                std_matrixx = np.std(sigdig_matrix)
                avr_matrixx = np.mean(sigdig_matrix)  
                sigdig_matrix[sigdig_matrix>=15] = np.nan
          
                # Create a dictionary to store data for this subject, session, and acquisition
                data = {
                    'repetition': rep,
                    'session': session,
                    'acquisition': acquisition,
                    'stacked_matrices': stacked_matrices,
                    'std_matrix':std_matrixx,
                    'avr_matrix':avr_matrixx,
                    'significant_digits_matrix': sigdig_matrix,
                    'std' : std,
                    'mean' : mean
                }
                
                # Append the dictionary to the list
                all_iteration_data.append(data)

# Convert the list of dictionaries to a DataFrame
Significantdigits_AnattableNoConf = pd.DataFrame(all_iteration_data)


In [ ]:
Significantdigits_AnattableNoConf

In [ ]:
Anatfiltered_dfNoConf= Significantdigits_AnattableNoConf[(Significantdigits_AnattableNoConf['acquisition']=='acq-AP')| (Significantdigits_AnattableNoConf['acquisition']=='acq-RL')| (Significantdigits_AnattableNoConf['acquisition']=='acq-ep2d')]
Anatfiltered_dfNoConf
#select 10 Deciles
Anatfiltered_dfNoConf = Anatfiltered_dfNoConf.dropna(subset=['avr_matrix'])
print(len(Anatfiltered_dfNoConf))
Anatsortedsigdig_onAvrg = Anatfiltered_dfNoConf.sort_values('avr_matrix')
Anattenselectsigdig_onAvrgNoConf=Anatsortedsigdig_onAvrg.iloc[[0,6,12,18,24,30,36,42,48,59]]

Anattenselectsigdig_onAvrgNoConf

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

# Assuming your DataFrame is named df and contains 'repetition', 'session', 'acquisition', 'std', 'mean', and 'significant_digits_matrix'
rows = 2
cols = 5
df=Anattenselectsigdig_onAvrgNoConf

# Create a subplot grid for all three types of heatmaps
fig = make_subplots(
    rows=rows * 3, cols=cols,  # Multiply rows by 3 to fit all heatmaps
    subplot_titles=[
        f"Repetition{row['repetition']} - session{row['session']} - {row['acquisition']}" 
        for _ in range(3) for _, row in df.iterrows()   # Repeat each title for 3 heatmaps
    ],
    shared_xaxes=True, shared_yaxes=True,
    vertical_spacing=0.05
)

# Loop through each unique combination in the DataFrame and add heatmaps
for i, (index, dfrow) in enumerate(df.iterrows()):
    # Calculate positions in the subplot grid for each type of heatmap
    base_row = (i // cols) + 1  # Each subject occupies 3 rows
    col_idx = i % cols + 1

    # 1. Significant Digits Heatmap
    sig_digits = np.flipud(dfrow['significant_digits_matrix'])
    fig.add_trace(
        go.Heatmap(
            z=sig_digits,
            colorscale='RdYlGn',
            coloraxis="coloraxis1",  # Shared color axis for significant digits
        ),
        row=base_row, col=col_idx
    )

    # 2. Standard Deviation Heatmap
    std_heatmap = np.flipud(np.log10(dfrow['std']))

    fig.add_trace(
        go.Heatmap(
            z=std_heatmap,
            colorscale='RdYlGn_r',
            coloraxis="coloraxis2",  # Separate color axis for standard deviation
        ),
        row=base_row + 2, col=col_idx
    )

    # 3. Mean Heatmap
    mean_heatmap = np.flipud(dfrow['mean'])

    fig.add_trace(
        go.Heatmap(
            z=mean_heatmap,
            coloraxis="coloraxis3",  # Use a shared color axis,s
            zmid=0,
            zmin=-1,
            zmax=1
        ),
        row=base_row +4 , col=col_idx
    )

fig.update_layout(
    title="Combined Heatmaps for withConfound: Significant Digits, Standard Deviation, and Mean",
    coloraxis1=dict(
        colorbar=dict(
            title="Significant Digits",
            ticks="outside",
            thickness=15,
            len=0.25,  # Adjust length to fit the subplot height
            x=1.18,   # Position to the right of the plot
            y=1.0,    # Align vertically at the top
            yanchor="top"  # Anchor to the top
        ),
        colorscale='RdYlGn',
        cmid=2,
        cmin=-0.5,
        cmax=3.5
    ),
    coloraxis2=dict(
        colorbar=dict(
            title="log(Standard Deviation)",
            ticks="outside",
            thickness=15,
            len=0.25,
            x=1.18,   # Slightly further right than coloraxis1
            y=0.65,
            yanchor="top"
        ),
        colorscale='RdYlGn_r',
        cmid=-2.5,
        cmin=-5,
        cmax=-0.5
    ),
    coloraxis3=dict(
        colorbar=dict(
            title="Mean",
            ticks="outside",
            thickness=15,
            len=0.25,
            x=1.18,   # Further right for better spacing
            y=0.28,
            yanchor="top"
        ),
        colorscale='brBG',
        cmid=0,
        cmin=-1,
        cmax=1
    ),
    height=2100,  # Adjust height to fit all subplots
    width=1800,  # Adjust width to fit all subplots
    annotationdefaults=dict(font=dict(size=8))  # Font size for subplot titles
)
# Add section titles for anatomical and numerical data
fig.add_annotation(
    text="Significant digits across Anatomical Variability",
    xref="paper", yref="paper",
    x=0.0, y=1.018,  # Centered horizontally above the first two rows
    showarrow=False,
)

fig.add_annotation(
    text="Standard deviation across Numerical Variability",
    xref="paper", yref="paper",
    x=0.0, y=0.68,  # Centered horizontally above the last two rows
    showarrow=False,
)
fig.add_annotation(
    text="Average across Numerical Variability",
    xref="paper", yref="paper",
    x=0.0, y=0.33,  # Centered horizontally above the last two rows
    showarrow=False,
)
# Update subplot titles for better readability
for annotation in fig['layout']['annotations']:
    annotation['font'] = dict(size=12)

# Show the figure
fig.show()


In [ ]:
import numpy as np
import pandas as pd

# Create an empty DataFrame with the defined columns
Significantdigits_NumtableNoConf = pd.DataFrame(columns=Numcolumns)
# Assuming you know the size of your correlation matrices (100, 100)
matrix_size = (100, 100)
# Columns to exclude
columns_to_exclude = [
    'degree_centralities', 'eigenvector_centralities', 
    'betweenness_centralities', 'clustering_coefficients', 
    'avg_shortest_path_length', 'small_worldness'
]

# Create an empty list to store dictionaries for each subject
all_subject_data = []

# Get unique subjects from the Results_table
subjects =np.unique(Results_tableNoConf['subject'])

# Loop over each subject
for subj in subjects:

    
    # Filter the DataFrame for each subject and drop unnecessary columns once
    filtered_df = Results_tableNoConf[Results_tableNoConf['subject'] == subj].drop(columns=columns_to_exclude)
    
    # Loop over each unique session and acquisition combination
    for session in np.unique(filtered_df['session']):
        for acquisition in np.unique(filtered_df['acquisition']):
            if acquisition not in ['acq-RLsplit1', 'acq-LRsplit1']:
                    # Pre-allocate a 3D array to store 10 correlation matrices for this subject
                stacked_matrices = np.zeros((10, *matrix_size))
                initial_stacked_matrices = stacked_matrices.copy()
                # Loop over each repetition to generate correlation matrices
                for rep in range(1, 11):
                    try:
                        # Filter for each specific repetition
                        filtered_rows = filtered_df[
                            (filtered_df['session'] == session) &
                            (filtered_df['acquisition'] == acquisition) &
                            (filtered_df['repetition'] == f'rep-{rep}')
                        ]
                            
                        # If no rows match the filter, skip to the next acquisition
                        if filtered_rows.empty:
                            # print(f"No data for {subj}, session {session}, acquisition {acquisition}, rep {rep}. Skipping acquisition.")
                            continue

                        # Assign the matrix to the corresponding slice of the 3D array
                        corr_matrix = filtered_rows['correlation_matrix'].values[0]
                        stacked_matrices[rep - 1] = corr_matrix  
                    except Exception as e:
                        print(f"Error with {subj}, session {session}, acquisition {acquisition}, rep {rep}: {e}")
                        continue
                # Check if stacked_matrices has changed
                if np.array_equal(stacked_matrices, initial_stacked_matrices):
                    # print(f"No changes in stacked_matrices for {subj}, session {session}, acquisition {acquisition}.")
                    continue  # Skip this session/acquisition if no changes occurred

                # Compute the significant digits matrix
                std = np.std(stacked_matrices, axis=0)
                mean = np.mean(stacked_matrices, axis=0)
                sigdig_matrix = sd.significant_digits(
                    stacked_matrices, 
                    reference=mean, 
                    basis=10
                )
                sigdig_matrix[sigdig_matrix<0] = 0   
                                # Compute the standard deviation along axis 0 (across the 10 matrices)
                std_matrixx = np.std(sigdig_matrix)
                avr_matrixx = np.mean(sigdig_matrix)  
                sigdig_matrix[sigdig_matrix>=15] = np.nan
          
                # Create a dictionary to store data for this subject, session, and acquisition
                data = {
                    'subject': subj,
                    'session': session,
                    'acquisition': acquisition,
                    'stacked_matrices': stacked_matrices,
                    'std_matrix':std_matrixx,
                    'avr_matrix':avr_matrixx,
                    'significant_digits_matrix': sigdig_matrix,
                    'std' : std,
                    'mean' : mean
                }
                
                # Append the dictionary to the list
                all_subject_data.append(data)

# Convert the list of dictionaries to a DataFrame
Significantdigits_NumtableNoConf = pd.DataFrame(all_subject_data)


In [ ]:
import numpy as np
import pandas as pd

# Create an empty DataFrame with the defined columns
Significantdigits_NumtableWConf = pd.DataFrame(columns=Numcolumns)
# Assuming you know the size of your correlation matrices (100, 100)
matrix_size = (100, 100)
# Columns to exclude
columns_to_exclude = [
    'degree_centralities', 'eigenvector_centralities', 
    'betweenness_centralities', 'clustering_coefficients', 
    'avg_shortest_path_length', 'small_worldness'
]

# Create an empty list to store dictionaries for each subject
all_subject_data = []

# Get unique subjects from the Results_table
subjects =np.unique(Results_tableWConf['subject'])

# Loop over each subject
for subj in subjects:

    
    # Filter the DataFrame for each subject and drop unnecessary columns once
    filtered_df = Results_tableWConf[Results_tableWConf['subject'] == subj].drop(columns=columns_to_exclude)
    
    # Loop over each unique session and acquisition combination
    for session in np.unique(filtered_df['session']):
        for acquisition in np.unique(filtered_df['acquisition']):
            if acquisition not in ['acq-RLsplit1', 'acq-LRsplit1']:
                    # Pre-allocate a 3D array to store 10 correlation matrices for this subject
                stacked_matrices = np.zeros((10, *matrix_size))
                initial_stacked_matrices = stacked_matrices.copy()
                # Loop over each repetition to generate correlation matrices
                for rep in range(1, 11):
                    try:
                        # Filter for each specific repetition
                        filtered_rows = filtered_df[
                            (filtered_df['session'] == session) &
                            (filtered_df['acquisition'] == acquisition) &
                            (filtered_df['repetition'] == f'rep-{rep}')
                        ]
                            
                        # If no rows match the filter, skip to the next acquisition
                        if filtered_rows.empty:
                            # print(f"No data for {subj}, session {session}, acquisition {acquisition}, rep {rep}. Skipping acquisition.")
                            continue

                        # Assign the matrix to the corresponding slice of the 3D array
                        corr_matrix = filtered_rows['correlation_matrix'].values[0]
                        stacked_matrices[rep - 1] = corr_matrix  
                    except Exception as e:
                        print(f"Error with {subj}, session {session}, acquisition {acquisition}, rep {rep}: {e}")
                        continue
                # Check if stacked_matrices has changed
                if np.array_equal(stacked_matrices, initial_stacked_matrices):
                    # print(f"No changes in stacked_matrices for {subj}, session {session}, acquisition {acquisition}.")
                    continue  # Skip this session/acquisition if no changes occurred

                # Compute the significant digits matrix
                std = np.std(stacked_matrices, axis=0)
                mean = np.mean(stacked_matrices, axis=0)
                sigdig_matrix = sd.significant_digits(
                    stacked_matrices, 
                    reference=mean, 
                    basis=10
                )
                sigdig_matrix[sigdig_matrix<0] = 0   
                                # Compute the standard deviation along axis 0 (across the 10 matrices)
                std_matrixx = np.std(sigdig_matrix)
                avr_matrixx = np.mean(sigdig_matrix)  
                sigdig_matrix[sigdig_matrix>=15] = np.nan
          
                # Create a dictionary to store data for this subject, session, and acquisition
                data = {
                    'subject': subj,
                    'session': session,
                    'acquisition': acquisition,
                    'stacked_matrices': stacked_matrices,
                    'std_matrix':std_matrixx,
                    'avr_matrix':avr_matrixx,
                    'significant_digits_matrix': sigdig_matrix,
                    'std' : std,
                    'mean' : mean
                }
                
                # Append the dictionary to the list
                all_subject_data.append(data)

# Convert the list of dictionaries to a DataFrame
Significantdigits_NumtableWConf = pd.DataFrame(all_subject_data)


In [ ]:
# Significantdigits_NumtableNoConf[Significantdigits_NumtableNoConf['subject']=='sub-140041']
# NoConf has one row less than WConf
len(Significantdigits_NumtableNoConf)
print (np.mean(Significantdigits_NumtableWConf['avr_matrix'])) 
np.mean(Significantdigits_NumtableNoConf['avr_matrix'])

# Identify missing items based on a unique identifier column (change 'ID' to the actual column name)
# Identify missing items based on subject and acquisition together
# missing_in_pkl2 = Significantdigits_NumtableNoConf[
#     ~Significantdigits_NumtableNoConf[['subject', 'acquisition','session']].apply(tuple, axis=1).isin(
#         Significantdigits_NumtableWConf[['subject', 'acquisition','session']].apply(tuple, axis=1)
#     )
# ]

# missing_in_pkl1 = Significantdigits_NumtableWConf[
#     ~Significantdigits_NumtableWConf[['subject', 'acquisition','session']].apply(tuple, axis=1).isin(
#         Significantdigits_NumtableNoConf[['subject', 'acquisition','session']].apply(tuple, axis=1)
#     )
# ]

# print(missing_in_pkl1['subject'])
# print(missing_in_pkl2['subject'])
# Significantdigits_NumtableWConf[Significantdigits_NumtableWConf['subject']=='sub-140041']

In [ ]:
filtered_tableW = Significantdigits_NumtableWConf[
    (Significantdigits_NumtableWConf['acquisition'] == 'acq-RL') |
    (Significantdigits_NumtableWConf['acquisition'] == 'acq-ep2d')|
    (Significantdigits_NumtableWConf['acquisition'] == 'acq-PA')
    
]
#select 10 Deciles
sortedsigdig_onAvrg = filtered_tableW.sort_values('avr_matrix')
tenselectsigdig_onAvrgWConf=sortedsigdig_onAvrg.iloc[[0,20,40,60,80,100,120,140,158,177]]
tenselectsigdig_onAvrgWConf


In [ ]:
# Find the decile subjects from Wbatch2 list in No confound batch2
matches = []  # List to store matched rows

# Loop through rows of tenselectedsub
for _, row in tenselectsigdig_onAvrgWConf.iterrows():
    subj = row['subject']
    session = row['session']
    acq = row['acquisition']

    # Filter rows from Significantdigits_Numtable matching the current row
    match = Significantdigits_NumtableNoConf[
        (Significantdigits_NumtableNoConf['subject'] == subj) &
        (Significantdigits_NumtableNoConf['session'] == str(session)) &
        (Significantdigits_NumtableNoConf['acquisition'] == acq)
    ]
    matches.append(match)

# Concatenate all matcnhed rows into a single DataFrame
tenselectsigdig_onAvrgNoConf = pd.concat(matches, ignore_index=True)
tenselectsigdig_onAvrgNoConf
# tenselectsigdig_onAvrg
# match=Significantdigits_Numtable[Significantdigits_Numtable['subject']=='' & Significantdigits_Numtable['session']=='' & Significantdigits_Numtable['acquisition']=='']

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

# Assuming your DataFrame is named df and contains 'repetition', 'session', 'acquisition', 'std', 'mean', and 'significant_digits_matrix'
rows = 2
cols = 5
df=tenselectsigdig_onAvrgWConf

# Create a subplot grid for all three types of heatmaps
fig = make_subplots(
    rows=rows * 3, cols=cols,  # Multiply rows by 3 to fit all heatmaps
    subplot_titles=[
        f"{row['subject']} - {row['session']} - {row['acquisition']}" 
        for _ in range(3) for _, row in df.iterrows()   # Repeat each title for 3 heatmaps
    ],
    shared_xaxes=True, shared_yaxes=True,
    vertical_spacing=0.05
)

# Loop through each unique combination in the DataFrame and add heatmaps
for i, (index, dfrow) in enumerate(df.iterrows()):
    # Calculate positions in the subplot grid for each type of heatmap
    base_row = (i // cols) + 1  # Each subject occupies 3 rows
    col_idx = i % cols + 1

    # 1. Significant Digits Heatmap
    sig_digits = np.flipud(dfrow['significant_digits_matrix'])
    fig.add_trace(
        go.Heatmap(
            z=sig_digits,
            colorscale='RdYlGn',
            coloraxis="coloraxis1",  # Shared color axis for significant digits
        ),
        row=base_row, col=col_idx
    )

    # 2. Standard Deviation Heatmap
    std_heatmap = np.flipud(np.log10(dfrow['std']))

    fig.add_trace(
        go.Heatmap(
            z=std_heatmap,
            colorscale='RdYlGn_r',
            coloraxis="coloraxis2",  # Separate color axis for standard deviation
        ),
        row=base_row + 2, col=col_idx
    )

    # 3. Mean Heatmap
    mean_heatmap = np.flipud(dfrow['mean'])

    fig.add_trace(
        go.Heatmap(
            z=mean_heatmap,
            coloraxis="coloraxis3",  # Use a shared color axis,s
            zmid=0,
            zmin=-1,
            zmax=1
        ),
        row=base_row +4 , col=col_idx
    )

fig.update_layout(
    title="Combined Heatmaps for withConfound: Significant Digits, Standard Deviation, and Mean",
    coloraxis1=dict(
        colorbar=dict(
            title="Significant Digits",
            ticks="outside",
            thickness=15,
            len=0.25,  # Adjust length to fit the subplot height
            x=1.18,   # Position to the right of the plot
            y=1.0,    # Align vertically at the top
            yanchor="top"  # Anchor to the top
        ),
        colorscale='RdYlGn',
        cmid=2,
        cmin=-0.5,
        cmax=3.5
    ),
    coloraxis2=dict(
        colorbar=dict(
            title="log(Standard Deviation)",
            ticks="outside",
            thickness=15,
            len=0.25,
            x=1.18,   # Slightly further right than coloraxis1
            y=0.65,
            yanchor="top"
        ),
        colorscale='RdYlGn_r',
        cmid=-2.5,
        cmin=-5,
        cmax=-0.5
    ),
    coloraxis3=dict(
        colorbar=dict(
            title="Mean",
            ticks="outside",
            thickness=15,
            len=0.25,
            x=1.18,   # Further right for better spacing
            y=0.28,
            yanchor="top"
        ),
        colorscale='brBG',
        cmid=0,
        cmin=-1,
        cmax=1
    ),
    height=2100,  # Adjust height to fit all subplots
    width=1800,  # Adjust width to fit all subplots
    annotationdefaults=dict(font=dict(size=8))  # Font size for subplot titles
)
# Add section titles for anatomical and numerical data
fig.add_annotation(
    text="Significant digits across Numerical Variability",
    xref="paper", yref="paper",
    x=0.0, y=1.018,  # Centered horizontally above the first two rows
    showarrow=False,
)

fig.add_annotation(
    text="Standard deviation across Numerical Variability",
    xref="paper", yref="paper",
    x=0.0, y=0.68,  # Centered horizontally above the last two rows
    showarrow=False,
)
fig.add_annotation(
    text="Average across Numerical Variability",
    xref="paper", yref="paper",
    x=0.0, y=0.33,  # Centered horizontally above the last two rows
    showarrow=False,
)
# Update subplot titles for better readability
for annotation in fig['layout']['annotations']:
    annotation['font'] = dict(size=12)

# Show the figure
fig.show()


In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

# Assuming your DataFrame is named df and contains 'repetition', 'session', 'acquisition', 'std', 'mean', and 'significant_digits_matrix'
rows = 2
cols = 5
df=tenselectsigdig_onAvrgNoConf

# Create a subplot grid for all three types of heatmaps
fig = make_subplots(
    rows=rows * 3, cols=cols,  # Multiply rows by 3 to fit all heatmaps
    subplot_titles=[
        f"{row['subject']} - {row['session']} - {row['acquisition']}" 
        for _ in range(3) for _, row in df.iterrows()   # Repeat each title for 3 heatmaps
    ],
    shared_xaxes=True, shared_yaxes=True,
    vertical_spacing=0.05
)

# Loop through each unique combination in the DataFrame and add heatmaps
for i, (index, dfrow) in enumerate(df.iterrows()):
    # Calculate positions in the subplot grid for each type of heatmap
    base_row = (i // cols) + 1  # Each subject occupies 3 rows
    col_idx = i % cols + 1

    # 1. Significant Digits Heatmap
    sig_digits = np.flipud(dfrow['significant_digits_matrix'])
    fig.add_trace(
        go.Heatmap(
            z=sig_digits,
            colorscale='RdYlGn',
            coloraxis="coloraxis1",  # Shared color axis for significant digits
        ),
        row=base_row, col=col_idx
    )

    # 2. Standard Deviation Heatmap
    std_heatmap = np.flipud(np.log10(dfrow['std']))

    fig.add_trace(
        go.Heatmap(
            z=std_heatmap,
            colorscale='RdYlGn_r',
            coloraxis="coloraxis2",  # Separate color axis for standard deviation
        ),
        row=base_row + 2, col=col_idx
    )

    # 3. Mean Heatmap
    mean_heatmap = np.flipud(dfrow['mean'])

    fig.add_trace(
        go.Heatmap(
            z=mean_heatmap,
            coloraxis="coloraxis3",  # Use a shared color axis,s
            zmid=0,
            zmin=-1,
            zmax=1
        ),
        row=base_row +4 , col=col_idx
    )

fig.update_layout(
    title="Combined Heatmaps for No Confound: Significant Digits, Standard Deviation, and Mean with same subjects for With Confounds",
    coloraxis1=dict(
        colorbar=dict(
            title="Significant Digits",
            ticks="outside",
            thickness=15,
            len=0.25,  # Adjust length to fit the subplot height
            x=1.18,   # Position to the right of the plot
            y=1.0,    # Align vertically at the top
            yanchor="top"  # Anchor to the top
        ),
        colorscale='RdYlGn',
        cmid=2,
        cmin=-0.5,
        cmax=3.5
    ),
    coloraxis2=dict(
        colorbar=dict(
            title="log(Standard Deviation)",
            ticks="outside",
            thickness=15,
            len=0.25,
            x=1.18,   # Slightly further right than coloraxis1
            y=0.65,
            yanchor="top"
        ),
        colorscale='RdYlGn_r',
        cmid=-2.5,
        cmin=-5,
        cmax=-0.5
    ),
    coloraxis3=dict(
        colorbar=dict(
            title="Mean",
            ticks="outside",
            thickness=15,
            len=0.25,
            x=1.18,   # Further right for better spacing
            y=0.28,
            yanchor="top"
        ),
        colorscale='brBG',
        cmid=0,
        cmin=-1,
        cmax=1
    ),
    height=2100,  # Adjust height to fit all subplots
    width=1800,  # Adjust width to fit all subplots
    annotationdefaults=dict(font=dict(size=8))  # Font size for subplot titles
)
# Add section titles for anatomical and numerical data
fig.add_annotation(
    text="Significant digits across Numerical Variability",
    xref="paper", yref="paper",
    x=0.0, y=1.018,  # Centered horizontally above the first two rows
    showarrow=False,
)

fig.add_annotation(
    text="Standard deviation across Numerical Variability",
    xref="paper", yref="paper",
    x=0.0, y=0.68,  # Centered horizontally above the last two rows
    showarrow=False,
)
fig.add_annotation(
    text="Average across Numerical Variability",
    xref="paper", yref="paper",
    x=0.0, y=0.33,  # Centered horizontally above the last two rows
    showarrow=False,
)
# Update subplot titles for better readability
for annotation in fig['layout']['annotations']:
    annotation['font'] = dict(size=12)

# Show the figure
fig.show()


In [ ]:
filtered_tableNo = Significantdigits_NumtableNoConf[
    (Significantdigits_NumtableNoConf['acquisition'] == 'acq-RL') |
    (Significantdigits_NumtableNoConf['acquisition'] == 'acq-ep2d')|
    (Significantdigits_NumtableNoConf['acquisition'] == 'acq-PA')
    
]
print(len(filtered_tableNo))
#select 10 Deciles
sortedsigdig_onAvrgNoConff = filtered_tableNo.sort_values('avr_matrix')
tenselectsigdig_onAvrgNoConff=sortedsigdig_onAvrgNoConff.iloc[[0,20,40,60,80,100,120,140,158,177]]
tenselectsigdig_onAvrgNoConff

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

# Assuming your DataFrame is named df and contains 'repetition', 'session', 'acquisition', 'std', 'mean', and 'significant_digits_matrix'
rows = 2
cols = 5
df=tenselectsigdig_onAvrgNoConff

# Create a subplot grid for all three types of heatmaps
fig = make_subplots(
    rows=rows * 3, cols=cols,  # Multiply rows by 3 to fit all heatmaps
    subplot_titles=[
        f"{row['subject']} - {row['session']} - {row['acquisition']}" 
        for _ in range(3) for _, row in df.iterrows()   # Repeat each title for 3 heatmaps
    ],
    shared_xaxes=True, shared_yaxes=True,
    vertical_spacing=0.05
)

# Loop through each unique combination in the DataFrame and add heatmaps
for i, (index, dfrow) in enumerate(df.iterrows()):
    # Calculate positions in the subplot grid for each type of heatmap
    base_row = (i // cols) + 1  # Each subject occupies 3 rows
    col_idx = i % cols + 1

    # 1. Significant Digits Heatmap
    sig_digits = np.flipud(dfrow['significant_digits_matrix'])
    fig.add_trace(
        go.Heatmap(
            z=sig_digits,
            colorscale='RdYlGn',
            coloraxis="coloraxis1",  # Shared color axis for significant digits
        ),
        row=base_row, col=col_idx
    )

    # 2. Standard Deviation Heatmap
    std_heatmap = np.flipud(np.log10(dfrow['std']))

    fig.add_trace(
        go.Heatmap(
            z=std_heatmap,
            colorscale='RdYlGn_r',
            coloraxis="coloraxis2",  # Separate color axis for standard deviation
        ),
        row=base_row + 2, col=col_idx
    )

    # 3. Mean Heatmap
    mean_heatmap = np.flipud(dfrow['mean'])

    fig.add_trace(
        go.Heatmap(
            z=mean_heatmap,
            coloraxis="coloraxis3",  # Use a shared color axis,s
            zmid=0,
            zmin=-1,
            zmax=1
        ),
        row=base_row +4 , col=col_idx
    )

fig.update_layout(
    title="Combined Heatmaps for No Confound: Significant Digits, Standard Deviation, and Mean",
    coloraxis1=dict(
        colorbar=dict(
            title="Significant Digits",
            ticks="outside",
            thickness=15,
            len=0.25,  # Adjust length to fit the subplot height
            x=1.18,   # Position to the right of the plot
            y=1.0,    # Align vertically at the top
            yanchor="top"  # Anchor to the top
        ),
        colorscale='RdYlGn',
        cmid=2,
        cmin=-0.5,
        cmax=3.5
    ),
    coloraxis2=dict(
        colorbar=dict(
            title="log(Standard Deviation)",
            ticks="outside",
            thickness=15,
            len=0.25,
            x=1.18,   # Slightly further right than coloraxis1
            y=0.65,
            yanchor="top"
        ),
        colorscale='RdYlGn_r',
        cmid=-2.5,
        cmin=-5,
        cmax=-0.5
    ),
    coloraxis3=dict(
        colorbar=dict(
            title="Mean",
            ticks="outside",
            thickness=15,
            len=0.25,
            x=1.18,   # Further right for better spacing
            y=0.28,
            yanchor="top"
        ),
        colorscale='brBG',
        cmid=0,
        cmin=-1,
        cmax=1
    ),
    height=2100,  # Adjust height to fit all subplots
    width=1800,  # Adjust width to fit all subplots
    annotationdefaults=dict(font=dict(size=8))  # Font size for subplot titles
)
# Add section titles for anatomical and numerical data
fig.add_annotation(
    text="Significant digits across Numerical Variability",
    xref="paper", yref="paper",
    x=0.0, y=1.018,  # Centered horizontally above the first two rows
    showarrow=False,
)

fig.add_annotation(
    text="Standard deviation across Numerical Variability",
    xref="paper", yref="paper",
    x=0.0, y=0.68,  # Centered horizontally above the last two rows
    showarrow=False,
)
fig.add_annotation(
    text="Average across Numerical Variability",
    xref="paper", yref="paper",
    x=0.0, y=0.33,  # Centered horizontally above the last two rows
    showarrow=False,
)
# Update subplot titles for better readability
for annotation in fig['layout']['annotations']:
    annotation['font'] = dict(size=12)

# Show the figure
fig.show()


In [ ]:
# Find the decile subjects from Wbatch2 list in No confound batch2
matchess = []  # List to store matched rows

# Loop through rows of tenselectedsub
for _, row in tenselectsigdig_onAvrgNoConff.iterrows():
    subj = row['subject']
    session = row['session']
    acq = row['acquisition']

    # Filter rows from Significantdigits_Numtable matching the current row
    match = Significantdigits_NumtableWConf[
        (Significantdigits_NumtableWConf['subject'] == subj) &
        (Significantdigits_NumtableWConf['session'] == str(session)) &
        (Significantdigits_NumtableWConf['acquisition'] == acq)
    ]
    matchess.append(match)

# Concatenate all matcnhed rows into a single DataFrame
tenselectsigdig_onAvrgWConfs = pd.concat(matchess, ignore_index=True)
tenselectsigdig_onAvrgWConfs

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

# Assuming your DataFrame is named df and contains 'repetition', 'session', 'acquisition', 'std', 'mean', and 'significant_digits_matrix'
rows = 2
cols = 5
df=tenselectsigdig_onAvrgWConfs

# Create a subplot grid for all three types of heatmaps
fig = make_subplots(
    rows=rows * 3, cols=cols,  # Multiply rows by 3 to fit all heatmaps
    subplot_titles=[
        f"{row['subject']} - {row['session']} - {row['acquisition']}" 
        for _ in range(3) for _, row in df.iterrows()   # Repeat each title for 3 heatmaps
    ],
    shared_xaxes=True, shared_yaxes=True,
    vertical_spacing=0.05
)

# Loop through each unique combination in the DataFrame and add heatmaps
for i, (index, dfrow) in enumerate(df.iterrows()):
    # Calculate positions in the subplot grid for each type of heatmap
    base_row = (i // cols) + 1  # Each subject occupies 3 rows
    col_idx = i % cols + 1

    # 1. Significant Digits Heatmap
    sig_digits = np.flipud(dfrow['significant_digits_matrix'])
    fig.add_trace(
        go.Heatmap(
            z=sig_digits,
            colorscale='RdYlGn',
            coloraxis="coloraxis1",  # Shared color axis for significant digits
        ),
        row=base_row, col=col_idx
    )

    # 2. Standard Deviation Heatmap
    std_heatmap = np.flipud(np.log10(dfrow['std']))

    fig.add_trace(
        go.Heatmap(
            z=std_heatmap,
            colorscale='RdYlGn_r',
            coloraxis="coloraxis2",  # Separate color axis for standard deviation
        ),
        row=base_row + 2, col=col_idx
    )

    # 3. Mean Heatmap
    mean_heatmap = np.flipud(dfrow['mean'])

    fig.add_trace(
        go.Heatmap(
            z=mean_heatmap,
            coloraxis="coloraxis3",  # Use a shared color axis,s
            zmid=0,
            zmin=-1,
            zmax=1
        ),
        row=base_row +4 , col=col_idx
    )

fig.update_layout(
    title="Combined Heatmaps for with Confound: Significant Digits, Standard Deviation, and Mean with no confond match subjects",
    coloraxis1=dict(
        colorbar=dict(
            title="Significant Digits",
            ticks="outside",
            thickness=15,
            len=0.25,  # Adjust length to fit the subplot height
            x=1.18,   # Position to the right of the plot
            y=1.0,    # Align vertically at the top
            yanchor="top"  # Anchor to the top
        ),
        colorscale='RdYlGn',
        cmid=2,
        cmin=-0.5,
        cmax=3.5
    ),
    coloraxis2=dict(
        colorbar=dict(
            title="log(Standard Deviation)",
            ticks="outside",
            thickness=15,
            len=0.25,
            x=1.18,   # Slightly further right than coloraxis1
            y=0.65,
            yanchor="top"
        ),
        colorscale='RdYlGn_r',
        cmid=-2.5,
        cmin=-5,
        cmax=-0.5
    ),
    coloraxis3=dict(
        colorbar=dict(
            title="Mean",
            ticks="outside",
            thickness=15,
            len=0.25,
            x=1.18,   # Further right for better spacing
            y=0.28,
            yanchor="top"
        ),
        colorscale='brBG',
        cmid=0,
        cmin=-1,
        cmax=1
    ),
    height=2100,  # Adjust height to fit all subplots
    width=1800,  # Adjust width to fit all subplots
    annotationdefaults=dict(font=dict(size=8))  # Font size for subplot titles
)
# Add section titles for anatomical and numerical data
fig.add_annotation(
    text="Significant digits across Numerical Variability",
    xref="paper", yref="paper",
    x=0.0, y=1.018,  # Centered horizontally above the first two rows
    showarrow=False,
)

fig.add_annotation(
    text="Standard deviation across Numerical Variability",
    xref="paper", yref="paper",
    x=0.0, y=0.68,  # Centered horizontally above the last two rows
    showarrow=False,
)
fig.add_annotation(
    text="Average across Numerical Variability",
    xref="paper", yref="paper",
    x=0.0, y=0.33,  # Centered horizontally above the last two rows
    showarrow=False,
)
# Update subplot titles for better readability
for annotation in fig['layout']['annotations']:
    annotation['font'] = dict(size=12)

# Show the figure
fig.show()


In [ ]:
#KS-test for NoConf and WConf with multiple comparison correction
def ksTestWithCorrection(wconf, noconf, alt, correction='bonferroni'):
    import numpy as np
    from scipy.stats import ks_2samp
    from statsmodels.stats.multitest import multipletests

    # Significant digit matrices for conf and non-conf groups
    conf_matrix = wconf 
    non_conf_matrix = noconf 

    # Initialize arrays to store KS-statistics and p-values
    n_nodes0= conf_matrix.shape[1] 
    n_nodes1 = conf_matrix.shape[2]    
    ks_connectome = np.zeros((n_nodes0, n_nodes1))
    pval_connectome = np.zeros((n_nodes0, n_nodes1))

    # Iterate over the upper triangle (excluding the diagonal)
    for i in range(n_nodes0):
        for j in range(i + 1, n_nodes1):  # Upper triangle only
            # Extract values across subjects for the edge (i, j)
            conf_values = conf_matrix[:, i, j]
            non_conf_values = non_conf_matrix[:, i, j]
            # Perform KS-test
            ks_stat, p_value = ks_2samp(conf_values,non_conf_values, alternative=alt)
            
            # Store results
            ks_connectome[i, j] = ks_stat
            pval_connectome[i, j] = p_value
    if  n_nodes1!=n_nodes0: 
        for i in range(100):
            # Extract values across subjects for the edge (i, j)
            conf_values = conf_matrix[:, 0, i]
            non_conf_values = non_conf_matrix[:, 0, i]
            # Perform KS-test
            ks_stat, p_value = ks_2samp(conf_values,non_conf_values, alternative=alt)
            
            # Store results
            ks_connectome[0,i] = ks_stat
            pval_connectome[0,i] = p_value

    if n_nodes1==n_nodes0:
        # Mirror the upper triangle to the lower triangle for symmetry
        ks_connectome = ks_connectome + ks_connectome.T
        pval_connectome = pval_connectome + pval_connectome.T
         # Flatten the p-value matrix and apply multiple comparison correction
        pval_flat = pval_connectome[np.triu_indices(n_nodes0, k=1)]  # Upper triangle only
        # print(pval_connectome.shape,pval_flat.shape)
        if correction == 'Bonferroni':
            # Bonferroni correction
            res, pval_corrected, _, _ = multipletests(pval_flat, method='bonferroni')
        elif correction == 'FDR':
            # Benjamini-Hochberg FDR correctionres
            res, pval_corrected, _, _ = multipletests(pval_flat, method='fdr_bh')
        else:
            raise ValueError("Invalid correction method. Use 'Bonferroni' or 'FDR'.")
    else:
        pval_flat = pval_connectome.flatten().tolist()
        # print(pval_flat)
        if correction == 'Bonferroni':
            # Bonferroni correction
            res, pval_corrected, _, _ = multipletests(pval_flat, method='bonferroni')
            pval_corrected_matrix=pval_corrected
            res_flt= res.astype(np.float32)
            # print(res_flt)
        elif correction == 'FDR':
            # Benjamini-Hochberg FDR correctionres
            res, pval_corrected, _, _ = multipletests(pval_flat, method='fdr_bh')
            pval_corrected_matrix=pval_corrected
            res_flt= res.astype(np.float32)
        else:
            raise ValueError("Invalid correction method. Use 'Bonferroni' or 'FDR'.")
    if n_nodes1==n_nodes0:
        # Map the corrected p-values back to the upper triangle of the matrix
        pval_corrected_matrix = np.zeros_like(pval_connectome)
        res_flt = np.zeros_like(ks_connectome)
        res_flt[np.triu_indices(n_nodes0, k=1)] = res.astype(np.float32)
        res_flt = res_flt + res_flt.T
        pval_corrected_matrix[np.triu_indices(n_nodes0, k=1)] = pval_corrected  #significant_mask = (pval_corrected < 0.05)
        pval_corrected_matrix = pval_corrected_matrix + pval_corrected_matrix.T
        # print(res_flt.shape,pval_corrected_matrix.shape)
        # print(res.shape,res_flt.shape)
    return ks_connectome, pval_connectome, pval_corrected_matrix,res_flt


In [ ]:
def stack_sub(d1,d2,mode):
    import numpy as np

    # Assuming you want a 1D array for storing the means of each subject

    # Initialize mat with the correct shape (number of subjects, rows, columns)
    d1n_subjects = len(d1)  # Number of subjects
    d2n_subjects = len(d2) 
    n_rows, n_columns = d1.iloc[0][mode].shape[0],d1.iloc[0][mode].shape[1] # Assuming each significant_digits_matrix is 100x100
    subj_WconfMat = np.zeros((d1n_subjects, n_rows, n_columns)) # Initialize mat with the number of subjects
    subj_NoconfMat = np.zeros((d2n_subjects, n_rows, n_columns)) # Initialize mat with the number of subjects
    
    # Iterate through each row of the DataFrame
    count=0
    for idx, row in d1.iterrows():
        # Extract the significant_digits_matrix for the current subject
        mat1 = row[mode]
        # Assign the 2D matrix to the correct position in mat
        subj_WconfMat[count] = np.array(mat1)  
        count +=1
    # Iterate through each row of the DataFrame
    count =0
    for idx, row in d2.iterrows():
        # Extract the significant_digits_matrix for the current subject
        mat2= row[mode]
        # Assign the 2D matrix to the correct position in mat
        subj_NoconfMat[count] = np.array(mat2)  
        count +=1
    return subj_WconfMat, subj_NoconfMat

In [ ]:
filtered_table = filtered_tableNo [ (filtered_tableNo['subject'] != 'sub-140041') & (filtered_tableNo['subject'] != 'sub-100898')

 ]  #exclude 100898 as it has 18 iteration and 140041 as in noconfound analysis has only one session but in with confoun dtwo session
filtered_tableWconf = filtered_tableW[(filtered_tableW['subject'] != 'sub-140041') & (filtered_tableW['subject'] != 'sub-100898')
    #                                                 
  ]

missing_subj = filtered_tableWconf[~filtered_tableWconf['subject'].isin(filtered_table['subject'])]
print("missing in noconf",missing_subj['subject'])

missing_subj = filtered_table[~filtered_table['subject'].isin(filtered_tableWconf['subject'])]
print("missing in Wconf",missing_subj['subject'])    

print(len(filtered_tableWconf))
print(len(filtered_table))
print(filtered_table.shape)

# print(len(filtered_table),len(filtered_tableWconf))
wconfsig,noconfsig=stack_sub(filtered_tableWconf,filtered_table,'significant_digits_matrix')#mode can be : std , mean , significant_digits_matrix
wconfstd,noconfstd=stack_sub(filtered_tableWconf,filtered_table,'std')#mode can be : std , mean , significant_digits_matrix
wconfavr,noconfavr=stack_sub(filtered_tableWconf,filtered_table,'mean')#mode can be : std , mean , significant_digits_matrix
print(wconfsig.shape)
kssigtwoside,pvalsigtwoside,pvalcorsigtwoside,ressig_twoside=ksTestWithCorrection(wconfsig,noconfsig,'two-side','Bonferroni') #FDR,Bonferroni
kssiggreater,pvalsiggreater,pvalcorsiggreater,ressig_greater=ksTestWithCorrection(wconfsig,noconfsig,'greater','Bonferroni') #FDR,Bonferroni
kssigless,pvalsigless,pvalcorsigless,ressig_less=ksTestWithCorrection(wconfsig,noconfsig,'less','Bonferroni') #FDR,Bonferroni
ksstdtwoside,pvalstdtwoside,pvalcorstdtwoside,resstd_twoside=ksTestWithCorrection(wconfstd,noconfstd,'two-side','Bonferroni') #FDR,Bonferroni
ksstdless,pvalstdless,pvalcorstdless,resstd_less=ksTestWithCorrection(wconfstd,noconfstd,'less','Bonferroni') #FDR,Bonferroni


In [ ]:
# wconfsig.shape

In [ ]:
wconfsig_avr=np.mean(wconfsig,axis=0)
noconfsig_avr=np.mean(noconfsig,axis=0)
wconfstd_avr=np.mean(wconfstd,axis=0)
noconfstd_avr=np.mean(noconfstd,axis=0)
wconfavr_avr=np.mean(wconfavr,axis=0)
noconfavr_avr=np.mean(noconfavr,axis=0)

In [ ]:
# import numpy as np
# import pandas as pd
# import plotly.express as px
# batch1_Noconfsigtable=pd.read_pickle('/home/ubuntu/Desktop/Thesis/overlap/allbatches/SigNumtableNoConf_batch2.pkl')
# batch1_Wconfsigtable=pd.read_pickle('/home/ubuntu/Desktop/Thesis/overlap/allbatches/SigNumtableWConf_batch2.pkl')

# # Store matched rows properly
# matches_batch1 = []  

# # Loop through rows of tenselectedsub
# for _, row in batch1_Noconfsigtable.iterrows():
#     subj = row['subject']
    
#     # Filter rows from filtered_tableNo matching the current row's subject
#     matched_rows = filtered_tableNo[filtered_tableNo['subject'] == subj]
    
#     matches_batch1.append(matched_rows)

# # Concatenate all matched rows
# Noconf_batch1 = pd.concat(matches_batch1, ignore_index=True)

# # Repeat for filtered_tableW
# matche_batch1 = []  

# for _, row in batch1_Wconfsigtable.iterrows():
#     subj = row['subject']
    
#     matched_rows = filtered_tableW[filtered_tableW['subject'] == subj]
    
#     matche_batch1.append(matched_rows)

# Wconf_batch1 = pd.concat(matche_batch1, ignore_index=True)

# # Exclude specific subjects
# filtered_table = Noconf_batch1[
#     (Noconf_batch1['subject'] != 'sub-140041') & 
#     (Noconf_batch1['subject'] != 'sub-100898')
# ]

# filtered_tableWconf = Wconf_batch1[
#     (Wconf_batch1['subject'] != 'sub-140041') & 
#     (Wconf_batch1['subject'] != 'sub-100898')
# ]

# # Compute statistics
# wconfsig, noconfsig = stack_sub(filtered_tableWconf, filtered_table, 'significant_digits_matrix')


# # Compute mean difference
# diff = noconfsig - wconfsig
# average_diff =np.mean(diff,axis=0)
# # Remove NaN or -inf values before computing mean
# cleaned_diff = average_diff[np.isfinite(average_diff)]
# print(np.mean(cleaned_diff))# Plot heatmap

# # Plot heatmap
# fig = px.imshow(
#     average_diff, color_continuous_scale='plasma',
#     title='Average Difference in Significant Digits (Without - With Confounds)'
# )
# fig.update_layout(coloraxis_colorbar_title='Difference')
# fig.show()


In [ ]:
import plotly.express as px

# Compute the difference
diff= noconfsig - wconfsig
average_diff =np.mean(diff,axis=0)
# Remove NaN or -inf values before computing mean
cleaned_diff = average_diff[np.isfinite(average_diff)]
print(np.mean(cleaned_diff))# Plot heatmap
fig = px.imshow(average_diff, color_continuous_scale='plasma',
                title='Average Difference in Significant Digits (Without - with Confounds)')
fig.update_layout(coloraxis_colorbar_title='Difference')
fig.show()

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

# Avoid log(0) errors
log_wconfstd_avr = np.log10(wconfstd_avr)  
log_noconfstd_avr = np.log10(noconfstd_avr)

avr1 =round(np.nanmean(wconfsig_avr[~np.isinf(wconfsig_avr)]),3)
avr2 = round(np.nanmean(wconfsig_avr[~np.isinf(noconfsig_avr)]),3)
avr3 = round(np.nanmean(wconfsig_avr[~np.isinf(log_wconfstd_avr)]),3)
avr4 = round(np.nanmean(wconfsig_avr[~np.isinf(log_noconfstd_avr)]),3)
avr5 = round(np.nanmean(wconfsig_avr[~np.isinf(wconfavr_avr)]),3)
avr6 = round(np.nanmean(wconfsig_avr[~np.isinf(noconfavr_avr)]),3)
# List of matrices, titles, shared color axes, and color scales
matrices = [
    (wconfsig_avr, f"With Confounds Significant Digits, with avrage values of= {avr1}", "coloraxis1", "RdYlGn"),
    (noconfsig_avr, f"No Confounds Significant Digits, with avrage values of= {avr2}", "coloraxis1", "RdYlGn"),
    (log_wconfstd_avr, f"With Confounds log(Standard Deviation), with avrage values of= {avr3}", "coloraxis2", "RdYlGn_r"),
    (log_noconfstd_avr, f"No Confounds log(Standard Deviation), with avrage values of= {avr4}", "coloraxis2", "RdYlGn_r"),
    (wconfavr_avr, f"With Confounds Average, with avrage values of= {avr5}", "coloraxis3", "brBG"),
    (noconfavr_avr, f"No Confounds Average, with avrage values of= {avr6}", "coloraxis3", "brBG")
]

# Create a subplot grid with 3 rows and 2 columns
fig = make_subplots(
    rows=3, cols=2,
    subplot_titles=[title for _, title, _, _ in matrices],
    shared_xaxes=True, shared_yaxes=True,
    vertical_spacing=0.15,  # Less space between rows
    horizontal_spacing=0.08  # Less space between columns
)

# Add each matrix as a heatmap to the appropriate subplot
for i, (matrix, _, coloraxis, colorscale) in enumerate(matrices):
    row = i // 2 + 1  
    col = i % 2 + 1   

    fig.add_trace(
        go.Heatmap(
            z=np.flipud(matrix),  
            coloraxis=coloraxis
        ),
        row=row, col=col
    )

# Configure the shared color axes for pairs
fig.update_layout(
    title="Heatmaps of Average Across Subjects (Confounds vs. No Confounds)",
    coloraxis1=dict(colorscale='RdYlGn',         colorbar=dict(
            title="Significant Digits",
            ticks="outside",
            thickness=15,
            len=0.25,  # Adjust length to fit the subplot height
            x=1.18,   # Position to the right of the plot
            y=1.0,    # Align vertically at the top
            yanchor="top"  # Anchor to the top
        ),    
        cmin=-0.5,
        cmax=3.5
        ),
    coloraxis2=dict(colorscale='RdYlGn_r',        colorbar=dict(
            title="log(Standard Deviation)",
            ticks="outside",
            thickness=15,
            len=0.25,
            x=1.18,   # Slightly further right than coloraxis1
            y=0.65,
            yanchor="top"
        ),  
        cmid=-2.5,
        cmin=-5,
        cmax=-0.5),
    coloraxis3=dict(colorscale='brBG',         colorbar=dict(
            title="Mean",
            ticks="outside",
            thickness=15,
            len=0.25,
            x=1.18,   # Further right for better spacing
            y=0.28,
            yanchor="top"
        ),   
        cmid=0,
        cmin=-1,
        cmax=1),
    height=1400,  
    width=1100,   
    plot_bgcolor="white",
    margin=dict(l=60, r=160, t=80, b=60)  # Adjust margins for better spacing
)

# Update subplot titles for better readability
for annotation in fig['layout']['annotations']:
    annotation['font'] = dict(size=12)

# Show the figure
fig.show()


In [ ]:
print(np.mean(wconfavr_avr))
print(np.mean(noconfavr_avr))

In [ ]:
import numpy as np
# Compute ECDF for each sample
def ecdf(data):
    x = np.sort(data)  # Sort data
    y = np.arange(1, len(x) + 1) / len(x)  # Compute cumulative probability
    return x, y
# Get upper triangle indices (excluding diagonal)
triu_indices = np.triu_indices(100, k=1)  # (row_idx, col_idx)

# Efficient extraction into NumPy array
subj_valwconfsig_matrix = wconfsig[:, triu_indices[0], triu_indices[1]].T  # Shape: (4950, 152)
subj_valNoconfsig_matrix = noconfsig[:, triu_indices[0], triu_indices[1]].T  # Shape: (4950, 152)
print(subj_valwconfsig_matrix.shape)
print(wconfsig.shape)

In [ ]:
# import numpy as np
# import plotly.graph_objects as go

# # Compute ECDF for a given data array
# def ecdf(data):
#     x = np.sort(data)  # Sort data
#     y = np.arange(1, len(x) + 1) / len(x)  # Compute cumulative probability
#     return x, y

# # Create a Plotly figure
# fig = go.Figure()

# # Add all ECDFs with transparency
# for idx in range(4949):
#     # Get ECDF for both conditions
#     x_wconf, y_wconf = ecdf(subj_valwconfsig_matrix[idx])
#     x_noconf, y_noconf = ecdf(subj_valNoconfsig_matrix[idx])
    
#     # Add traces with very low opacity for individual lines
#     fig.add_trace(go.Scatter(
#         x=x_wconf, 
#         y=y_wconf, 
#         mode='lines', 
#         showlegend=False,
#         line=dict(color='rgba(31, 119, 180, 0.02)'),  # Blue with very low opacity
#         hoverinfo='skip'
#     ))
    
#     fig.add_trace(go.Scatter(
#         x=x_noconf, 
#         y=y_noconf, 
#         mode='lines', 
#         showlegend=False,
#         line=dict(color='rgba(255, 127, 14, 0.02)'),  # Orange with very low opacity
#         hoverinfo='skip'
#     ))

# # Add two traces for the legend
# fig.add_trace(go.Scatter(
#     x=[None], y=[None],
#     mode='lines',
#     name='With Confound',
#     line=dict(color='rgb(31, 119, 180)', width=2)
# ))

# fig.add_trace(go.Scatter(
#     x=[None], y=[None],
#     mode='lines',
#     name='No Confound',
#     line=dict(color='rgb(255, 127, 14)', width=2)
# ))

# # Update layout for better readability
# fig.update_layout(
#     title="ECDF of Significant digits  Values across all 4949 Regions pairs",
#     xaxis_title=" Significant digits Value",
#     yaxis_title="Cumulative Probability",
#     legend_title="Condition",
#     template="plotly_white"
# )

# # Show the figure
# fig.show()





In [ ]:
import numpy as np
import plotly.graph_objects as go

# Compute ECDF for a given data array
def ecdf(data):
    x = np.sort(data)  # Sort data
    y = np.arange(1, len(x) + 1) / len(x)  # Compute cumulative probability
    return x, y

# Store x-values where ECDF crosses y=0.5
half_max_valuesw = []
half_max_valuesno=[]
# Iterate through all regions
for idx in range(4949):
    # Get ECDF for both conditions
    x_wconf, y_wconf = ecdf(subj_valwconfsig_matrix[idx])
    x_noconf, y_noconf = ecdf(subj_valNoconfsig_matrix[idx])
    
    # Find x-values where y is closest to 0.5
    x_50_wconf = x_wconf[np.argmin(np.abs(y_wconf - 0.5))]
    x_50_noconf = x_noconf[np.argmin(np.abs(y_noconf - 0.5))]
    
    # Store the difference between the two conditions at y=0.5
    half_max_valuesw.append( x_50_wconf)
    half_max_valuesno.append(x_50_noconf)

# Convert to a NumPy array for easier operations
half_max_valuesw = np.array(half_max_valuesw)
half_max_valuesno = np.array(half_max_valuesno)
# print(half_max_valuesno)
# Find the best and worst regions based on max difference
idxw_max,idxw_min=np.argmax(half_max_valuesw),np.argmin(half_max_valuesw)
idxN_max,idxN_min=np.argmax(half_max_valuesno),np.argmin(half_max_valuesno)
# print(idxN_max)
maxdifw = half_max_valuesw[idxw_max] - half_max_valuesw[idxw_min]
maxdifno = half_max_valuesno[idxN_max] - half_max_valuesno[idxN_min]

print(f'Max difference for with confound is = {maxdifw},for the best region: {idxw_max}, and the worse region: {idxw_min}')
print(f'Max difference for without confound is = {maxdifno},for the best region: {idxN_max}, and the worse region: {idxN_min}')

# # Print results
# print(f"Best Region (Max x at y=0.5): {region_idx_best}, Value: {half_max_values[region_idx_best, 1]}")
# print(f"Worst Region (Min x at y=0.5): {region_idx_worst}, Value: {half_max_values[region_idx_worst, 1]}")
# print(f"Maximum Difference in x at y=0.5: {max_difference}")
# Max difference for with confound is = 0.23861758665730415,for the best region: 4, and the worse region: 2
# Max difference for without confound is = 0.38311376159520716,for the best region: 4, and the worse region: 2

In [ ]:
import numpy as np
import plotly.graph_objects as go
import plotly.express as px

# Compute ECDF for a given data array
def ecdf(data):
    x = np.sort(data)  # Sort data
    y = np.arange(1, len(x) + 1) / len(x)  # Compute cumulative probability
    return x, y

# Choose a subset to plot (e.g., first 10 ECDFs)
num_to_plot = 10  # Adjust as needed
selected_indices = [3724,2584,3550]

# Get a list of colors from Plotly's qualitative palette
colors = px.colors.qualitative.Plotly

# Create a Plotly figure
fig = go.Figure()

# Add ECDF traces for selected elements
for i, idx in enumerate(selected_indices):
    color = colors[i % len(colors)]  # Cycle through colors
    x_wconf, y_wconf = ecdf(subj_valwconfsig_matrix[idx])
    x_noconf, y_noconf = ecdf(subj_valNoconfsig_matrix[idx])
    
    fig.add_trace(go.Scatter(x=x_wconf, y=y_wconf, mode='lines', name=f'With Confound {idx}', line=dict(color=color)))
    fig.add_trace(go.Scatter(x=x_noconf, y=y_noconf, mode='lines', name=f'No Confound {idx}', line=dict(color=color, dash='dash')))

# Update layout for better readability
fig.update_layout(
    title="ECDF of Significant digits  Values across all 4949 Regions pairs",
    xaxis_title=" Significant digits Value",
    yaxis_title="Cumulative Probability",
    legend_title="Legend",
    template="plotly_white"
)

# Show the figure
fig.show()


In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

# List of matrices and their titles
matrices = [
    (ressig_greater, "Rejected null hypothesis for alternative= greater for significant digit"),
    (ressig_twoside, "Rejected null hypothesis for alternative= two side for significant digit")
]

# Create a subplot grid with 1 row and 2 columns
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=[title for _, title in matrices],
    shared_xaxes=True, shared_yaxes=True,
    vertical_spacing=0.1,  # Adjust spacing between rows
    horizontal_spacing=0.1  # Adjust spacing between columns
)

# Add each matrix as a heatmap to the appropriate subplot
for i, (matrix, _) in enumerate(matrices):
    row = 1  # Only one row
    col = i + 1  # Columns are sequential (1-based index)

    # Add heatmap to the subplot
    fig.add_trace(
        go.Heatmap(
            z=np.flipud(matrix),  # Flip matrix vertically if needed
            coloraxis="coloraxis"  # All share the same color axis
        ),
        row=row, col=col
    )

# Configure the shared color axis
fig.update_layout(
    title="Heatmaps of KS and P-Value with Bonferroni correction (conf_values, non_conf_values, alternative=alt) for significant digits",
    coloraxis=dict(
        colorscale=[[0, 'black'], [1, 'white']],  # Black for 0, white for 1
        colorbar=dict(
            title="Decision",
            tickvals=[0, 1],   # Only show values 0 and 1
            ticktext=["Not Rejected", "Rejected"],  # Custom labels
            ticks="outside",
            thickness=15,
            len=0.7,  # Adjust the length of the colorbar
            x=1.1,    # Position to the right of the plots
            y=0.5,    # Center vertically
            yanchor="middle"
        )
    ),
    height=600,  # Adjust height for better visualization
    width=1100   # Adjust width for better visualization
)

# Update subplot titles for better readability
for annotation in fig['layout']['annotations']:
    annotation['font'] = dict(size=10)

# Show the figure
fig.show()


In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

# List of matrices and their titles
matrices = [
    (resstd_less, "Rejected null hypothesis for alternative= less for standard deviation"),
    (resstd_twoside, "Rejected null hypothesis for alternative= two side for standard deviation")
]

# Create a subplot grid with 1 row and 2 columns
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=[title for _, title in matrices],
    shared_xaxes=True, shared_yaxes=True,
    vertical_spacing=0.1,  # Adjust spacing between rows
    horizontal_spacing=0.1  # Adjust spacing between columns
)

# Add each matrix as a heatmap to the appropriate subplot
for i, (matrix, _) in enumerate(matrices):
    row = 1  # Only one row
    col = i + 1  # Columns are sequential (1-based index)

    # Add heatmap to the subplot
    fig.add_trace(
        go.Heatmap(
            z=np.flipud(matrix),  # Flip matrix vertically if needed
            coloraxis="coloraxis"  # All share the same color axis
        ),
        row=row, col=col
    )

# Configure the shared color axis

fig.update_layout(
    title="Heatmaps of KS and P-Value with Bonferroni correction (conf_values, non_conf_values, alternative=alt) for standard deviation",
    coloraxis=dict(
        colorscale=[[0, 'black'], [1, 'white']],  # Black for 0, white for 1
        colorbar=dict(
            title="Decision",
            tickvals=[0, 1],   # Only show values 0 and 1
            ticktext=["Not Rejected", "Rejected"],  # Custom labels
            ticks="outside",
            thickness=15,
            len=0.7,  # Adjust the length of the colorbar
            x=1.1,    # Position to the right of the plots
            y=0.5,    # Center vertically
            yanchor="middle"
        )
    ),
    height=600,  # Adjust height for better visualization
    width=1100   # Adjust width for better visualization
)

# Update subplot titles for better readability
for annotation in fig['layout']['annotations']:
    annotation['font'] = dict(size=10)

# Show the figure
fig.show()


In [ ]:
dfW=pd.read_pickle('/home/ubuntu/Desktop/Thesis/overlap/allbatches/Result_tableWConf.pkl')
dfN=pd.read_pickle('/home/ubuntu/Desktop/Thesis/overlap/allbatches/Result_tableNoConf.pkl')

In [ ]:
dfhc_Noconf_num=pd.read_pickle('/home/ubuntu/Desktop/Thesis/overlap/allbatches/NumRegionalGmetric_NoConf_batchhc.pkl')
dfhc_Noconf_anat=pd.read_pickle('/home/ubuntu/Desktop/Thesis/overlap/allbatches/AnatRegionalGmetric_NoConf_batchhc.pkl')
dfhc_Wconf_num=pd.read_pickle('/home/ubuntu/Desktop/Thesis/overlap/allbatches/NumRegionalGmetric_WConf_batchhc.pkl')
dfhc_Wconf_anat=pd.read_pickle('/home/ubuntu/Desktop/Thesis/overlap/allbatches/AnatRegionalGmetric_WConf_batchhc.pkl')


In [ ]:
def makestdevNum_metric(df):
    # Initialize DataFrame
    # Ensure columns are explicitly set to object dtype
    columns_to_set_as_object = [
        'degree_(numericalVar)', 'betweeness_(numericalVar)', 
        'clusteringcoef_(numericalVar)', 'eigenvec_(numericalVar)', 
        'smallworldness(numericalVar)', 'avg_shortestPathLength(numericalVar)'
    ]

    # Initialize DataFrame and set appropriate dtypes
    avrofStd_WithinSubject = pd.DataFrame(
        columns=columns_to_set_as_object + ['subject', 'session', 'acquisition']
    )

    avrofStd_WithinSubject = avrofStd_WithinSubject.astype({col: 'object' for col in columns_to_set_as_object})

    # Get unique subjects from the Results_table
    subjects = np.unique(df['subject'])

    i = 0
    # Loop over each subject
    for subj in subjects:
        filtered_df = df[df['subject'] == subj]

        # Loop over each session and acquisition
        for session in np.unique(filtered_df['session']):
            for acquisition in np.unique(filtered_df['acquisition']):
                if acquisition not in ['acq-RLsplit1', 'acq-LRsplit1']:
                    # Initialize lists to collect values for each repetition
                    mat_degree_centrality, mat_betweeness_centrality, mat_eigen_centrality, mat_clustering_coef = [], [], [], []
                    std_degree_centrality, std_betweeness_centrality, std_eigen_centrality, std_clustering_coef,std_smallworldness, std_avg_shortestPathLength  = [], [], [], [],[],[]
                    smallworldness, avg_shortestPathLength = [], []

                    # Loop over each repetition to collect centrality metrics
                    for rep in range(1, 11):
                        filtered_rows = filtered_df[
                            (filtered_df['session'] == session) &
                            (filtered_df['acquisition'] == acquisition) &
                            (filtered_df['repetition'] == f'rep-{rep}')
                        ]
                        if filtered_rows.empty:
                            continue

                        # Extract centrality measures and clustering coefficients
                        degree_values = list(filtered_rows['degree_centralities'].values[0].values())
                        betweeness_values = list(filtered_rows['betweenness_centralities'].values[0].values())
                        eigen_values = list(filtered_rows['eigenvector_centralities'].values[0].values())
                        clustering_values = list(filtered_rows['clustering_coefficients'].values[0].values())
                        
                        # Append values for each repetition
                        mat_degree_centrality.append(degree_values)
                        mat_betweeness_centrality.append(betweeness_values)
                        mat_eigen_centrality.append(eigen_values)
                        mat_clustering_coef.append(clustering_values)
                        smallworldness.append(filtered_rows['small_worldness'].values[0])
                        avg_shortestPathLength.append(filtered_rows['avg_shortest_path_length'].values[0])
                    # Calculate the standard deviation of the centrality metrics across repetitions
                    if len(mat_degree_centrality)==0:
                        continue
                    std_degree_centrality.append(np.std(mat_degree_centrality, axis=0))
                    std_betweeness_centrality.append(np.std(mat_betweeness_centrality, axis=0))
                    std_eigen_centrality.append(np.std(mat_eigen_centrality, axis=0))
                    std_clustering_coef.append(np.std(mat_clustering_coef, axis=0))
                    std_smallworldness=np.std(smallworldness,axis=0)
                    std_avg_shortestPathLength =np.std(avg_shortestPathLength, axis=0)
                    # Store the other metrics (assuming these are scalar values)
                    # Collect the row as a dictionary
                    row_data = {
                        'degree_(numericalVar)': std_degree_centrality,
                        'betweeness_(numericalVar)': std_betweeness_centrality,
                        'clusteringcoef_(numericalVar)': std_clustering_coef,
                        'eigenvec_(numericalVar)': std_eigen_centrality,
                        'smallworldness(numericalVar)':  std_smallworldness,
                        'avg_shortestPathLength(numericalVar)':std_avg_shortestPathLength,
                        'subject': subj,
                        'session': session,
                        'acquisition': acquisition
                    }

                    # Assign the row to the DataFrame
                    avrofStd_WithinSubject.loc[i] = row_data

                    i += 1
    return avrofStd_WithinSubject

In [ ]:
def makestedvAnat_metric(df):
    # Initialize DataFrame
    # Ensure columns are explicitly set to object dtype
    columns_to_set_as_object = [
        'degree_(AnatomicalVar)', 'betweeness_(AnatomicalVar)', 
        'clusteringcoef_(AnatomicalVar)', 'eigenvec_(AnatomicalVar)', 
        'smallworldness(AnatomicalVar)', 'avg_shortestPathLength(AnatomicalVar)'
    ]

    # Initialize DataFrame and set appropriate dtypes
    avrofStd_BetweenSubject = pd.DataFrame(
        columns=columns_to_set_as_object + ['iteration', 'session', 'acquisition']
    )

    avrofStd_BetweenSubject = avrofStd_BetweenSubject.astype({col: 'object' for col in columns_to_set_as_object})

    # Get unique subjects from the Results_table
    subjects = np.unique(df['subject'])
    # Initialize a list to hold data entries for the final DataFrame
    subject_data = []
    i=0
    # Loop over each repetition
    for rep in range(1, 11):
        # Filter DataFrame for the current repetition and drop unnecessary columns
        filtered_df = df[df['repetition'] == f'rep-{rep}']

        # Loop over each session
        for session in np.unique(filtered_df['session']):
            # Loop over each acquisition type
            for acquisition in np.unique(filtered_df['acquisition']):
                # Skip unwanted acquisitions
                if acquisition in ['acq-RLsplit1', 'acq-LRsplit1']:
                    continue
                
                # Initialize lists to store results for each iteration across subjects
                acrosSub_mat_degree_centrality = []
                acrosSub_mat_betweeness_centrality = []
                acrosSub_mat_eigen_centrality = []
                acrosSub_mat_clustering_coef = []
                acrosSub_std_degree_centrality = []
                acrosSub_std_betweeness_centrality = []
                acrosSub_std_eigen_centrality = []
                acrosSub_std_clustering_coef = []
                acrosSub_std_smallworld=[]
                acrosSub_std_avrgShortestPathLength=[]
                acrosSub_smallworld=[]
                acrosSub_avrgShortestPathLength=[]
                
                # Loop over each subject
                for subj in subjects:
                    # Filter rows for the specific subject, session, and acquisition
                    filtered_rows = filtered_df[
                        (filtered_df['session'] == session) &
                        (filtered_df['acquisition'] == acquisition) &
                        (filtered_df['subject'] == subj)
                    ]

                    if filtered_rows.empty:
                        continue
                    degree_values = list(filtered_rows['degree_centralities'].values[0].values())
                    betweeness_values = list(filtered_rows['betweenness_centralities'].values[0].values())
                    eigen_values = list(filtered_rows['eigenvector_centralities'].values[0].values())
                    clustering_values = list(filtered_rows['clustering_coefficients'].values[0].values())

                    # Append averages for each subject
                    acrosSub_mat_degree_centrality.append(degree_values)
                    acrosSub_mat_betweeness_centrality.append(betweeness_values)
                    acrosSub_mat_eigen_centrality.append(eigen_values)
                    acrosSub_mat_clustering_coef.append(clustering_values)
                    acrosSub_smallworld.append(filtered_rows['small_worldness'].values[0])
                    acrosSub_avrgShortestPathLength.append(filtered_rows['avg_shortest_path_length'].values[0])

                if len(acrosSub_mat_degree_centrality)<2:
                    continue         
                # Append standard deviations for each subject
                acrosSub_std_degree_centrality.append(np.std(acrosSub_mat_degree_centrality,axis=0))
                acrosSub_std_betweeness_centrality.append(np.std(acrosSub_mat_betweeness_centrality,axis=0))
                acrosSub_std_eigen_centrality.append(np.std(acrosSub_mat_eigen_centrality,axis=0))
                acrosSub_std_clustering_coef.append(np.std(acrosSub_mat_clustering_coef,axis=0))
                acrosSub_std_smallworld=np.std(acrosSub_smallworld,axis=0)
                acrosSub_std_avrgShortestPathLength=np.std(acrosSub_avrgShortestPathLength,axis=0)

                # Collect the row as a dictionary
                row_data = {
                    'degree_(AnatomicalVar)': acrosSub_std_degree_centrality,
                    'betweeness_(AnatomicalVar)': acrosSub_std_betweeness_centrality,
                    'clusteringcoef_(AnatomicalVar)': acrosSub_std_eigen_centrality,
                    'eigenvec_(AnatomicalVar)': acrosSub_std_clustering_coef,
                    'smallworldness(AnatomicalVar)':acrosSub_std_smallworld,
                    'avg_shortestPathLength(AnatomicalVar)':acrosSub_std_avrgShortestPathLength,
                    'iteration': f'iter_{rep}',
                    'session': session,
                    'acquisition': acquisition
                }

                # Assign the row to the DataFrame
                avrofStd_BetweenSubject.loc[i] = row_data
                i=i+1

    return avrofStd_BetweenSubject

In [ ]:
df_Noconf_num=makestdevNum_metric(dfN)
df_Noconf_anat=makestedvAnat_metric(dfN)
df_Wconf_num=makestdevNum_metric(dfW)
df_Wconf_anat=makestedvAnat_metric(dfW)

In [ ]:
from scipy.stats import ks_2samp
from statsmodels.stats.multitest import multipletests
Noconfmet=Results_tableNoConf  #[Results_tableNoConf['acquisition']=='acq-RL']
Wconfmet=Results_tableWConf    #[Results_tableWConf['acquisition']=='acq-RL']

# Define the graph metric columns (which contain 100 values each)
graph_metrics = [
    'degree_centralities', 'betweenness_centralities', 
    'eigenvector_centralities', 'clustering_coefficients'
]
Noconfmet_selected = pd.DataFrame()
Wconfmet_selected = pd.DataFrame()


for metric in graph_metrics:
    Noconfmet_selected[metric] = Noconfmet[metric].apply(lambda d: np.array(list(d.values())).reshape(1, -1))
    Wconfmet_selected[metric] = Wconfmet[metric].apply(lambda d: np.array(list(d.values())).reshape(1, -1))

wconfdeg,noconfdeg=stack_sub(Wconfmet_selected,Noconfmet_selected,'degree_centralities')#mode can be : std , mean , significant_digits_matrix
wconfbet,noconfbet=stack_sub(Wconfmet_selected,Noconfmet_selected,'betweenness_centralities')#mode can be : std , mean , significant_digits_matrix
wconfclcf,noconfclcf=stack_sub(Wconfmet_selected,Noconfmet_selected,'clustering_coefficients')#mode can be : std , mean , significant_digits_matrix
wconfeigen,noconfeigen=stack_sub(Wconfmet_selected,Noconfmet_selected,'eigenvector_centralities')#mode can be : std , mean , significant_digits_matrix
# wconfmet.shape
_,__,pvalcordegtwoside,resdeg_twoside=ksTestWithCorrection(wconfdeg,noconfdeg,'two-side','Bonferroni') #FDR,Bonferroni
_,__,pvalcordeggreater,resdeg_greater=ksTestWithCorrection(wconfdeg,noconfdeg,'greater','Bonferroni') #FDR,Bonferroni
_,__,pvalcordegless,resdeg_less=ksTestWithCorrection(wconfdeg,noconfdeg,'less','Bonferroni') #FDR,Bonferroni
_,__,pvalcorbettwoside,resbet_twoside=ksTestWithCorrection(wconfbet,noconfbet,'two-side','Bonferroni') #FDR,Bonferroni
_,__,pvalcorbetgreater,resbet_greater=ksTestWithCorrection(wconfbet,noconfbet,'greater','Bonferroni') #FDR,Bonferroni
_,__,pvalcorbetless,resbet_less=ksTestWithCorrection(wconfbet,noconfbet,'less','Bonferroni') #FDR,Bonferroni
_,__,pvalcordclcftwoside,resclcf_twoside=ksTestWithCorrection(wconfclcf,noconfclcf,'two-side','Bonferroni') #FDR,Bonferroni
_,__,pvalcordclcfgreater,resclcf_greater=ksTestWithCorrection(wconfclcf,noconfclcf,'greater','Bonferroni') #FDR,Bonferroni
_,__,pvalcorclcfless,resclcf_less=ksTestWithCorrection(wconfclcf,noconfclcf,'less','Bonferroni') #FDR,Bonferroni
_,__,pvalcoreigentwoside,reseigen_twoside=ksTestWithCorrection(wconfeigen,noconfeigen,'two-side','Bonferroni') #FDR,Bonferroni
_,__,pvalcoreigengreater,reseigen_greater=ksTestWithCorrection(wconfeigen,noconfeigen,'greater','Bonferroni') #FDR,Bonferroni
_,__,pvalcoreigenless,reseigen_less=ksTestWithCorrection(wconfeigen,noconfeigen,'less','Bonferroni') #FDR,Bonferroni
##############Global metric
res1smWt,res2smWt=ks_2samp(Wconfmet['small_worldness'],Noconfmet['small_worldness'],'two-side') #FDR,Bonferroni
res1smWg,res2smWg=ks_2samp(Wconfmet['small_worldness'],Noconfmet['small_worldness'],'greater') #FDR,Bonferroni
res1smWl,res2smWl=ks_2samp(Wconfmet['small_worldness'],Noconfmet['small_worldness'],'less') #FDR,Bonferroni
res1pathLt,res2pathLt=ks_2samp(Wconfmet['avg_shortest_path_length'],Noconfmet['avg_shortest_path_length'],'two-side') #FDR,Bonferroni
res1pathLg,res2pathLg=ks_2samp(Wconfmet['avg_shortest_path_length'],Noconfmet['avg_shortest_path_length'],'greater') #FDR,Bonferroni
res1pathLl,res2pathLl=ks_2samp(Wconfmet['avg_shortest_path_length'],Noconfmet['avg_shortest_path_length'],'less') #FDR,Bonferroni
ressmWt, pval_correctedsmWt, _, _ = multipletests(res2smWt, method='bonferroni')
ressmWg, pval_correctedsmWg, _, _ = multipletests(res2smWg, method='bonferroni')
ressmWl, pval_correctedsmWl, _, _ = multipletests(res2smWl, method='bonferroni')
respathLt, pval_correctedpathLt, _, _ = multipletests(res2pathLt, method='bonferroni')
respathLg, pval_correctedpathLg, _, _ = multipletests(res2pathLg, method='bonferroni')
respathLl, pval_correctedpathLl, _, _ = multipletests(res2pathLl, method='bonferroni')
print(ressmWt,ressmWg,ressmWl)
print(respathLt,respathLg,respathLl)
# pvalcordeggreater

In [ ]:
# from scipy.stats import ks_2samp
# from statsmodels.stats.multitest import multipletests

# columns_to_convert = ['degree_centralities', 'betweenness_centralities',	'eigenvector_centralities','clustering_coefficients'] 
# # 
# # Convert specific columns from list to numpy array
# for column in columns_to_convert:
#     if column in Noconfmet.columns:
#         Noconfmet[column] = Noconfmet[column].apply(np.array)
#         Wconfmet[column] = Wconfmet[column].apply(np.array)
# # For Wconfmet DataFrame
# Wconfmet_selected = Wconfmet.loc[:, columns_to_convert]
# # For Noconfmet DataFrame
# Noconfmet_selected = Noconfmet.loc[:, columns_to_convert]

# # Noconfmet['degree_(numericalVar)'].iloc[0].shape[0]
# wconfdeg,noconfdeg=stack_sub(Wconfmet_selected,Noconfmet_selected,'degree_centralities')#mode can be : std , mean , significant_digits_matrix
# wconfbet,noconfbet=stack_sub(Wconfmet_selected,Noconfmet_selected,'betweenness_centralities')#mode can be : std , mean , significant_digits_matrix
# wconfclcf,noconfclcf=stack_sub(Wconfmet_selected,Noconfmet_selected,'clustering_coefficients')#mode can be : std , mean , significant_digits_matrix
# wconfeigen,noconfeigen=stack_sub(Wconfmet_selected,Noconfmet_selected,'eigenvector_centralities')#mode can be : std , mean , significant_digits_matrix
# print(wconfdeg.shape)
# # wconfmet.shape
# _,__,pvalcordegtwoside,resdeg_twoside=ksTestWithCorrection(wconfdeg,noconfdeg,'two-side','Bonferroni') #FDR,Bonferroni
# _,__,pvalcordeggreater,resdeg_greater=ksTestWithCorrection(wconfdeg,noconfdeg,'greater','Bonferroni') #FDR,Bonferroni
# _,__,pvalcordegless,resdeg_less=ksTestWithCorrection(wconfdeg,noconfdeg,'less','Bonferroni') #FDR,Bonferroni
# _,__,pvalcorbettwoside,resbet_twoside=ksTestWithCorrection(wconfbet,noconfbet,'two-side','Bonferroni') #FDR,Bonferroni
# _,__,pvalcorbetgreater,resbet_greater=ksTestWithCorrection(wconfbet,noconfbet,'greater','Bonferroni') #FDR,Bonferroni
# _,__,pvalcorbetless,resbet_less=ksTestWithCorrection(wconfbet,noconfbet,'less','Bonferroni') #FDR,Bonferroni
# _,__,pvalcordclcftwoside,resclcf_twoside=ksTestWithCorrection(wconfclcf,noconfclcf,'two-side','Bonferroni') #FDR,Bonferroni
# _,__,pvalcordclcfgreater,resclcf_greater=ksTestWithCorrection(wconfclcf,noconfclcf,'greater','Bonferroni') #FDR,Bonferroni
# _,__,pvalcorclcfless,resclcf_less=ksTestWithCorrection(wconfclcf,noconfclcf,'less','Bonferroni') #FDR,Bonferroni
# _,__,pvalcoreigentwoside,reseigen_twoside=ksTestWithCorrection(wconfeigen,noconfeigen,'two-side','Bonferroni') #FDR,Bonferroni
# _,__,pvalcoreigengreater,reseigen_greater=ksTestWithCorrection(wconfeigen,noconfeigen,'greater','Bonferroni') #FDR,Bonferroni
# _,__,pvalcoreigenless,reseigen_less=ksTestWithCorrection(wconfeigen,noconfeigen,'less','Bonferroni') #FDR,Bonferroni

# ##############Global metric
# res1smWt,res2smWt=ks_2samp(Wconfmet['small_worldness'],Noconfmet['small_worldness'],'two-side') #FDR,Bonferroni
# res1smWg,res2smWg=ks_2samp(Wconfmet['small_worldness'],Noconfmet['small_worldness'],'greater') #FDR,Bonferroni
# res1smWl,res2smWl=ks_2samp(Wconfmet['small_worldness'],Noconfmet['small_worldness'],'less') #FDR,Bonferroni
# res1pathLt,res2pathLt=ks_2samp(Wconfmet['avg_shortest_path_length'],Noconfmet['avg_shortest_path_length'],'two-side') #FDR,Bonferroni
# res1pathLg,res2pathLg=ks_2samp(Wconfmet['avg_shortest_path_length'],Noconfmet['avg_shortest_path_length'],'greater') #FDR,Bonferroni
# res1pathLl,res2pathLl=ks_2samp(Wconfmet['avg_shortest_path_length'],Noconfmet['aavg_shortest_path_length'],'less') #FDR,Bonferroni
# ressmWt, pval_correctedsmWt, _, _ = multipletests(res2smWt, method='bonferroni')
# ressmWg, pval_correctedsmWg, _, _ = multipletests(res2smWg, method='bonferroni')
# ressmWl, pval_correctedsmWl, _, _ = multipletests(res2smWl, method='bonferroni')
# respathLt, pval_correctedpathLt, _, _ = multipletests(res2pathLt, method='bonferroni')
# respathLg, pval_correctedpathLg, _, _ = multipletests(res2pathLg, method='bonferroni')
# respathLl, pval_correctedpathLl, _, _ = multipletests(res2pathLl, method='bonferroni')
# # print(ressmWt,ressmWg,ressmWl)
# print(respathLt,respathLg,respathLl)


In [ ]:
# # # Reshape arrays
# wconfdeg = wconfdeg.reshape(100, 144)
# noconfdeg = noconfdeg.reshape(100, 144)
# # wconfbet = wconfbet.reshape(100, 147)
# # noconfbet = noconfbet.reshape(100, 144)
# # wconfclcf = wconfclcf.reshape(100, 147)
# # noconfclcf = noconfclcf.reshape(100, 144)
# # wconfeigen = wconfeigen.reshape(100, 147)
# # noconfeigen = noconfeigen.reshape(100, 144)

# import numpy as np
# import plotly.graph_objects as go
# import plotly.express as px

# # Compute ECDF for a given data array
# def ecdf(data):
#     x = np.sort(data)  # Sort data
#     y = np.arange(1, len(x) + 1) / len(x)  # Compute cumulative probability
#     return x, y

# # Choose a subset to plot (e.g., first 10 ECDFs)
# num_to_plot = 100  # Adjust as needed
# selected_indices = np.linspace(0, 99, num_to_plot, dtype=int)  # Evenly spaced indices

# # Get a list of colors from Plotly's qualitative palette
# colors = px.colors.qualitative.Plotly

# # Create a Plotly figure
# fig = go.Figure()

# # Add ECDF traces for selected elements
# for i, idx in enumerate(selected_indices):
#     color = colors[i % len(colors)]  # Cycle through colors
#     # if idx==56:
#     #     print(wconfdeg[56])
#     x_wconf, y_wconf = ecdf(wconfdeg[idx])
#     x_noconf, y_noconf = ecdf(noconfdeg[idx])
    
#     fig.add_trace(go.Scatter(x=x_wconf, y=y_wconf, mode='lines', name=f'With Confound {idx}', line=dict(color=color)))
#     fig.add_trace(go.Scatter(x=x_noconf, y=y_noconf, mode='lines', name=f'No Confound {idx}', line=dict(color=color, dash='dash')))

# # Update layout for better readability
# fig.update_layout(
#     title="ECDF of Connectivity Values (Subset of 100 regions)",
#     xaxis_title="Connectivity Value",
#     yaxis_title="Cumulative Probability",
#     legend_title="Legend",
#     template="plotly_white"
# )

# # Show the figure
# fig.show()


In [ ]:
# import numpy as np
# import plotly.graph_objects as go
# from plotly.subplots import make_subplots
# # Reshape arrays
# wconfdeg = wconfdeg.reshape(100, 144)
# noconfdeg = noconfdeg.reshape(100, 144)
# wconfbet = wconfbet.reshape(100, 144)
# noconfbet = noconfbet.reshape(100, 144)
# wconfclcf = wconfclcf.reshape(100, 144)
# noconfclcf = noconfclcf.reshape(100, 144)
# wconfeigen = wconfeigen.reshape(100, 144)
# noconfeigen = noconfeigen.reshape(100, 144)
# # Compute ECDF for a given data array
# import numpy as np
# import plotly.graph_objects as go
# from plotly.subplots import make_subplots

# # Compute ECDF for a given data array
# def ecdf(data):
#     x = np.sort(data)  # Sort data
#     y = np.arange(1, len(x) + 1) / len(x)  # Compute cumulative probability
#     return x, y

# # Create a subplot figure with 2x2 grid
# fig = make_subplots(
#     rows=2, 
#     cols=2,
#     subplot_titles=[
#         "Degree Centrality ECDF", 
#         "Betweenness Centrality ECDF",
#         "Closeness Centrality ECDF",
#         "Eigenvector Centrality ECDF"
#     ],
#     vertical_spacing=0.12,
#     horizontal_spacing=0.08
# )

# # Define axis ranges for each metric
# x_ranges = {
#     'degree': [0,0.05], # np.max(np.concatenate([wconfdeg.flatten(), noconfdeg.flatten()]))],
#     'betweenness': [0,0.0001] ,#np.max(np.concatenate([wconfbet.flatten(), noconfbet.flatten()]))],
#     'closeness': [0,0.005 ], #np.max(np.concatenate([wconfclcf.flatten(), noconfclcf.flatten()]))],
#     'eigenvector': [0,0.005 ], #np.max(np.concatenate([wconfeigen.flatten(), noconfeigen.flatten()]))]
# }

# # Plot all regions with transparency for each metric
# # 1. Degree Centrality
# for i in range(100):
#     x_wconf, y_wconf = ecdf(wconfdeg[i])
#     x_noconf, y_noconf = ecdf(noconfdeg[i])
    
#     fig.add_trace(
#         go.Scatter(
#             x=x_wconf, 
#             y=y_wconf, 
#             mode='lines', 
#             showlegend=False,
#             line=dict(color='rgba(31, 119, 180, 0.1)'),  # Blue with low opacity
#             hoverinfo='skip'
#         ),
#         row=1, col=1
#     )
    
#     fig.add_trace(
#         go.Scatter(
#             x=x_noconf, 
#             y=y_noconf, 
#             mode='lines', 
#             showlegend=False,
#             line=dict(color='rgba(255, 127, 14, 0.1)'),  # Orange with low opacity
#             hoverinfo='skip'
#         ),
#         row=1, col=1
#     )

# # 2. Betweenness Centrality
# for i in range(100):
#     x_wconf, y_wconf = ecdf(wconfbet[i])
#     x_noconf, y_noconf = ecdf(noconfbet[i])
    
#     fig.add_trace(
#         go.Scatter(
#             x=x_wconf, 
#             y=y_wconf, 
#             mode='lines', 
#             showlegend=False,
#             line=dict(color='rgba(31, 119, 180, 0.1)'),  # Blue with low opacity
#             hoverinfo='skip'
#         ),
#         row=1, col=2
#     )
    
#     fig.add_trace(
#         go.Scatter(
#             x=x_noconf, 
#             y=y_noconf, 
#             mode='lines', 
#             showlegend=False,
#             line=dict(color='rgba(255, 127, 14, 0.1)'),  # Orange with low opacity
#             hoverinfo='skip'
#         ),
#         row=1, col=2
#     )

# # 3. Closeness Centrality
# for i in range(100):
#     x_wconf, y_wconf = ecdf(wconfclcf[i])
#     x_noconf, y_noconf = ecdf(noconfclcf[i])
    
#     fig.add_trace(
#         go.Scatter(
#             x=x_wconf, 
#             y=y_wconf, 
#             mode='lines', 
#             showlegend=False,
#             line=dict(color='rgba(31, 119, 180, 0.1)'),  # Blue with low opacity
#             hoverinfo='skip'
#         ),
#         row=2, col=1
#     )
    
#     fig.add_trace(
#         go.Scatter(
#             x=x_noconf, 
#             y=y_noconf, 
#             mode='lines', 
#             showlegend=False,
#             line=dict(color='rgba(255, 127, 14, 0.1)'),  # Orange with low opacity
#             hoverinfo='skip'
#         ),
#         row=2, col=1
#     )

# # 4. Eigenvector Centrality
# for i in range(100):
#     x_wconf, y_wconf = ecdf(wconfeigen[i])
#     x_noconf, y_noconf = ecdf(noconfeigen[i])
    
#     fig.add_trace(
#         go.Scatter(
#             x=x_wconf, 
#             y=y_wconf, 
#             mode='lines', 
#             showlegend=False,
#             line=dict(color='rgba(31, 119, 180, 0.1)'),  # Blue with low opacity
#             hoverinfo='skip'
#         ),
#         row=2, col=2
#     )
    
#     fig.add_trace(
#         go.Scatter(
#             x=x_noconf, 
#             y=y_noconf, 
#             mode='lines', 
#             showlegend=False,
#             line=dict(color='rgba(255, 127, 14, 0.1)'),  # Orange with low opacity
#             hoverinfo='skip'
#         ),
#         row=2, col=2
#     )

# # Add two traces for the legend (only need to add once for the whole figure)
# fig.add_trace(
#     go.Scatter(
#         x=[None], y=[None],
#         mode='lines',
#         name='With Confound',
#         line=dict(color='rgb(31, 119, 180)', width=2)
#     ),
#     row=1, col=1
# )

# fig.add_trace(
#     go.Scatter(
#         x=[None], y=[None],
#         mode='lines',
#         name='No Confound',
#         line=dict(color='rgb(255, 127, 14)', width=2)
#     ),
#     row=1, col=1
# )

# # Update x-axis ranges for each subplot
# fig.update_xaxes(range=x_ranges['degree'], title_text="Degree Centrality Value", row=1, col=1)
# fig.update_xaxes(range=x_ranges['betweenness'], title_text="Betweenness Centrality Value", row=1, col=2)
# fig.update_xaxes(range=x_ranges['closeness'], title_text="Clustering coefficient Value", row=2, col=1)
# fig.update_xaxes(range=x_ranges['eigenvector'], title_text="Eigenvector Centrality Value", row=2, col=2)

# # Update y-axis title for all subplots
# fig.update_yaxes(title_text="Cumulative Probability", row=1, col=1)
# fig.update_yaxes(title_text="Cumulative Probability", row=1, col=2)
# fig.update_yaxes(title_text="Cumulative Probability", row=2, col=1)
# fig.update_yaxes(title_text="Cumulative Probability", row=2, col=2)

# # Update layout for the entire figure
# fig.update_layout(
#     title="ECDF Comparison of Network Centrality Metrics (All 100 Regions)",
#     legend_title="Condition",
#     template="plotly_white",
#     height=1000,  # Adjust for better visualization
#     width=1800,  # Adjust for better visualization
#     legend=dict(
#         orientation="h",
#         yanchor="bottom",
#         y=1.02,
#         xanchor="center",
#         x=0.5
#     )
# )

# # Show the figure
# fig.show()

In [ ]:
# import plotly.graph_objects as go
# from plotly.subplots import make_subplots
# import numpy as np

# # List of matrices and their titles
# matrices = [
#     (np.expand_dims(resdeg_twoside, axis=0), "Rejected null hypothesis (two-sided) for Degree Centrality"),
#     (np.expand_dims(resdeg_greater, axis=0), "Rejected null hypothesis (greater) for Degree Centrality"),
#     (np.expand_dims(resdeg_less, axis=0), "Rejected null hypothesis (less) for Degree Centrality"),

#     (np.expand_dims(resbet_twoside, axis=0), "Rejected null hypothesis (two-sided) for Betweenness Centrality"),
#     (np.expand_dims(resbet_greater, axis=0), "Rejected null hypothesis (greater) for Betweenness Centrality"),
#     (np.expand_dims(resbet_less, axis=0), "Rejected null hypothesis (less) for Betweenness Centrality"),

#     (np.expand_dims(resclcf_twoside, axis=0), "Rejected null hypothesis (two-sided) for Clustering Coefficient"),
#     (np.expand_dims(resclcf_greater, axis=0), "Rejected null hypothesis (greater) for Clustering Coefficient"),
#     (np.expand_dims(resclcf_less, axis=0), "Rejected null hypothesis (less) for Clustering Coefficient"),

#     (np.expand_dims(reseigen_twoside, axis=0), "Rejected null hypothesis (two-sided) for Eigenvector Centrality"),
#     (np.expand_dims(reseigen_greater, axis=0), "Rejected null hypothesis (greater) for Eigenvector Centrality"),
#     (np.expand_dims(reseigen_less, axis=0), "Rejected null hypothesis (less) for Eigenvector Centrality")
# ]

# # Adjust subplot layout (4 rows, 3 columns)
# fig = make_subplots(
#     rows=4, cols=3,
#     subplot_titles=[title for _, title in matrices],
#     vertical_spacing=0.15, horizontal_spacing=0.1
# )

# # Discrete color map
# color_map = [[0, 'black'], [1, 'white']]

# # Add each matrix to the heatmap
# for i, (matrix, _) in enumerate(matrices):
#     row = i // 3 + 1
#     col = i % 3 + 1

#     fig.add_trace(
#         go.Heatmap(
#             z=matrix, 
#             coloraxis="coloraxis"
#         ),
#         row=row, col=col
#     )

#     # Update the axis to hide y-tick labels for each subplot
#     fig.update_yaxes(showticklabels=False, row=row, col=col)

# # Update layout with a shared color axis
# fig.update_layout(
#     title="KS Test Results with Bonferroni Correction for Graph Metrics",
#     coloraxis=dict(
#         colorscale=color_map,
#         colorbar=dict(
#             title="Decision",
#             tickvals=[0, 1],
#             ticktext=["Not Rejected: 0", "Rejected: 1"],
#             ticks="outside",
#             thickness=15,
#             len=0.7,
#             x=1.1,
#             y=0.5,
#             yanchor="middle"
#         )
#     ),
#     height=900,  # Increased height
#     width=1300   # Increased width
# )

# # Improve readability of subplot titles
# for annotation in fig['layout']['annotations']:
#     annotation['font'] = dict(size=11)

# fig.show()


In [ ]:
import numpy as np
import pandas as pd
def compute_average_num(df_Noconf_num,df_Wconf_num):
    # Assuming 'df' has already been extracted
    df = df_Noconf_num[(df_Noconf_num['acquisition'] == 'acq-RL') & 
                                (df_Noconf_num['session'] == '1')]

    # df = df.drop(columns=['session', 'acquisition', 'subject' # Remove the unnecessary column
    # ])


    cols=['degree_(numericalVar)',	'betweeness_(numericalVar)',	'clusteringcoef_(numericalVar)'	,'eigenvec_(numericalVar)'	,'smallworldness(numericalVar)','avg_shortestPathLength(numericalVar)']
    # Initialize a new DataFrame for the final stacked result
    stacked_row = {}

    # Loop through each column
    for col in cols:
        # If column contains matrices or arrays, stack them vertically
        stacked_row[col] = np.vstack(df[col].values)

    # Convert the dictionary to a DataFrame with one row
    stacked_df = pd.DataFrame([stacked_row])
    # Create a DataFrame to store the averaged values
    avrg_Noconf_num = pd.DataFrame({
        col: [np.mean(stacked_df.iloc[0][col], axis=0)] for col in ['degree_(numericalVar)',	'betweeness_(numericalVar)',	'clusteringcoef_(numericalVar)' , 'eigenvec_(numericalVar)']
    })
    # Add 'small_worldness' and 'avg_shortest_path_length' to avrg_Wconf_anat
    avrg_Noconf_num['smallworldness(numericalVar)'] =stacked_df['smallworldness(numericalVar)']
    avrg_Noconf_num['avg_shortestPathLength(numericalVar)'] = stacked_df['avg_shortestPathLength(numericalVar)']

    ##########################################################################################3
    # Assuming 'df' has already been extracted
    df1= df_Wconf_num[(df_Wconf_num['acquisition'] == 'acq-RL') & 
                                (df_Wconf_num['session'] == '1')]

    # df1 = df1.drop(columns=['session', 'acquisition', 'subject'])



    # Initialize a new DataFrame for the final stacked result
    stacked_row = {}

    # Loop through each column
    for col in cols:
        # If column contains matrices or arrays, stack them vertically
        stacked_row[col] = np.vstack(df1[col].values)

    # Convert the dictionary to a DataFrame with one row
    stacked_df1 = pd.DataFrame([stacked_row])
    # Create a DataFrame to store the averaged values
    avrg_Wconf_num = pd.DataFrame({
        col: [np.mean(stacked_df1.iloc[0][col], axis=0)] for col in ['degree_(numericalVar)',	'betweeness_(numericalVar)',	'clusteringcoef_(numericalVar)' , 'eigenvec_(numericalVar)']
    })
    # Add 'small_worldness' and 'avg_shortest_path_length' to avrg_Wconf_anat
    avrg_Wconf_num['smallworldness(numericalVar)'] =stacked_df1['smallworldness(numericalVar)']
    avrg_Wconf_num['avg_shortestPathLength(numericalVar)'] = stacked_df1['avg_shortestPathLength(numericalVar)']
    avrg_Wconf_num = avrg_Wconf_num.rename(columns={'degree_(numericalVar)': 'degree_(numericalVarW)', 'betweeness_(numericalVar)': 'betweeness_(numericalVarW)' , 'clusteringcoef_(numericalVar)':'clusteringcoef_(numericalVarW)'   , 'eigenvec_(numericalVar)':'eigenvec_(numericalVarW)'  ,'smallworldness(numericalVar)':'smallworldness(numericalVarW)','avg_shortestPathLength(numericalVar)':'avg_shortestPathLength(numericalVarW)'

	})
    return avrg_Noconf_num,avrg_Wconf_num


In [ ]:
import numpy as np
import pandas as pd
def compute_average_anat(df_Noconf_anat,df_Wconf_anat):
    # Assuming 'df' has already been extracted
    df = df_Noconf_anat[(df_Noconf_anat['acquisition'] == 'acq-RL') & 
                                (df_Noconf_anat['session'] == '1')&(df_Noconf_anat['iteration']=="iter_1")]

    # df = df.drop(columns=['session', 'acquisition', 'iteration'])

    cols=['degree_(AnatomicalVar)'	,'betweeness_(AnatomicalVar)',	'clusteringcoef_(AnatomicalVar)'	,'eigenvec_(AnatomicalVar)',	'smallworldness(AnatomicalVar)',	'avg_shortestPathLength(AnatomicalVar)']


    # Initialize a new DataFrame for the final stacked result
    stacked_row1= {}

    # Loop through each column
    for col in cols:
        # If column contains matrices or arrays, stack them vertically
        stacked_row1[col] = np.vstack(df[col].values)

    # Convert the dictionary to a DataFrame with one row
    stacked_df = pd.DataFrame([stacked_row1])
    print(stacked_df['degree_(AnatomicalVar)'].values[0].shape)

    # Create a DataFrame to store the averaged values
    avrg_Noconf_anat = pd.DataFrame({
        col: [np.mean(stacked_df.iloc[0][col], axis=0)] for col in ['degree_(AnatomicalVar)',	'betweeness_(AnatomicalVar)',	'clusteringcoef_(AnatomicalVar)' , 'eigenvec_(AnatomicalVar)']
    })
    # Add 'small_worldness' and 'avg_shortest_path_length' to avrg_Wconf_anat
    avrg_Noconf_anat['smallworldness(AnatomicalVar)'] =stacked_df['smallworldness(AnatomicalVar)']
    avrg_Noconf_anat['avg_shortestPathLength(AnatomicalVar)'] = stacked_df['avg_shortestPathLength(AnatomicalVar)']
    ###########################################################################################3
    # Assuming 'df' has already been extracted
    df1 = df_Wconf_anat[(df_Wconf_anat['acquisition'] == 'acq-RL') & 
                                (df_Wconf_anat['session'] == '1')& (df_Wconf_anat['iteration']=="iter_1")]

    # df1 = df1.drop(columns=['session', 'acquisition', 'iteration'])



    # Initialize a new DataFrame for the final stacked result
    stacked_row1= {}

    # Loop through each column
    for col in cols:
        # If column contains matrices or arrays, stack them vertically
        stacked_row1[col] = np.vstack(df1[col].values)

    # Convert the dictionary to a DataFrame with one row
    stacked_df1 = pd.DataFrame([stacked_row1])
    # Create a DataFrame to store the averaged values(df_Noconf_anat['iteration']=="iter_1")
    # Create a DataFrame to store the averaged values
    avrg_Wconf_anat = pd.DataFrame({
        col: [np.mean(stacked_df1.iloc[0][col], axis=0)] for col in ['degree_(AnatomicalVar)',	'betweeness_(AnatomicalVar)',	'clusteringcoef_(AnatomicalVar)' , 'eigenvec_(AnatomicalVar)']
    })
    # Add 'small_worldness' and 'avg_shortest_path_length' to avrg_Wconf_anat
    avrg_Wconf_anat['smallworldness(AnatomicalVar)'] =stacked_df1['smallworldness(AnatomicalVar)']
    avrg_Wconf_anat['avg_shortestPathLength(AnatomicalVar)'] = stacked_df1['avg_shortestPathLength(AnatomicalVar)']
    avrg_Wconf_anat = avrg_Wconf_anat.rename(columns={'degree_(AnatomicalVar)': 'degree_(AnatomicalVarW)', 'betweeness_(AnatomicalVar)': 'betweeness_(AnatomicalVarW)' , 'clusteringcoef_(AnatomicalVar)':'clusteringcoef_(AnatomicalVarW)'   , 'eigenvec_(AnatomicalVar)':'eigenvec_(AnatomicalVarW)','smallworldness(AnatomicalVar)':'smallworldness(AnatomicalVarW)','avg_shortestPathLength(AnatomicalVar)':'avg_shortestPathLength(AnatomicalVarW)'  })

    return avrg_Noconf_anat,avrg_Wconf_anat

avrg_Noconf_num,avrg_Wconf_num=compute_average_num(df_Noconf_num,df_Wconf_num)
avrg_Noconf_anat,avrg_Wconf_anat=compute_average_anat(df_Noconf_anat,df_Wconf_anat)

avrghc_Noconf_num,avrghc_Wconf_num=compute_average_num(dfhc_Noconf_num,dfhc_Wconf_num)
avrghc_Noconf_anat,avrghc_Wconf_anat=compute_average_anat(dfhc_Noconf_anat,dfhc_Wconf_anat)

# stacked_df1
# np.array(stacked_df1.iloc[0]['smallworldness(AnatomicalVar)']).shape


In [ ]:
np.std(avrg_Noconf_anat['betweeness_(AnatomicalVar)'].values[0])

In [ ]:
np.std(avrghc_Noconf_anat['betweeness_(AnatomicalVar)'].values[0])

In [ ]:
def change(arr,col):
    # Create the 'region' column with labels
    
    regions = [f"region_{i+1}" for i in range(len(arr))]

    # Create the DataFrame
    df = pd.DataFrame({'region': regions, f'{col}': arr})
    return df

In [ ]:
# import plotly.graph_objects as go
# import plotly.express as px
# import pandas as pd
# from plotly.subplots import make_subplots

# # List of metrics
# metrics = ['degree', 'betweeness', 'eigenvec', 'clusteringcoef']

# # Create subplots
# fig = make_subplots(
#     rows=len(metrics), 
#     cols=1,  
#     vertical_spacing=0.05  
#     # row_titles=["x","y","z","t"],
#     # subplot_titles=[
#     #     f"Degree Centrality with anatomical mean = {avr_BetweenSubject['degree_(AnatomicalVar)'].mean()}",
#     #     f"Betweeness Centrality with anatomical mean = {avr_BetweenSubject['betweeness_(AnatomicalVar)'].mean()}",
#     #     f"Eigen Vector Centrality with anatomical mean = {avr_BetweenSubject['eigenvec_(AnatomicalVar)'].mean()}",
#     #     f"Clustering Coefficient with anatomical mean = {avr_BetweenSubject['clusteringcoef_(AnatomicalVar)'].mean()}"
#     # ]
# )



# for i, metric in enumerate(metrics):
#     # Extract columns for each metric
#     columns_to_extract_bet1 =  [col for col in avrg_Wconf_anat.columns if metric in col]
#     columns_to_extract_with1 = [col for col in avrg_Wconf_num.columns if metric in col]
#     statics_bet1 = change(avrg_Wconf_anat[columns_to_extract_bet1].iloc[0][avrg_Wconf_anat[columns_to_extract_bet1].columns[0]],avrg_Wconf_anat[columns_to_extract_bet1].columns[0])
#     statics_with1 = change(avrg_Wconf_num[columns_to_extract_with1].iloc[0][avrg_Wconf_num[columns_to_extract_with1].columns[0]],avrg_Wconf_num[columns_to_extract_with1].columns[0])

#     columns_to_extract_bet = [col for col in avrg_Noconf_anat.columns if metric in col]
#     columns_to_extract_with =  [col for col in avrg_Noconf_num.columns if metric in col]
#     statics_bet = change(avrg_Noconf_anat[columns_to_extract_bet].iloc[0][avrg_Noconf_anat[columns_to_extract_bet].columns[0]],avrg_Noconf_anat[columns_to_extract_bet].columns[0])
#     statics_with = change(avrg_Noconf_num[columns_to_extract_with].iloc[0][avrg_Noconf_num[columns_to_extract_with].columns[0]],avrg_Noconf_num[columns_to_extract_with].columns[0])
    
#     # Melt the DataFrames
#     bet_melted1 = statics_bet1.melt(id_vars=['region'], var_name='metric', value_name='value')
#     with_melted1 = statics_with1.melt(id_vars=['region'], var_name='metric', value_name='value')

#     bet_melted = statics_bet.melt(id_vars=['region'], var_name='metric', value_name='value')
#     with_melted = statics_with.melt(id_vars=['region'], var_name='metric', value_name='value')   
#     # Plot box plots for Between Subject
#     fig.add_trace(
#         go.Box(
#             x=bet_melted1['metric'], 
#             y=bet_melted1['value'],
#             showlegend=False,
#             name=f"Between Subject {metric.capitalize()}",
#             boxpoints='all',  # Add stripplot-like points
#             jitter=0.3,
#             pointpos=0,
#             marker=dict(
#                         color='rgb(34,139,34)'),
#             width=0.2
#         ),
#         row=i + 1, col=1
#     )
#         # Plot box plots for Between Subject
#     fig.add_trace(
#         go.Box(
#             x=bet_melted['metric'], 
#             y=bet_melted['value'],
#             showlegend=False,
#             name=f"Between Subject {metric.capitalize()}",
#             boxpoints='all',  # Add stripplot-like points
#             jitter=0.3,
#             pointpos=0,
#             marker=dict(
#                         color='rgb(34,139,34)'),
#             width=0.2
#         ),
#         row=i + 1, col=1
#     )
#     # Plot box plots for Within Subject
#     fig.add_trace(
#         go.Box(
#             x=with_melted1['metric'], 
#             y=with_melted1['value'],
#             showlegend=False,
#             width=0.2,  # Smaller box width,
#             name=f"Within Subject {metric.capitalize()}",
#             boxpoints='suspectedoutliers', # only suspected outliers
#             jitter=0.3,
#             pointpos=0.1,
#             marker=dict(
#                         color='rgb(8,81,156)',
#                         outliercolor='rgb(8,81,156)',
#                         line=dict(
#                             outliercolor='rgb(8,81,156)',
#                             outlierwidth=2)),
            
#             text=with_melted1['region'],  # Hover text showing subject ID
#             hovertemplate=(
#                 "<b>region:</b> %{text}<br>"  # Display subject ID
#                 "<b>Metric:</b> %{x}<br>"      # Display metric
#                 "<b>Value:</b> %{y}<br>"       # Display value
#                 "<extra></extra>"              # Remove default extra info
#         )
#         ),
#         row=i + 1, col=1
#     )
#         # Plot box plots for Within Subject
#     fig.add_trace(
#         go.Box(
#             x=with_melted['metric'], 
#             y=with_melted['value'],
#             showlegend=False,
#             width=0.2,  # Smaller box width,
#             name=f"Within Subject {metric.capitalize()}",
#             boxpoints='suspectedoutliers', # only suspected outliers
#             jitter=0.3,
#             pointpos=0.1,
#             marker=dict(
#                         color='rgb(8,81,156)',
#                         outliercolor='rgb(8,81,156)',
#                         line=dict(
#                             outliercolor='rgb(8,81,156)',
#                             outlierwidth=2)),
            
#             text=with_melted['region'],  # Hover text showing subject ID
#             hovertemplate=(
#                 "<b>region:</b> %{text}<br>"  # Display subject ID
#                 "<b>Metric:</b> %{x}<br>"      # Display metric
#                 "<b>Value:</b> %{y}<br>"       # Display value
#                 "<extra></extra>"              # Remove default extra info
#         )
#         ),
#         row=i + 1, col=1
#     )

# # Update layout
# fig.update_layout(
#     title="Regional Variability (Local Graph Metric)",
#     #    (title=dict(text="Your Title Here", )x=0.5, y=0.95),  # Center title and move it up
#     # margin=dict(t=100)  # Increase top margin to avoid overlap
#     height=150 * len(metrics),
#     width=900,
#     showlegend=True,
#     legend_title="Legend",
#     margin=dict(l=50, r=50, t=50, b=100),
#     template='plotly'
# )
# for row in range(1, len(metrics) + 1):
#     fig.update_yaxes(tickformat=".1e", row=row, col=1)

# # Iterate through rows to remove x-axis tick labels for all subplots
# for row in range(1, len(metrics) + 1):
#     fig.update_xaxes(showticklabels=False, row=row, col=1)

# fig.update_xaxes(
#     #title_text="Anatomical Variability with confounds" + "&nbsp;" *10 +  "Anatomical Variability No confounds" + "&nbsp;" *10  + "Numerical Variability with confounds" + "&nbsp;" *10 + "Numerical Variability No confounds",
#     row=len(metrics),
#     col=1,
#     side='bottom',
#     title_font=dict(size=11)  # Adjust the font size here

# )
# fig.add_annotation(
#     text="Degree Centrality",
#     xref="paper", yref="paper",
#     x=0.0, y=1.038,  # Centered horizontally above the first two rows
#     showarrow=False,
# )

# fig.add_annotation(
#     text="Betweeness Centrality",
#     xref="paper", yref="paper",
#     x=0.0, y=0.79 , # Centered horizontally above the last two rows
#     showarrow=False,
# )
# fig.add_annotation(
#     text="Eigen vector Centrality",
#     xref="paper", yref="paper",
#     x=0.0, y=0.5,  # Centered horizontally above the last two rows
#     showarrow=False,
# )

# fig.add_annotation(
#     text="Clustering coefficient",
#     xref="paper", yref="paper",
#     x=0.0, y=0.23,  # Centered horizontally above the last two rows
#     showarrow=False,
# )
# fig.add_annotation(
#     x=0.0,  # Adjust position as needed
#     y=-0.05,  # Adjust to place below the x-axis
#     xref="paper", yref="paper",
#     text="Anatomical Variability with confounds" + "&nbsp;" * 10 +
#          "Anatomical Variability No confounds" + "&nbsp;" * 10 +
#          "Numerical Variability with confounds" + "&nbsp;" * 10 +
#          "Numerical Variability No confounds",
#     showarrow=False,
#     font=dict(size=11),
#     align="center"
# )

# fig.write_image("LocalMetricplot.png", scale=3)


# # # Remove annotations for avr_bet
# # import plotly.io as pio
# # pio.write_html(fig, "LocalMetricplot.html")
# fig.show()

In [ ]:
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
from plotly.subplots import make_subplots

# List of metrics
metrics = ['degree', 'betweeness', 'eigenvec', 'clusteringcoef']

# Create subplots
fig = make_subplots(
    rows=len(metrics), 
    cols=1,  
    vertical_spacing=0.05
)

# Colors for different groups
colors = {
    'pd_anat_with_conf': 'rgb(34,139,34)',      # Dark Green
    'pd_anat_no_conf': 'rgb(60,179,60)',        # Light Green
    'pd_num_with_conf': 'rgb(8,81,156)',        # Dark Blue
    'pd_num_no_conf': 'rgb(49,130,189)',        # Light Blue
    'hc_anat_with_conf': 'rgb(178,34,34)',           # Dark Red
    'hc_anat_no_conf': 'rgb(255,69,69)',             # Light Red
    'hc_num_with_conf': 'rgb(148,0,211)',            # Dark Purple
    'hc_num_no_conf': 'rgb(186,85,211)'              # Light Purple
}

for i, metric in enumerate(metrics):
    # Patient group data processing
    # With confounds - Anatomical
    columns_to_extract_bet1 = [col for col in avrg_Wconf_anat.columns if metric in col]
    statics_bet1 = change(avrg_Wconf_anat[columns_to_extract_bet1].iloc[0][avrg_Wconf_anat[columns_to_extract_bet1].columns[0]], avrg_Wconf_anat[columns_to_extract_bet1].columns[0])
    bet_melted1 = statics_bet1.melt(id_vars=['region'], var_name='metric', value_name='value')
    
    # With confounds - Numerical
    columns_to_extract_with1 = [col for col in avrg_Wconf_num.columns if metric in col]
    statics_with1 = change(avrg_Wconf_num[columns_to_extract_with1].iloc[0][avrg_Wconf_num[columns_to_extract_with1].columns[0]], avrg_Wconf_num[columns_to_extract_with1].columns[0])
    with_melted1 = statics_with1.melt(id_vars=['region'], var_name='metric', value_name='value')
    
    # No confounds - Anatomical
    columns_to_extract_bet = [col for col in avrg_Noconf_anat.columns if metric in col]
    statics_bet = change(avrg_Noconf_anat[columns_to_extract_bet].iloc[0][avrg_Noconf_anat[columns_to_extract_bet].columns[0]], avrg_Noconf_anat[columns_to_extract_bet].columns[0])
    bet_melted = statics_bet.melt(id_vars=['region'], var_name='metric', value_name='value')
    
    # No confounds - Numerical
    columns_to_extract_with = [col for col in avrg_Noconf_num.columns if metric in col]
    statics_with = change(avrg_Noconf_num[columns_to_extract_with].iloc[0][avrg_Noconf_num[columns_to_extract_with].columns[0]], avrg_Noconf_num[columns_to_extract_with].columns[0])
    with_melted = statics_with.melt(id_vars=['region'], var_name='metric', value_name='value')
    
    # Healthy Control group data processing
    # With confounds - Anatomical
    columns_to_extract_hc_bet1 = [col for col in avrghc_Wconf_anat.columns if metric in col]
    statics_hc_bet1 = change(avrghc_Wconf_anat[columns_to_extract_hc_bet1].iloc[0][avrghc_Wconf_anat[columns_to_extract_hc_bet1].columns[0]], avrghc_Wconf_anat[columns_to_extract_hc_bet1].columns[0])
    hc_bet_melted1 = statics_hc_bet1.melt(id_vars=['region'], var_name='metric', value_name='value')
    
    # With confounds - Numerical
    columns_to_extract_hc_with1 = [col for col in avrghc_Wconf_num.columns if metric in col]
    statics_hc_with1 = change(avrghc_Wconf_num[columns_to_extract_hc_with1].iloc[0][avrghc_Wconf_num[columns_to_extract_hc_with1].columns[0]], avrghc_Wconf_num[columns_to_extract_hc_with1].columns[0])
    hc_with_melted1 = statics_hc_with1.melt(id_vars=['region'], var_name='metric', value_name='value')
    
    # No confounds - Anatomical
    columns_to_extract_hc_bet = [col for col in avrghc_Noconf_anat.columns if metric in col]
    statics_hc_bet = change(avrghc_Noconf_anat[columns_to_extract_hc_bet].iloc[0][avrghc_Noconf_anat[columns_to_extract_hc_bet].columns[0]], avrghc_Noconf_anat[columns_to_extract_hc_bet].columns[0])
    hc_bet_melted = statics_hc_bet.melt(id_vars=['region'], var_name='metric', value_name='value')
    
    # No confounds - Numerical
    columns_to_extract_hc_with = [col for col in avrghc_Noconf_num.columns if metric in col]
    statics_hc_with = change(avrghc_Noconf_num[columns_to_extract_hc_with].iloc[0][avrghc_Noconf_num[columns_to_extract_hc_with].columns[0]], avrghc_Noconf_num[columns_to_extract_hc_with].columns[0])
    hc_with_melted = statics_hc_with.melt(id_vars=['region'], var_name='metric', value_name='value')
    
    # Adjust x positions for better separation of groups
    x_positions = {
        'pd_anat_with_conf': 1,
        'hc_anat_with_conf':2,
        'pd_anat_no_conf': 3,
        'hc_anat_no_conf':4,
        'pd_num_with_conf': 5,
        'hc_num_with_conf':6,
        'pd_num_no_conf': 7,
        'hc_num_no_conf':8
        
    }
    
    # Patient group - Anatomical with confounds
    fig.add_trace(
        go.Box(
            x=[x_positions['pd_anat_with_conf']] * len(bet_melted1),
            y=bet_melted1['value'],
            name="Pd Anat W/Conf",
            boxpoints='all',
            jitter=0.3,
            pointpos=0,
            marker=dict(color=colors['pd_anat_with_conf']),
            width=0.5,
            showlegend=False,  # Show legend only for the first metric
            text=bet_melted1['region'],
            hovertemplate="<b>Region:</b> %{text}<br><b>Value:</b> %{y}<br><extra>pd Anat W/Conf</extra>"
        ),
        row=i + 1, col=1
    )
    
    # Patient group - Anatomical no confounds
    fig.add_trace(
        go.Box(
            x=[x_positions['hc_anat_with_conf']] * len(bet_melted),
            y=bet_melted['value'],
            name="hc Anat W/Conf",
            boxpoints='all',
            jitter=0.3,
            pointpos=0,
            marker=dict(color=colors['hc_anat_with_conf']),
            width=0.5,
            showlegend=False,
            text=bet_melted['region'],
            hovertemplate="<b>Region:</b> %{text}<br><b>Value:</b> %{y}<br><extra>hc Anat W/Conf</extra>"
        ),
        row=i + 1, col=1
    )
    
    # Patient group - Numerical with confounds
    fig.add_trace(
        go.Box(
            x=[x_positions['pd_anat_no_conf']] * len(with_melted1),
            y=with_melted1['value'],
            name="pd Anat no/Conf",
            boxpoints='suspectedoutliers',
            jitter=0.3,
            pointpos=0,
            marker=dict(
                color=colors['pd_anat_no_conf'],
                outliercolor=colors['pd_anat_no_conf'],
                line=dict(outliercolor=colors['pd_anat_no_conf'], outlierwidth=2)
            ),
            width=0.5,
            showlegend=False,
            text=with_melted1['region'],
            hovertemplate="<b>Region:</b> %{text}<br><b>Value:</b> %{y}<br><extra>pd Anat no/Conf</extra>"
        ),
        row=i + 1, col=1
    )
    
    # Patient group - Numerical no confounds
    fig.add_trace(
        go.Box(
            x=[x_positions['hc_anat_no_conf']] * len(with_melted),
            y=with_melted['value'],
            name="hc Anat No/Conf",
            boxpoints='suspectedoutliers',
            jitter=0.3,
            pointpos=0,
            marker=dict(
                color=colors['hc_anat_no_conf'],
                outliercolor=colors['hc_anat_no_conf'],
                line=dict(outliercolor=colors['hc_anat_no_conf'], outlierwidth=2)
            ),
            width=0.5,
            showlegend=False,
            text=with_melted['region'],
            hovertemplate="<b>Region:</b> %{text}<br><b>Value:</b> %{y}<br><extra>hc Anat No/Conf</extra>"
        ),
        row=i + 1, col=1
    )
    
    # HC group - Anatomical with confounds
    fig.add_trace(
        go.Box(
            x=[x_positions['pd_num_with_conf']] * len(hc_bet_melted1),
            y=hc_bet_melted1['value'],
            name="pd Num W/Conf",
            boxpoints='all',
            jitter=0.3,
            pointpos=0,
            marker=dict(color=colors['pd_num_with_conf']),
            width=0.5,
            showlegend=False,
            text=hc_bet_melted1['region'],
            hovertemplate="<b>Region:</b> %{text}<br><b>Value:</b> %{y}<br><extra>pd Num W/Conf</extra>"
        ),
        row=i + 1, col=1
    )
    
    # HC group - Anatomical no confounds
    fig.add_trace(
        go.Box(
            x=[x_positions['hc_num_with_conf']] * len(hc_bet_melted),
            y=hc_bet_melted['value'],
            name="HC Num W/Conf",
            boxpoints='all',
            jitter=0.3,
            pointpos=0,
            marker=dict(color=colors['hc_num_no_conf']),
            width=0.5,
            showlegend=False,
            text=hc_bet_melted['region'],
            hovertemplate="<b>Region:</b> %{text}<br><b>Value:</b> %{y}<br><extra>HC Num W/Conf</extra>"
        ),
        row=i + 1, col=1
    )
    
    # HC group - Numerical with confounds
    fig.add_trace(
        go.Box(
            x=[x_positions['pd_num_no_conf']] * len(hc_with_melted1),
            y=hc_with_melted1['value'],
            name="pd Num No/Conf",
            boxpoints='suspectedoutliers',
            jitter=0.3,
            pointpos=0,
            marker=dict(
                color=colors['pd_num_no_conf'],
                outliercolor=colors['pd_num_no_conf'],
                line=dict(outliercolor=colors['pd_num_with_conf'], outlierwidth=2)
            ),
            width=0.5,
            showlegend=False,
            text=hc_with_melted1['region'],
            hovertemplate="<b>Region:</b> %{text}<br><b>Value:</b> %{y}<br><extra>pd Num No/Conf</extra>"
        ),
        row=i + 1, col=1
    )
    
    # HC group - Numerical no confounds
    fig.add_trace(
        go.Box(
            x=[x_positions['hc_num_no_conf']] * len(hc_with_melted),
            y=hc_with_melted['value'],
            name="HC Num No/Conf",
            boxpoints='suspectedoutliers',
            jitter=0.3,
            pointpos=0,
            marker=dict(
                color=colors['hc_num_no_conf'],
                outliercolor=colors['hc_num_no_conf'],
                line=dict(outliercolor=colors['hc_num_no_conf'], outlierwidth=2)
            ),
            width=0.5,
            showlegend=False,
            text=hc_with_melted['region'],
            hovertemplate="<b>Region:</b> %{text}<br><b>Value:</b> %{y}<br><extra>HC Num No/Conf</extra>"
        ),
        row=i + 1, col=1
    )

# Update layout
fig.update_layout(
    title="Regional Variability (Local Graph Metric) - Pd vs. Healthy Controls",
    height=200 * len(metrics),  # Increased height to accommodate more data
    width=1200,  # Increased width
    showlegend=False,
    # legend_title="Groups",
    # legend=dict(
    #     orientation="h",
    #     yanchor="bottom",
    #     y=-0.15,
    #     xanchor="center",
    #     x=0.5
    # ),
    margin=dict(l=50, r=50, t=80, b=150),  # Increased bottom margin for legend
    template='plotly'
)

# Update y-axes with scientific notation format
for row in range(1, len(metrics) + 1):
    fig.update_yaxes(tickformat=".1e", row=row, col=1)

# Create custom x-axis ticks
x_ticks = list(x_positions.values())
x_tick_labels = [
    "pd Anat W/Conf", "hc Anat No/Conf", 
    "pd Num W/Conf", "hc Num No/Conf",
    "pd Anat W/Conf", "hc Anat No/Conf", 
    "pd Num W/Conf", "hc Num No/Conf"
]

# Remove x-axis tick labels for all but the last subplot
for row in range(1, len(metrics)):
    fig.update_xaxes(showticklabels=False, row=row, col=1)

# Add x-axis labels only to the bottom subplot
fig.update_xaxes(
    tickvals=x_ticks,
    ticktext=x_tick_labels,
    tickangle=45,
    row=len(metrics),
    col=1
)

# Add annotations for metric names on the left side
metric_labels = ["Degree Centrality", "Betweeness Centrality", "Eigen vector Centrality", "Clustering coefficient"]
y_positions = [1.038,0.79,0.5,0.23]  # Adjusted for 4 metrics

for i, (label, y_pos) in enumerate(zip(metric_labels, y_positions)):
    fig.add_annotation(
        text=label,
        xref="paper", yref="paper",
        x=0.0, y=y_pos,
        showarrow=False,
        xanchor="left"
    )

# Save figure
fig.write_image("LocalMetricPlot_Patients_vs_HC.png", scale=3)
fig.write_html("LocalMetricPlot_Patients_vs_HC.html")

# Show figure
fig.show()

In [ ]:
# # List of metrics
# metrics = ['degree', 'betweeness', 'eigenvec', 'clusteringcoef']

# # Create subplots
# fig = make_subplots(
#     rows=len(metrics), 
#     cols=1,  
#     vertical_spacing=0.05   
# )

# # Function to add jitter
# def add_jitter(x_values, scale=0.2):
#     return x_values + np.random.uniform(-scale, scale, size=len(x_values))

# for i, metric in enumerate(metrics):
#     # Extract columns for each metric
#     columns_to_extract_bet1 = [col for col in avrg_Wconf_anat.columns if metric in col]
#     columns_to_extract_with1 = [col for col in avrg_Wconf_num.columns if metric in col]
#     statics_bet1 = change(avrg_Wconf_anat[columns_to_extract_bet1].iloc[0][avrg_Wconf_anat[columns_to_extract_bet1].columns[0]],avrg_Wconf_anat[columns_to_extract_bet1].columns[0])
#     statics_with1 = change(avrg_Wconf_num[columns_to_extract_with1].iloc[0][avrg_Wconf_num[columns_to_extract_with1].columns[0]],avrg_Wconf_num[columns_to_extract_with1].columns[0])

#     columns_to_extract_bet = [col for col in avrg_Noconf_anat.columns if metric in col]
#     columns_to_extract_with = [col for col in avrg_Noconf_num.columns if metric in col]
#     statics_bet = change(avrg_Noconf_anat[columns_to_extract_bet].iloc[0][avrg_Noconf_anat[columns_to_extract_bet].columns[0]],avrg_Noconf_anat[columns_to_extract_bet].columns[0])
#     statics_with = change(avrg_Noconf_num[columns_to_extract_with].iloc[0][avrg_Noconf_num[columns_to_extract_with].columns[0]],avrg_Noconf_num[columns_to_extract_with].columns[0])

#     # Melt the DataFrames
#     bet_melted1 = statics_bet1.melt(id_vars=['region'], var_name='metric', value_name='value')
#     with_melted1 = statics_with1.melt(id_vars=['region'], var_name='metric', value_name='value')

#     bet_melted = statics_bet.melt(id_vars=['region'], var_name='metric', value_name='value')
#     with_melted = statics_with.melt(id_vars=['region'], var_name='metric', value_name='value')   
    
#     # Assign x positions with jitter for each dataset
#     x_bet1 = add_jitter(np.full_like(bet_melted1['value'], i * 4 + 0))
#     x_bet = add_jitter(np.full_like(bet_melted['value'], i * 4 + 1))
#     x_with1 = add_jitter(np.full_like(with_melted1['value'], i * 4 + 2))
#     x_with = add_jitter(np.full_like(with_melted['value'], i * 4 + 3))

#     # Plot scatter for Between Subject (confounds)
#     fig.add_trace(
#         go.Scatter(
#             x=x_bet1, 
#             y=bet_melted1['value'],
#             mode='markers',
#             marker=dict(color='rgb(34,139,34)', size=8, opacity=0.7),
#             name=f"Between Subject {metric.capitalize()} (Confounds)",
#             text=bet_melted1['region'],
#             hoverinfo="text+y"
#         ),
#         row=i + 1, col=1
#     )

#     # Plot scatter for Between Subject (No confounds)
#     fig.add_trace(
#         go.Scatter(
#             x=x_bet, 
#             y=bet_melted['value'],
#             mode='markers',
#             marker=dict(color='rgb(34,139,34)', size=8, opacity=0.7),
#             name=f"Between Subject {metric.capitalize()} (No Confounds)",
#             text=bet_melted['region'],
#             hoverinfo="text+y"
#         ),
#         row=i + 1, col=1
#     )

#     # Plot scatter for Within Subject (confounds)
#     fig.add_trace(
#         go.Scatter(
#             x=x_with1, 
#             y=with_melted1['value'],
#             mode='markers',
#             marker=dict(color='rgb(8,81,156)', size=8, opacity=0.7),
#             name=f"Within Subject {metric.capitalize()} (Confounds)",
#             text=with_melted1['region'],
#             hoverinfo="text+y"
#         ),
#         row=i + 1, col=1
#     )

#     # Plot scatter for Within Subject (No confounds)
#     fig.add_trace(
#         go.Scatter(
#             x=x_with, 
#             y=with_melted['value'],
#             mode='markers',
#             marker=dict(color='rgb(8,81,156)', size=8, opacity=0.7),
#             name=f"Within Subject {metric.capitalize()} (No Confounds)",
#             text=with_melted['region'],
#             hoverinfo="text+y"
#         ),
#         row=i + 1, col=1
#     )

# # Update layout
# fig.update_layout(
#     title="Anatomical and Numerical Variability (Local Graph Metric)",
#     height=300 * len(metrics),
#     showlegend=True,
#     legend_title="Legend",
#     margin=dict(l=50, r=50, t=50, b=100),
#     template='plotly'
# )

# fig.show()

In [ ]:
import numpy as np
import pandas as pd

# Assuming 'df' has already been extracted
dfG = df_Noconf_num[(df_Noconf_num['acquisition'] == 'acq-RL') & 
                             (df_Noconf_num['session'] == '1')]

dfGNo_num = dfG.drop(columns=['degree_(numericalVar)',	'betweeness_(numericalVar)',	'clusteringcoef_(numericalVar)'	,'eigenvec_(numericalVar)' ])# Remove the unnecessary column



##########################################################################################3
# Assuming 'df' has already been extracted
df1G= df_Wconf_num[(df_Wconf_num['acquisition'] == 'acq-RL') & 
                             (df_Wconf_num['session'] == '1')]

# df1 = df1.drop(columns=['session', 'acquisition', 'subject'])
df1GW_num = df1G.drop(columns=['degree_(numericalVar)',	'betweeness_(numericalVar)',	'clusteringcoef_(numericalVar)'	,'eigenvec_(numericalVar)' ])# Remove the unnecessary column



In [ ]:
import numpy as np
import pandas as pd

# Assuming 'df' has already been extracted
dfG = df_Noconf_anat[(df_Noconf_anat['acquisition'] == 'acq-RL') & 
                             (df_Noconf_anat['session'] == '1')]

dfGNo_anat = dfG.drop(columns=['degree_(AnatomicalVar)'	,'betweeness_(AnatomicalVar)',	'clusteringcoef_(AnatomicalVar)'	,'eigenvec_(AnatomicalVar)'])


###########################################################################################3
# Assuming 'df' has already been extracted
df1G = df_Wconf_anat[(df_Wconf_anat['acquisition'] == 'acq-RL') & 
                             (df_Wconf_anat['session'] == '1')]

df1GW_anat= df1G.drop(columns=['degree_(AnatomicalVar)'	,'betweeness_(AnatomicalVar)',	'clusteringcoef_(AnatomicalVar)'	,'eigenvec_(AnatomicalVar)'])
df1GW_num = df1GW_num.rename(columns={'smallworldness(numericalVar)':'smallworldness(numericalVarW)','avg_shortestPathLength(numericalVar)':'avg_shortestPathLength(numericalVarW)'})

df1GW_anat = df1GW_anat.rename(columns={'smallworldness(AnatomicalVar)':'smallworldness(AnatomicalVarW)','avg_shortestPathLength(AnatomicalVar)':'avg_shortestPathLength(AnatomicalVarW)'  })

df1GW_num


In [ ]:
# import plotly.graph_objects as go
# import plotly.express as px
# import pandas as pd
# from plotly.subplots import make_subplots

# # List of metrics
# metrics = ['smallworldness', 'avg_shortestPathLength']

# # Create subplots
# fig = make_subplots(
#     rows=len(metrics), 
#     cols=1,  
#     vertical_spacing=0.05,     
#     # subplot_titles=[
#     #     f"Small worldness with anatomical mean = {avr_BetweenSubject['smallworldness_(AnatomicalVar)'].mean()}",
#     #     f"Avrg shortest path with anatomical mean = {avr_BetweenSubject['avg_shortestPathLength_(AnatomicalVar)'].mean()}"
#     # ]
# )



# for i, metric in enumerate(metrics):

#     # Extract columns for each metric
#     columns_to_extract_bet1 = ['iteration'] + [col for col in df1GW_anat.columns if metric in col]
#     columns_to_extract_with1 = ['subject'] + [col for col in df1GW_num.columns if metric in col]
    
#     statics_bet1 = df1GW_anat[columns_to_extract_bet1]
#     statics_with1 = df1GW_num[columns_to_extract_with1]

#     columns_to_extract_bet = ['iteration'] + [col for col in dfGNo_anat.columns if metric in col]
#     columns_to_extract_with = ['subject'] + [col for col in dfGNo_num.columns if metric in col]
    
#     statics_bet = dfGNo_anat[columns_to_extract_bet]
#     statics_with = dfGNo_num[columns_to_extract_with]
#     bet_melted1 = statics_bet1.melt(id_vars=['iteration'], var_name='metric', value_name='value')
#     with_melted1 = statics_with1.melt(id_vars=['subject'], var_name='metric', value_name='value')

#     bet_melted = statics_bet.melt(id_vars=['iteration'], var_name='metric', value_name='value')
#     with_melted = statics_with.melt(id_vars=['subject'], var_name='metric', value_name='value')  
#     # Plot box plots for Between Subject
#     fig.add_trace(
#         go.Box(
#             x=bet_melted1['metric'], 
#             y=bet_melted1['value'],
#             showlegend=False,
#             name=f"Between Subject With confounds{metric.capitalize()}",
#             boxpoints='all',  # Add stripplot-like points
#             jitter=0.3,
#             pointpos=0,
#             marker=dict(
#                         color='rgb(34,139,34)'),
#             width=0.2
#         ),
#         row=i + 1, col=1
#     )
#      # Plot box plots for Between Subject
#     fig.add_trace(
#         go.Box(
#             x=bet_melted['metric'], 
#             y=bet_melted['value'],
#             showlegend=False,
#             name=f"Between Subject {metric.capitalize()}",
#             boxpoints='all',  # Add stripplot-like points
#             jitter=0.3,
#             pointpos=0,
#             marker=dict(
#                         color='rgb(34,139,34)'),
#             width=0.2
#         ),
#         row=i + 1, col=1
#     )   
#     # Plot box plots for Within Subject
#     fig.add_trace(
#         go.Box(
#             x=with_melted1['metric'], 
#             y=with_melted1['value'],
#             showlegend=False,
#             width=0.2,  # Smaller box width,
#             name=f"Within Subject With confounds{metric.capitalize()}",
#             boxpoints='suspectedoutliers', # only suspected outliers
#             jitter=0.3,
#             pointpos=0.1,
#             marker=dict(
#                         color='rgb(8,81,156)',
#                         outliercolor='rgb(8,81,156)',
#                         line=dict(
#                             outliercolor='rgb(8,81,156)',
#                             outlierwidth=2)),
            
#             text=with_melted1['subject'],  # Hover text showing subject ID
#             hovertemplate=(
#                 "<b>region:</b> %{text}<br>"  # Display subject ID
#                 "<b>Metric:</b> %{x}<br>"      # Display metric
#                 "<b>Value:</b> %{y}<br>"       # Display value
#                 "<extra></extra>"              # Remove default extra info
#         )
#         ),
#         row=i + 1, col=1
#     )
#     # Plot box plots for Within Subject
#     fig.add_trace(
#         go.Box(
#             x=with_melted['metric'], 
#             y=with_melted['value'],
#             showlegend=False,
#             width=0.2,  # Smaller box width,
#             name=f"Within Subject {metric.capitalize()}",
#             boxpoints='suspectedoutliers', # only suspected outliers
#             jitter=0.3,
#             pointpos=0.1,
#             marker=dict(
#                         color='rgb(8,81,156)',
#                         outliercolor='rgb(8,81,156)',
#                         line=dict(
#                             outliercolor='rgb(8,81,156)',
#                             outlierwidth=2)),
            
#             text=with_melted['subject'],  # Hover text showing subject ID
#             hovertemplate=(
#                 "<b>region:</b> %{text}<br>"  # Display subject ID
#                 "<b>Metric:</b> %{x}<br>"      # Display metric
#                 "<b>Value:</b> %{y}<br>"       # Display value
#                 "<extra></extra>"              # Remove default extra info
#         )
#         ),
#         row=i + 1, col=1
#     )

# # Update layout
# fig.update_layout(
#     title="Anatomical and Numerical Variability (Global Graph Metric)",
#     height=300 * len(metrics),
#     showlegend=True,
#     legend_title="Legend",
#     margin=dict(l=50, r=50, t=50, b=100),
#     template='plotly'
# )

# # Iterate through rows to remove x-axis tick labels for all subplots
# for row in range(1, len(metrics) + 1):
#     fig.update_xaxes(showticklabels=False, row=row, col=1)

# fig.update_xaxes(
#     title_text="Anatomical Variability with confounds" + "&nbsp;" *40 +  "Anatomical Variability No confounds" + "&nbsp;" *40  + "Numerical Variability with confounds" + "&nbsp;" *40 + "Numerical Variability No confounds",
#     row=len(metrics),
#     col=1,
#     side='bottom'
# )
# # Add section titles for anatomical and numerical data
# fig.add_annotation(
#     text="Smallworldness",
#     xref="paper", yref="paper",
#     x=0.0, y=1.035,  # Centered horizontally above the first two rows
#     showarrow=False,
# )

# fig.add_annotation(
#     text="Average shortest path",
#     xref="paper", yref="paper",
#     x=0.0, y=0.50 , # Centered horizontally above the last two rows
#     showarrow=False,
# )
# import plotly.io as pio
# pio.write_html(fig, "GlobalMetricplot.html")
# # Remove annotations for avr_bet
# fig.show()

In [ ]:
def stdofavrg_with(Results_table) :
   # Initialize DataFrames
    stdofavr_WithinSubject = pd.DataFrame()  # Std over averaged across runs
    avr_WithinSubject = pd.DataFrame()  # Average across runs

    # Get unique subjects from the Results_table
    subjects =np.unique(Results_table['subject'])


    i=0
    # Loop over each subject
    for subj in subjects: 
        # Filter the DataFrame for each subject and drop unnecessary columns once
        filtered_df = Results_table[Results_table['subject'] == subj]
        
        # Loop over each unique session and acquisition combination
        for session in np.unique(filtered_df['session']):
            for acquisition in np.unique(filtered_df['acquisition']):
                if acquisition not in ['acq-RLsplit1', 'acq-LRsplit1']:
                    avr_degree_centrality, avr_betweeness_centrality, avr_eigen_centrality, avr_clustering_coef = [], [], [], []
                    std_degree_centrality, std_betweeness_centrality, std_eigen_centrality, std_clustering_coef = [], [], [], []
                    smallworldness, avg_shortestPathLength = [], []                # Loop over each repetition to generate correlation matrices
                    for rep in range(1, 11):
                        # Filter for each specific repetition
                        filtered_rows = filtered_df[
                            (filtered_df['session'] == session) &
                            (filtered_df['acquisition'] == acquisition) &
                            (filtered_df['repetition'] == f'rep-{rep}')
                        ]
                        if filtered_rows.empty:
                            continue
                        degree_values = list(filtered_rows['degree_centralities'].values[0].values())
                        betweeness_values = list(filtered_rows['betweenness_centralities'].values[0].values())
                        eigen_values = list(filtered_rows['eigenvector_centralities'].values[0].values())
                        clustering_values = list(filtered_rows['clustering_coefficients'].values[0].values())
                        # Append averages for each iteration
                        avr_degree_centrality.append(np.mean(degree_values))
                        avr_betweeness_centrality.append(np.mean(betweeness_values))
                        avr_eigen_centrality.append(np.mean(eigen_values))
                        avr_clustering_coef.append(np.mean(clustering_values))
                        # Append standard deviations for each
                        std_degree_centrality.append(np.std(degree_values))
                        std_betweeness_centrality.append(np.std(betweeness_values))
                        std_eigen_centrality.append(np.std(eigen_values))
                        std_clustering_coef.append(np.std(clustering_values))
                        smallworldness.append(filtered_rows['small_worldness'])
                        avg_shortestPathLength.append(filtered_rows['avg_shortest_path_length'])
                                # Calculate and store statistics for each subject after all iterations
                    if not avr_degree_centrality:
                        continue
                    stdofavr_WithinSubject.loc[i, 'degree_(numericalVar)'] = np.std(avr_degree_centrality)
                    stdofavr_WithinSubject.loc[i, 'betweeness_(numericalVar)'] = np.std(avr_betweeness_centrality)
                    stdofavr_WithinSubject.loc[i, 'clusteringcoef_(numericalVar)'] = np.std(avr_clustering_coef)
                    stdofavr_WithinSubject.loc[i, 'eigenvec_(numericalVar)'] = np.std(avr_eigen_centrality)
                    stdofavr_WithinSubject.loc[i, 'smallworldness(numericalVar)'] = np.std(smallworldness)
                    stdofavr_WithinSubject.loc[i, 'avg_shortestPathLength(numericalVar)'] = np.std(avg_shortestPathLength)

                    stdofavr_WithinSubject.loc[i, 'subject']=subj
                    stdofavr_WithinSubject.loc[i, 'session']= session
                    stdofavr_WithinSubject.loc[i, 'acquisition']= acquisition
                    
                    avr_WithinSubject.loc[i, 'degree_(numericalVar)'] = np.mean(avr_degree_centrality)
                    avr_WithinSubject.loc[i, 'betweeness_(numericalVar)'] = np.mean(avr_betweeness_centrality)
                    avr_WithinSubject.loc[i, 'clusteringcoef_(numericalVar)'] = np.mean(avr_clustering_coef)
                    avr_WithinSubject.loc[i, 'eigenvec_(numericalVar)'] = np.mean(avr_eigen_centrality)
                    avr_WithinSubject.loc[i, 'smallworldness(numericalVar)'] = np.mean(smallworldness)
                    avr_WithinSubject.loc[i, 'avg_shortestPathLength(numericalVar)'] = np.mean(avg_shortestPathLength)
                    avr_WithinSubject.loc[i, 'subject']=subj
                    avr_WithinSubject.loc[i, 'session']= session
                    avr_WithinSubject.loc[i, 'acquisition']= acquisition
                    
                    i=i+1
    stdofavr_WithinSubject=stdofavr_WithinSubject.dropna()
    avr_WithinSubject=avr_WithinSubject.dropna()
    return stdofavr_WithinSubject

In [ ]:
def stdofavrg_bet(Results_table):
    # Initialize DataFrames for between-subject analysis
    stdofavr_BetweenSubject = pd.DataFrame()  # Std over averaged across subjects
    avr_BetweenSubject = pd.DataFrame()  # Average across subjects


    # Get unique subjects from the Results_table
    subjects = np.unique(Results_table['subject'])

    # Initialize a list to hold data entries for the final DataFrame
    subject_data = []
    i=0
    # Loop over each repetition
    for rep in range(1, 11):
        # Filter DataFrame for the current repetition and drop unnecessary columns
        filtered_df = Results_table[Results_table['repetition'] == f'rep-{rep}']

        # Loop over each session
        for session in np.unique(filtered_df['session']):
            # Loop over each acquisition type
            for acquisition in np.unique(filtered_df['acquisition']):
                # Skip unwanted acquisitions
                if acquisition in ['acq-RLsplit1', 'acq-LRsplit1']:
                    continue
                
                # Initialize lists to store results for each iteration across subjects
                acrosSub_avr_degree_centrality = []
                acrosSub_avr_betweeness_centrality = []
                acrosSub_avr_eigen_centrality = []
                acrosSub_avr_clustering_coef = []
                acrosSub_std_degree_centrality = []
                acrosSub_std_betweeness_centrality = []
                acrosSub_std_eigen_centrality = []
                acrosSub_std_clustering_coef = []
                acrosSub_smallworld=[]
                acrosSub_avrgShortestPathLength=[]
                
                # Loop over each subject
                for subj in subjects:
                    # Filter rows for the specific subject, session, and acquisition
                    filtered_rows = filtered_df[
                        (filtered_df['session'] == session) &
                        (filtered_df['acquisition'] == acquisition) &
                        (filtered_df['subject'] == subj)
                    ]

                    if filtered_rows.empty:
                        continue
                    degree_values = list(filtered_rows['degree_centralities'].values[0].values())
                    betweeness_values = list(filtered_rows['betweenness_centralities'].values[0].values())
                    eigen_values = list(filtered_rows['eigenvector_centralities'].values[0].values())
                    clustering_values = list(filtered_rows['clustering_coefficients'].values[0].values())

                    # Append averages for each subject
                    acrosSub_avr_degree_centrality.append(np.mean(degree_values))
                    acrosSub_avr_betweeness_centrality.append(np.mean(betweeness_values))
                    acrosSub_avr_eigen_centrality.append(np.mean(eigen_values))
                    acrosSub_avr_clustering_coef.append(np.mean(clustering_values))

                    # Append standard deviations for each subject
                    acrosSub_std_degree_centrality.append(np.std(degree_values))
                    acrosSub_std_betweeness_centrality.append(np.std(betweeness_values))
                    acrosSub_std_eigen_centrality.append(np.std(eigen_values))
                    acrosSub_std_clustering_coef.append(np.std(clustering_values))
                    acrosSub_smallworld.append(filtered_rows['small_worldness'])
                    acrosSub_avrgShortestPathLength.append(filtered_rows['avg_shortest_path_length'])
                # Calculate and store statistics for each iteration across subjects
                if len(acrosSub_avr_degree_centrality)<2:
                    continue
                if rep==8 and session=='1' and acquisition=='acq-RL':
                     print(np.std(acrosSub_avr_degree_centrality))
                     print(acrosSub_avr_degree_centrality)
                stdofavr_BetweenSubject.loc[i, 'degree_(AnatomicalVar)'] = np.std(acrosSub_avr_degree_centrality)
                stdofavr_BetweenSubject.loc[i, 'betweeness_(AnatomicalVar)'] = np.std(acrosSub_avr_betweeness_centrality)
                stdofavr_BetweenSubject.loc[i, 'clusteringcoef_(AnatomicalVar)'] = np.std(acrosSub_avr_clustering_coef)
                stdofavr_BetweenSubject.loc[i, 'eigenvec_(AnatomicalVar)'] = np.std(acrosSub_avr_eigen_centrality)
                stdofavr_BetweenSubject.loc[i, 'smallworldness_(AnatomicalVar)'] = np.std(acrosSub_smallworld)
                stdofavr_BetweenSubject.loc[i, 'avg_shortestPathLength_(AnatomicalVar)'] = np.std(acrosSub_avrgShortestPathLength)
            
                stdofavr_BetweenSubject.loc[i, 'iter']=f'iter_{rep}'
                stdofavr_BetweenSubject.loc[i, 'session']= session
                stdofavr_BetweenSubject.loc[i, 'acquisition']= acquisition

                avr_BetweenSubject.loc[i, 'degree_(AnatomicalVar)'] = np.mean(acrosSub_avr_degree_centrality)
                avr_BetweenSubject.loc[i, 'betweeness_(AnatomicalVar)'] = np.mean(acrosSub_avr_betweeness_centrality)
                avr_BetweenSubject.loc[i, 'clusteringcoef_(AnatomicalVar)'] = np.mean(acrosSub_avr_clustering_coef)
                avr_BetweenSubject.loc[i, 'eigenvec_(AnatomicalVar)'] = np.mean(acrosSub_avr_eigen_centrality)
                avr_BetweenSubject.loc[i, 'smallworldness_(AnatomicalVar)'] = np.mean(acrosSub_smallworld)
                avr_BetweenSubject.loc[i, 'avg_shortestPathLength_(AnatomicalVar)'] = np.mean(acrosSub_avrgShortestPathLength)

                avr_BetweenSubject.loc[i, 'iter']=f'iter_{rep}'
                avr_BetweenSubject.loc[i, 'session']= session
                avr_BetweenSubject.loc[i, 'acquisition']= acquisition

                i=i+1
    stdofavr_BetweenSubject=stdofavr_BetweenSubject.dropna()
    avr_BetweenSubject=avr_BetweenSubject.dropna()
    return stdofavr_BetweenSubject

In [ ]:
stdofavr_WithinSubject= stdofavrg_with(dfN)
stdofavr_BetweenSubject=stdofavrg_bet(dfN)
df_stdofavr_WithinSubject= stdofavrg_with(dfW)
df_stdofavr_BetweenSubject=stdofavrg_bet(dfW)

In [ ]:
# Filter the DataFrame
s = dfN[(dfN['repetition'] == 'rep-8') & 
        (dfN['acquisition'] == 'acq-RL') & 
        (dfN['session'] == '1')]['degree_centralities']

# Compute mean of each dictionary row
row_means = s.apply(lambda d: pd.Series(d).mean())

# Compute standard deviation of those means
overall_std = row_means.std()

print("Standard deviation of row means:", overall_std)
row_means

In [ ]:

df_stdofavr_WithinSubjecthc=pd.read_pickle('/home/ubuntu/Desktop/Thesis/overlap/allbatches/NumGmetricstd_WConf_batchhc.pkl')
df_stdofavr_BetweenSubjecthc=pd.read_pickle('/home/ubuntu/Desktop/Thesis/overlap/allbatches/AnatGmetricstd_WConf_batchhc.pkl')
df_stdofavr_WithinSubjecthcN=pd.read_pickle('/home/ubuntu/Desktop/Thesis/overlap/allbatches/NumGmetricstd_NoConf_batchhc.pkl')
df_stdofavr_BetweenSubjecthcN=pd.read_pickle('/home/ubuntu/Desktop/Thesis/overlap/allbatches/AnatGmetricstd_NoConf_batchhc.pkl')

In [ ]:
stdofavr_BetweenSubject[(stdofavr_BetweenSubject['session']=='1') & (stdofavr_BetweenSubject['acquisition']=='acq-RL')]

In [ ]:
stdofavr_WithinSubject1 = stdofavr_WithinSubject[
    (stdofavr_WithinSubject['session'] == '1') & 
    (stdofavr_WithinSubject['acquisition'] == 'acq-RL')
].drop(columns=['session', 'acquisition','smallworldness(numericalVar)','avg_shortestPathLength(numericalVar)'])

# Rename specific columns
stdofavr_WithinSubject1 = stdofavr_WithinSubject1.rename(columns={'degree_(numericalVar)': 'degree_(numericalVar1)', 'betweeness_(numericalVar)': 'betweeness_(numericalVar1)' , 'clusteringcoef_(numericalVar)':'clusteringcoef_(numericalVar1)'   , 'eigenvec_(numericalVar)':'eigenvec_(numericalVar1)'  })

stdofavr_BetweenSubject1 = stdofavr_BetweenSubject[
    (stdofavr_BetweenSubject['session'] == '1') & 
    (stdofavr_BetweenSubject['acquisition'] == 'acq-RL')
].drop(columns=['session', 'acquisition','smallworldness_(AnatomicalVar)','avg_shortestPathLength_(AnatomicalVar)'])

stdofavr_BetweenSubject1 = stdofavr_BetweenSubject1.rename(columns={'degree_(AnatomicalVar)': 'degree_(AnatomicalVar1)', 'betweeness_(AnatomicalVar)': 'betweeness_(AnatomicalVar1)' , 'clusteringcoef_(AnatomicalVar)':'clusteringcoef_(AnatomicalVar1)'   , 'eigenvec_(AnatomicalVar)':'eigenvec_(AnatomicalVar1)'  })


In [ ]:
stdofavr_WithinSubjectWconf = df_stdofavr_WithinSubject[
    (df_stdofavr_WithinSubject['session'] == '1') & 
    (df_stdofavr_WithinSubject['acquisition'] == 'acq-RL')
].drop(columns=['session', 'acquisition','smallworldness(numericalVar)','avg_shortestPathLength(numericalVar)'])
stdofavr_BetweenSubjectWconf = df_stdofavr_BetweenSubject[
    (df_stdofavr_BetweenSubject['session'] == '1') & 
    (df_stdofavr_BetweenSubject['acquisition'] == 'acq-RL')
].drop(columns=['session', 'acquisition','smallworldness_(AnatomicalVar)','avg_shortestPathLength_(AnatomicalVar)'])

stdofavr_WithinSubjectWconf = stdofavr_WithinSubjectWconf.rename(columns={'degree_(numericalVar)': 'degree_(numericalVarW)', 'betweeness_(numericalVar)': 'betweeness_(numericalVarW)' , 'clusteringcoef_(numericalVar)':'clusteringcoef_(numericalVarW)'   , 'eigenvec_(numericalVar)':'eigenvec_(numericalVarW)'  })
stdofavr_BetweenSubjectWconf = stdofavr_BetweenSubjectWconf.rename(columns={'degree_(AnatomicalVar)': 'degree_(AnatomicalVarW)', 'betweeness_(AnatomicalVar)': 'betweeness_(AnatomicalVarW)' , 'clusteringcoef_(AnatomicalVar)':'clusteringcoef_(AnatomicalVarW)'   , 'eigenvec_(AnatomicalVar)':'eigenvec_(AnatomicalVarW)'  })
# ####################3
stdofavr_WithinSubjectWconfhc = df_stdofavr_WithinSubjecthc[
    (df_stdofavr_WithinSubjecthc['session'] == '1') & 
    (df_stdofavr_WithinSubjecthc['acquisition'] == 'acq-RL')
].drop(columns=['session', 'acquisition','smallworldness(numericalVar)','avg_shortestPathLength(numericalVar)'])
stdofavr_BetweenSubjectWconfhc = df_stdofavr_BetweenSubjecthc[
    (df_stdofavr_BetweenSubjecthc['session'] == '1') & 
    (df_stdofavr_BetweenSubjecthc['acquisition'] == 'acq-RL')
].drop(columns=['session', 'acquisition','smallworldness_(AnatomicalVar)','avg_shortestPathLength_(AnatomicalVar)'])

stdofavr_WithinSubjectWconfhc= stdofavr_WithinSubjectWconfhc.rename(columns={'degree_(numericalVar)': 'degree_(numericalVarWhc)', 'betweeness_(numericalVar)': 'betweeness_(numericalVarWhc)' , 'clusteringcoef_(numericalVar)':'clusteringcoef_(numericalVarWhc)'   , 'eigenvec_(numericalVar)':'eigenvec_(numericalVarWhc)'  })
stdofavr_BetweenSubjectWconfhc = stdofavr_BetweenSubjectWconfhc.rename(columns={'degree_(AnatomicalVar)': 'degree_(AnatomicalVarWhc)', 'betweeness_(AnatomicalVar)': 'betweeness_(AnatomicalVarWhc)' , 'clusteringcoef_(AnatomicalVar)':'clusteringcoef_(AnatomicalVarWhc)'   , 'eigenvec_(AnatomicalVar)':'eigenvec_(AnatomicalVarWhc)'  })
################################################
stdofavr_WithinSubjectWconfhcN = df_stdofavr_WithinSubjecthcN[
    (df_stdofavr_WithinSubjecthcN['session'] == '1') & 
    (df_stdofavr_WithinSubjecthcN['acquisition'] == 'acq-RL')
].drop(columns=['session', 'acquisition','smallworldness(numericalVar)','avg_shortestPathLength(numericalVar)'])
stdofavr_BetweenSubjectWconfhcN = df_stdofavr_BetweenSubjecthcN[
    (df_stdofavr_BetweenSubjecthcN['session'] == '1') & 
    (df_stdofavr_BetweenSubjecthcN['acquisition'] == 'acq-RL')
].drop(columns=['session', 'acquisition','smallworldness_(AnatomicalVar)','avg_shortestPathLength_(AnatomicalVar)'])

stdofavr_WithinSubjectWconfhcN= stdofavr_WithinSubjectWconfhcN.rename(columns={'degree_(numericalVar)': 'degree_(numericalVarWhcN)', 'betweeness_(numericalVar)': 'betweeness_(numericalVarWhcN)' , 'clusteringcoef_(numericalVar)':'clusteringcoef_(numericalVarWhcN)'   , 'eigenvec_(numericalVar)':'eigenvec_(numericalVarWhcN)'  })
stdofavr_BetweenSubjectWconfhcN = stdofavr_BetweenSubjectWconfhcN.rename(columns={'degree_(AnatomicalVar)': 'degree_(AnatomicalVarWhcN)', 'betweeness_(AnatomicalVar)': 'betweeness_(AnatomicalVarWhcN)' , 'clusteringcoef_(AnatomicalVar)':'clusteringcoef_(AnatomicalVarWhcN)'   , 'eigenvec_(AnatomicalVar)':'eigenvec_(AnatomicalVarWhcN)'  })


In [ ]:
# import plotly.graph_objects as go
# import plotly.express as px
# import pandas as pd

# # List of metrics
# metrics = ['degree', 'betweeness', 'eigenvec', 'clusteringcoef']
# stdofavr_BetweenSubjectWconf
# # Create subplots
# from plotly.subplots import make_subplots
# fig = make_subplots(
#     rows=len(metrics), 
#     cols=1,  
#     vertical_spacing=0.05,     
#     subplot_titles=[
#         f"Degree Centrality with anatomical mean = {avr_BetweenSubject['degree_(AnatomicalVar)'].mean()}",
#         f"Betweeness Centrality with anatomical mean = {avr_BetweenSubject['betweeness_(AnatomicalVar)'].mean()}",
#         f"Eigen Vector Centrality with anatomical mean = {avr_BetweenSubject['eigenvec_(AnatomicalVar)'].mean()}",
#         f"Clustering Coefficient with anatomical mean = {avr_BetweenSubject['clusteringcoef_(AnatomicalVar)'].mean()}"
#     ]
# )



# for i, metric in enumerate(metrics):
#     # Extract columns for each metric
#     columns_to_extract_bet1 = ['iter'] + [col for col in stdofavr_BetweenSubjectWconf .columns if metric in col]
#     columns_to_extract_with1 = ['subject'] + [col for col in stdofavr_WithinSubjectWconf .columns if metric in col]
    
#     statics_bet1 = stdofavr_BetweenSubjectWconf[columns_to_extract_bet1]
#     statics_with1 = stdofavr_WithinSubjectWconf[columns_to_extract_with1]

#     columns_to_extract_bet = ['iter'] + [col for col in stdofavr_BetweenSubject1.columns if metric in col]
#     columns_to_extract_with = ['subject'] + [col for col in stdofavr_WithinSubject1.columns if metric in col]
    
#     statics_bet = stdofavr_BetweenSubject1[columns_to_extract_bet]
#     statics_with = stdofavr_WithinSubject1[columns_to_extract_with]
#     # print(statics_bet)
#     # # Calculate standard deviations
#     # std_devs_bet = statics_bet.drop(['iter'], axis=1).std()
#     # std_devs_with = statics_with.drop(['subject'], axis=1).std()
    
#     # statics_bet = pd.concat([statics_bet, pd.DataFrame([std_devs_bet], columns=std_devs_bet.index)], ignore_index=True)
#     # statics_bet.at[statics_bet.index[-1], 'iter'] = 'overall_std'

#     # statics_with = pd.concat([statics_with, pd.DataFrame([std_devs_with], columns=std_devs_with.index)], ignore_index=True)
#     # statics_with.at[statics_with.index[-1], 'subject'] = 'overall_std'
    
#     # Melt the DataFrames
#     bet_melted1 = statics_bet1.melt(id_vars=['iter'], var_name='metric', value_name='value')
#     with_melted1 = statics_with1.melt(id_vars=['subject'], var_name='metric', value_name='value')

#     bet_melted = statics_bet.melt(id_vars=['iter'], var_name='metric', value_name='value')
#     with_melted = statics_with.melt(id_vars=['subject'], var_name='metric', value_name='value')   
#     # Plot box plots for Between Subject
#     fig.add_trace(
#         go.Box(
#             x=bet_melted1['metric'], 
#             y=bet_melted1['value'],
#             showlegend=False,
#             name=f"Between Subject {metric.capitalize()}",
#             boxpoints='all',  # Add stripplot-like points
#             jitter=0.3,
#             pointpos=0,
#             marker=dict(
#                         color='rgb(34,139,34)'),
#             width=0.2
#         ),
#         row=i + 1, col=1
#     )
#         # Plot box plots for Between Subject
#     fig.add_trace(
#         go.Box(
#             x=bet_melted['metric'], 
#             y=bet_melted['value'],
#             showlegend=False,
#             name=f"Between Subject {metric.capitalize()}",
#             boxpoints='all',  # Add stripplot-like points
#             jitter=0.3,
#             pointpos=0,
#             marker=dict(
#                         color='rgb(34,139,34)'),
#             width=0.2
#         ),
#         row=i + 1, col=1
#     )
#     # Plot box plots for Within Subject
#     fig.add_trace(
#         go.Box(
#             x=with_melted1['metric'], 
#             y=with_melted1['value'],
#             showlegend=False,
#             width=0.2,  # Smaller box width,
#             name=f"Within Subject {metric.capitalize()}",
#             boxpoints='suspectedoutliers', # only suspected outliers
#             jitter=0.3,
#             pointpos=0.1,
#             marker=dict(
#                         color='rgb(8,81,156)',
#                         outliercolor='rgb(8,81,156)',
#                         line=dict(
#                             outliercolor='rgb(8,81,156)',
#                             outlierwidth=2)),
            
#             text=with_melted1['subject'],  # Hover text showing subject ID
#             hovertemplate=(
#                 "<b>Subject:</b> %{text}<br>"  # Display subject ID
#                 "<b>Metric:</b> %{x}<br>"      # Display metric
#                 "<b>Value:</b> %{y}<br>"       # Display value
#                 "<extra></extra>"              # Remove default extra info
#         )
#         ),
#         row=i + 1, col=1
#     )
#         # Plot box plots for Within Subject
#     fig.add_trace(
#         go.Box(
#             x=with_melted['metric'], 
#             y=with_melted['value'],
#             showlegend=False,
#             width=0.2,  # Smaller box width,
#             name=f"Within Subject {metric.capitalize()}",
#             boxpoints='suspectedoutliers', # only suspected outliers
#             jitter=0.3,
#             pointpos=0.1,
#             marker=dict(
#                         color='rgb(8,81,156)',
#                         outliercolor='rgb(8,81,156)',
#                         line=dict(
#                             outliercolor='rgb(8,81,156)',
#                             outlierwidth=2)),
            
#             text=with_melted['subject'],  # Hover text showing subject ID
#             hovertemplate=(
#                 "<b>Subject:</b> %{text}<br>"  # Display subject ID
#                 "<b>Metric:</b> %{x}<br>"      # Display metric
#                 "<b>Value:</b> %{y}<br>"       # Display value
#                 "<extra></extra>"              # Remove default extra info
#         )
#         ),
#         row=i + 1, col=1
#     )

# # Update layout
# fig.update_layout(
#     title="Metrics with Anatomical and Numerical Variability",
#     height=150 * len(metrics),
#     width=900,
#     showlegend=True,
#     legend_title="Legend",
#     margin=dict(l=50, r=50, t=50, b=100)
# )

# # Iterate through rows to remove x-axis tick labels for all subplots
# for row in range(1, len(metrics) + 1):
#     fig.update_xaxes(showticklabels=False, row=row, col=1)

# fig.update_xaxes(
#     title_text="Anatomical Variability with confounds" + "&nbsp;" *10 +  "Anatomical Variability No confounds" + "&nbsp;" *10  + "Numerical Variability with confounds" + "&nbsp;" *10 + "Numerical Variability No confounds",
#     row=len(metrics),
#     col=1,
#     side='bottom',
#     title_font=dict(size=11)  # Adjust the font size here
# )
# # Remove annotations for avr_bet
# fig.show()

In [ ]:
import plotly.graph_objects as go
import pandas as pd
from plotly.subplots import make_subplots

# List of metrics
metrics = ['degree', 'betweeness', 'eigenvec', 'clusteringcoef']

# Create subplots
fig = make_subplots(
    rows=len(metrics), 
    cols=1,  
    vertical_spacing=0.05,     
    # subplot_titles=[
    #     f"Degree Centrality with anatomical mean = {avr_BetweenSubject['degree_(AnatomicalVar)'].mean()}",
    #     f"Betweeness Centrality with anatomical mean = {avr_BetweenSubject['betweeness_(AnatomicalVar)'].mean()}",
    #     f"Eigen Vector Centrality with anatomical mean = {avr_BetweenSubject['eigenvec_(AnatomicalVar)'].mean()}",
    #     f"Clustering Coefficient with anatomical mean = {avr_BetweenSubject['clusteringcoef_(AnatomicalVar)'].mean()}"
    # ]
)

for i, metric in enumerate(metrics):
    # Extract columns
    datasets = [
        ('BetweenSubjectWconf', stdofavr_BetweenSubjectWconf),
        ('BetweenSubject1', stdofavr_BetweenSubject1),
        ('BetweenSubjectWconfhc', stdofavr_BetweenSubjectWconfhc),
        ('BetweenSubjectWconfhcN', stdofavr_BetweenSubjectWconfhcN),
        ('WithinSubjectWconf', stdofavr_WithinSubjectWconf),
        ('WithinSubject1', stdofavr_WithinSubject1),
        ('WithinSubjectWconfhc', stdofavr_WithinSubjectWconfhc),
        ('WithinSubjectWconfhcN', stdofavr_WithinSubjectWconfhcN)
    ]
    
    colors = ['rgb(34,139,34)', 'rgb(8,81,156)', 'rgb(255,69,0)', 'rgb(75,0,130)']
    
    for j, (name, df) in enumerate(datasets):
        id_var = 'iter' if 'Between' in name else 'subject'
        columns_to_extract = [id_var] + [col for col in df.columns if metric in col]
        statics = df[columns_to_extract]
        melted = statics.melt(id_vars=[id_var], var_name='metric', value_name='value')
        
        fig.add_trace(
            go.Box(
                x=melted['metric'],
                y=melted['value'],
                showlegend=(i == 0),
                name=f"{name} {metric.capitalize()}",
                boxpoints='suspectedoutliers' if 'Within' in name else 'all',
                jitter=0.3,
                pointpos=0,
                marker=dict(color=colors[j % len(colors)]),
                width=0.2
            ),
            row=i + 1, col=1
        )

# Update layout
fig.update_layout(
    title="Metrics with Anatomical and Numerical Variability",
    height=200 * len(metrics),
    width=1300,
    showlegend=True,
    legend_title="Legend",
    margin=dict(l=50, r=50, t=50, b=100)
)

# Hide x-axis labels except for the last row
for row in range(1, len(metrics)+1):
    fig.update_xaxes(showticklabels=False, row=row, col=1)

fig.update_xaxes(
    title_text="Wconf__anat_PPMI" + "\xa0" * 10 + "Noconf_anat_PPMI"  + "\xa0" * 10 + "Wconf__anat_HC" + "\xa0" * 10 + "Noconf__anat_HC" + "\xa0" * 10 + 
    "Wconf__num_PPMI" + "\xa0" * 10 + "Noconf_num_PPMI"  + "\xa0" * 10 + "Wconf__num_HC" + "\xa0" * 10 + "Noconf__num_HC"
               ,
    row=len(metrics),
    col=1,
    side='bottom',
    title_font=dict(size=11)
)

fig.show()


In [ ]:
globalMettric_WithinSubject = stdofavr_WithinSubject[
    (stdofavr_WithinSubject['session'] == '1') & 
    (stdofavr_WithinSubject['acquisition'] == 'acq-RL')
].drop(columns=['session', 'acquisition',	'degree_(numericalVar)','betweeness_(numericalVar)','clusteringcoef_(numericalVar)','eigenvec_(numericalVar)'])


globalMettric_BetweenSubject= stdofavr_BetweenSubject[
    (stdofavr_BetweenSubject['session'] == '1') & 
    (stdofavr_BetweenSubject['acquisition'] == 'acq-RL')].drop(columns=['session', 'acquisition','degree_(AnatomicalVar)','betweeness_(AnatomicalVar)',	'clusteringcoef_(AnatomicalVar)','eigenvec_(AnatomicalVar)'])


globalMettric_WithinSubjectWconf = df_stdofavr_WithinSubject[
    (df_stdofavr_WithinSubject['session'] == '1') & 
    (df_stdofavr_WithinSubject['acquisition'] == 'acq-RL')
].drop(columns=['session', 'acquisition',	'degree_(numericalVar)','betweeness_(numericalVar)','clusteringcoef_(numericalVar)','eigenvec_(numericalVar)'])


globalMettric_BetweenSubjectWconf= df_stdofavr_BetweenSubject[
    (df_stdofavr_BetweenSubject['session'] == '1') & 
    (df_stdofavr_BetweenSubject['acquisition'] == 'acq-RL')].drop(columns=['session', 'acquisition','degree_(AnatomicalVar)','betweeness_(AnatomicalVar)',	'clusteringcoef_(AnatomicalVar)','eigenvec_(AnatomicalVar)'])
globalMettric_WithinSubjectWconf = globalMettric_WithinSubjectWconf.rename(columns={'smallworldness(numericalVar)': 'smallworldness(numericalVarW)', 'avg_shortestPathLength(numericalVar)': 'avg_shortestPathLength(numericalVarW)'   })
globalMettric_BetweenSubjectWconf = globalMettric_BetweenSubjectWconf.rename(columns={'smallworldness_(AnatomicalVar)': 'smallworldness_(AnatomicalVarW)', 'avg_shortestPathLength_(AnatomicalVar)': 'avg_shortestPathLength_(AnatomicalVarW)' })


In [ ]:
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
from plotly.subplots import make_subplots

# List of metrics
metrics = ['smallworldness', 'avg_shortestPathLength']

# Create subplots
fig = make_subplots(
    rows=len(metrics), 
    cols=1,  
    vertical_spacing=0.1,     
    subplot_titles=[
        f"Small worldness with anatomical mean = {avr_BetweenSubject['smallworldness_(AnatomicalVar)'].mean()}",
        f"Avrg shortest path with anatomical mean = {avr_BetweenSubject['avg_shortestPathLength_(AnatomicalVar)'].mean()}"
    ]
)



for i, metric in enumerate(metrics):

        # Extract columns for each metric
    columns_to_extract_bet1 = ['iter'] + [col for col in globalMettric_BetweenSubjectWconf.columns if metric in col]
    columns_to_extract_with1 = ['subject'] + [col for col in globalMettric_WithinSubjectWconf.columns if metric in col]
    
    statics_bet1 = globalMettric_BetweenSubjectWconf[columns_to_extract_bet1]
    statics_with1 = globalMettric_WithinSubjectWconf[columns_to_extract_with1]
    # Extract columns for each metric
    columns_to_extract_bet = ['iter'] + [col for col in globalMettric_BetweenSubject.columns if metric in col]
    columns_to_extract_with = ['subject'] + [col for col in globalMettric_WithinSubject.columns if metric in col]
    
    statics_bet = globalMettric_BetweenSubject[columns_to_extract_bet]
    statics_with = globalMettric_WithinSubject[columns_to_extract_with]
    
    # # Calculate standard deviations
    # std_devs_bet = statics_bet.drop(['iter'], axis=1).std()
    # std_devs_with = statics_with.drop(['subject'], axis=1).std()
    
    # statics_bet = pd.concat([statics_bet, pd.DataFrame([std_devs_bet], columns=std_devs_bet.index)], ignore_index=True)
    # statics_bet.at[statics_bet.index[-1], 'iter'] = 'overall_std'

    # statics_with = pd.concat([statics_with, pd.DataFrame([std_devs_with], columns=std_devs_with.index)], ignore_index=True)
    # statics_with.at[statics_with.index[-1], 'subject'] = 'overall_std'
    
    # Melt the DataFrames
    bet_melted1 = statics_bet1.melt(id_vars=['iter'], var_name='metric', value_name='value')
    with_melted1 = statics_with1.melt(id_vars=['subject'], var_name='metric', value_name='value')
    # Melt the DataFrames
    bet_melted = statics_bet.melt(id_vars=['iter'], var_name='metric', value_name='value')
    with_melted = statics_with.melt(id_vars=['subject'], var_name='metric', value_name='value')
    # Plot box plots for Between Subject
    fig.add_trace(
        go.Box(
            x=bet_melted1['metric'], 
            y=bet_melted1['value'],
            showlegend=False,
            name=f"Between Subject With confounds{metric.capitalize()}",
            boxpoints='all',  # Add stripplot-like points
            jitter=0.3,
            pointpos=0,
            marker=dict(
                        color='rgb(34,139,34)'),
            width=0.2
        ),
        row=i + 1, col=1
    )
     # Plot box plots for Between Subject
    fig.add_trace(
        go.Box(
            x=bet_melted['metric'], 
            y=bet_melted['value'],
            showlegend=False,
            name=f"Between Subject {metric.capitalize()}",
            boxpoints='all',  # Add stripplot-like points
            jitter=0.3,
            pointpos=0,
            marker=dict(
                        color='rgb(34,139,34)'),
            width=0.2
        ),
        row=i + 1, col=1
    )   
    # Plot box plots for Within Subject
    fig.add_trace(
        go.Box(
            x=with_melted1['metric'], 
            y=with_melted1['value'],
            showlegend=False,
            width=0.2,  # Smaller box width,
            name=f"Within Subject With confounds{metric.capitalize()}",
            boxpoints='suspectedoutliers', # only suspected outliers
            jitter=0.3,
            pointpos=0.1,
            marker=dict(
                        color='rgb(8,81,156)',
                        outliercolor='rgb(8,81,156)',
                        line=dict(
                            outliercolor='rgb(8,81,156)',
                            outlierwidth=2)),
            
            text=with_melted1['subject'],  # Hover text showing subject ID
            hovertemplate=(
                "<b>Subject:</b> %{text}<br>"  # Display subject ID
                "<b>Metric:</b> %{x}<br>"      # Display metric
                "<b>Value:</b> %{y}<br>"       # Display value
                "<extra></extra>"              # Remove default extra info
        )
        ),
        row=i + 1, col=1
    )
    # Plot box plots for Within Subject
    fig.add_trace(
        go.Box(
            x=with_melted['metric'], 
            y=with_melted['value'],
            showlegend=False,
            width=0.2,  # Smaller box width,
            name=f"Within Subject {metric.capitalize()}",
            boxpoints='suspectedoutliers', # only suspected outliers
            jitter=0.3,
            pointpos=0.1,
            marker=dict(
                        color='rgb(8,81,156)',
                        outliercolor='rgb(8,81,156)',
                        line=dict(
                            outliercolor='rgb(8,81,156)',
                            outlierwidth=2)),
            
            text=with_melted['subject'],  # Hover text showing subject ID
            hovertemplate=(
                "<b>Subject:</b> %{text}<br>"  # Display subject ID
                "<b>Metric:</b> %{x}<br>"      # Display metric
                "<b>Value:</b> %{y}<br>"       # Display value
                "<extra></extra>"              # Remove default extra info
        )
        ),
        row=i + 1, col=1
    )

# Update layout
fig.update_layout(
    title="Metrics with Anatomical and Numerical Variability",
    height=200* len(metrics),
    width=900,
    showlegend=True,
    legend_title="Legend",
    margin=dict(l=50, r=50, t=50, b=100)
)

# Iterate through rows to remove x-axis tick labels for all subplots
for row in range(1, len(metrics) + 1):
    fig.update_xaxes(showticklabels=False, row=row, col=1)

fig.update_xaxes(
    #title_text="Anatomical Variability with confounds" + "&nbsp;" *10 +  "Anatomical Variability No confounds" + "&nbsp;" *10  + "Numerical Variability with confounds" + "&nbsp;" *10 + "Numerical Variability No confounds",
    row=len(metrics),
    col=1,
    title_font=dict(size=11)  # Adjust the font size here

)

# Remove annotations for avr_bet
fig.show()

In [ ]:
column_names = ['subject_id', 'session','acquisition','rep', 'repp', 'region_label', 'dice_coef']

# Load the CSV file into a DataFrame with specified column names
df_ROI_atlasinT1W = pd.read_csv('/home/ubuntu/Desktop/Thesis/overlap/allbatches/dice_reg_inverse.csv', header=None, names=column_names)
df_ROI_atlasinT1W = df_ROI_atlasinT1W.drop(0).reset_index(drop=True)
df_ROI_atlasinT1W
df_ROI_atlasinT1W['region_label'] = df_ROI_atlasinT1W['region_label'].astype('int64')
df_ROI_atlasinT1W['session'] = df_ROI_atlasinT1W['session'].astype('int64')



# Load the CSV file into a DataFrame with specified column names
df_ROI_atlasinT1W2Bold = pd.read_csv('/home/ubuntu/Desktop/Thesis/overlap/allbatches/dice_coreg_inverse.csv', header=None, names=column_names)
# df1 = pd.read_csv('/home/ubuntu/Desktop/Thesis/overlap/batch2/dice_coreg_inverseadd.csv', header=None, names=column_names)
# Drop the first row (column names) from the second DataFrame
# df1 = df1[1:].reset_index(drop=True)
# df_ROI_atlasinT1W2Bold=pd.concat([df, df1], ignore_index=True)
df_ROI_atlasinT1W2Bold = df_ROI_atlasinT1W2Bold.drop(0).reset_index(drop=True)
df_ROI_atlasinT1W2Bold['region_label'] = df_ROI_atlasinT1W2Bold['region_label'].astype('int64')
df_ROI_atlasinT1W2Bold['session'] = df_ROI_atlasinT1W2Bold['session'].astype('int64')



In [ ]:
import numpy as np

def create_stackmatrix_OfDice(result_df):
    num_sub =len(result_df['subject_id'].unique()) # number of subjects
    num_regions = 100  # number of regions
    num_MCApairs = 45  # number of dice coefficient combinations
    # Initialize stack matrix to hold the data for each subject
    stack_matrix = np.empty((num_sub, num_regions, num_MCApairs))
    index_to_subject = {}

    for index, sub in enumerate(result_df['subject_id'].unique()[:num_sub]):
        Dice_coef = np.zeros((num_regions, num_MCApairs))
        # Store the mapping of index to subject ID
        index_to_subject[index] = sub  
        # Filter the DataFrame for the current subject
        subject_data = result_df[result_df['subject_id'] == sub]
        # print(sub, subject_data)
        for region in range(1, num_regions + 1):
            # Get the dice coefficients for the current region and subject
            region_data = subject_data[subject_data['region_label'] == region]['dice_coef'].values.tolist()[0]
            # Ensure we have the expected number of coefficients (45) for each region
            if len(region_data) == num_MCApairs:
                Dice_coef[region - 1, :] = region_data
            else:
                print(f"Warning: Region {region} for subject {sub} has {len(region_data)} values instead of {num_MCApairs}.")
        
        # Assign the computed Dice_coef matrix for the current subject to stack_matrix
        stack_matrix[index] = Dice_coef
    return stack_matrix,index_to_subject


In [ ]:
result_df_atlasinT1W=df_ROI_atlasinT1W[(df_ROI_atlasinT1W['session']==1) & (df_ROI_atlasinT1W['acquisition']=='acq-RL')] 
result_df_atlasinT1W=result_df_atlasinT1W.drop(['session', 'acquisition'], axis=1)
# Group the DataFrame by subject ID and aggregate the region_label and dice_coef into lists
result_df_atlasinT1W = result_df_atlasinT1W.groupby(['subject_id', 'region_label']).agg({

    'dice_coef': list
}).reset_index()

result_df_ROI_atlasinT1W2Bold=df_ROI_atlasinT1W2Bold[(df_ROI_atlasinT1W2Bold['session']==1) & (df_ROI_atlasinT1W2Bold['acquisition']=='acq-RL')] 
# Group the DataFrame by subject ID and aggregate the region_label and dice_coef into lists
result_df_ROI_atlasinT1W2Bold = result_df_ROI_atlasinT1W2Bold.groupby(['subject_id', 'region_label']).agg({

    'dice_coef': list
}).reset_index()


In [ ]:
import nibabel as nib
import numpy as np
import pandas as pd

# Load the Schaefer atlas NIfTI file
atlas_path = "/home/ubuntu/Desktop/Thesis/overlap/tpl-MNI152NLin2009cAsym_res-01_atlas-Schaefer2018_desc-100Parcels7Networks_dseg.nii.gz"
atlas_img = nib.load(atlas_path)
atlas_data = atlas_img.get_fdata()

# Load the TSV file with atlas labels
tsv_file_path = "/home/ubuntu/Desktop/Thesis/overlap/tpl-MNI152NLin2009cAsym_atlas-Schaefer2018_desc-100Parcels7Networks_dseg.tsv"
atlas_df = pd.read_csv(tsv_file_path, delimiter='\t')

# Extract labels
atlas_labels = atlas_df['name'].tolist()

# Get unique region labels (excluding background if label = 0)
unique_labels = np.unique(atlas_data)
unique_labels = unique_labels[unique_labels > 0]  # Remove background (0)

# Compute voxel count per region
region_sizes = {int(label): np.sum(atlas_data == label) for label in unique_labels}

# Get voxel size in mm³ (multiply dimensions)
voxel_volume = np.prod(atlas_img.header.get_zooms())  # Voxel size in mm³

# Convert voxel count to mm³
region_sizes_mm3 = {label: count * voxel_volume for label, count in region_sizes.items()}

# Create a DataFrame for clarity
region_size_df = pd.DataFrame({
    'Region': atlas_labels,
    'Label': list(region_sizes.keys()),
    'Voxel_Count': list(region_sizes.values()),
    'Size_mm3': list(region_sizes_mm3.values())
})

# Display region sizes
print(region_size_df)


# Sort regions by size (largest to smallest)
region_size_df_sorted = region_size_df.sort_values(by="Size_mm3", ascending=False)

# Display sorted region sizes
print(region_size_df_sorted)



In [ ]:
#select one acusition for subject
stack_matrix_T1,indexT=create_stackmatrix_OfDice(result_df_atlasinT1W)
stack_matrix_bold,indexB=create_stackmatrix_OfDice(result_df_ROI_atlasinT1W2Bold)

Numerical_min_T1=  np.min(stack_matrix_T1,axis=2)
numerical_min_bold=np.min(stack_matrix_bold,axis=2)
min_T1=np.mean(Numerical_min_T1,axis=0)
min_bold=np.mean(numerical_min_bold,axis=0)


In [ ]:
#removing worse subjects
# Assuming stack_matrix is your (112, 100, 45) matrix
average_per_subject = np.mean(stack_matrix_T1, axis=(1, 2))  # Shape will be (112,)
top_min_indices = np.argsort(average_per_subject)[:10]
print(top_min_indices)
average_per_subjectb = np.mean(stack_matrix_bold, axis=(1, 2))  # Shape will be (112,)
btop_min_indices = np.argsort(average_per_subjectb)[:10]
print(btop_min_indices)

filtered_indexT = {k: v for k, v in indexT.items() if k not in top_min_indices}
filtered_indexB = {k: v for k, v in indexB.items() if k not in btop_min_indices}

stack_matrix_T1_filtered = np.delete(stack_matrix_T1, top_min_indices, axis=0)
stack_matrix_bold_filtered = np.delete(stack_matrix_bold, btop_min_indices, axis=0)
Numerical_min_T1_filtered=  np.min(stack_matrix_T1_filtered,axis=2)
numerical_min_bold_filtered=np.min(stack_matrix_bold_filtered,axis=2)
min_T1_filtered=np.mean(Numerical_min_T1_filtered,axis=0)
min_bold_filtered=np.mean(numerical_min_bold_filtered,axis=0)


In [ ]:
print(stack_matrix_bold.shape)

In [ ]:
from nilearn import plotting, datasets, image
import numpy as np
import nibabel as nib
def Return_Metric_Img(metric_matrix):
    # Load an example atlas, e.g., MNI152 template
    # atlas = datasets.fetch_atlas_schaefer_2018(n_rois=100, resolution_mm=2)

    # # Assume `metric_values` is a list or array of values for the 100 regions
    metric_values = metric_matrix # Replace with your actual metric values

    # # Load the brain atlas and get the atlas regions
    # atlas_img = nib.load(atlas.maps)
    # Load your atlas NIfTI file
    atlas='/home/ubuntu/Desktop/Thesis/overlap/tpl-MNI152NLin2009cAsym_res-01_atlas-Schaefer2018_desc-100Parcels7Networks_dseg.nii.gz'
    # Define the path to your TSV file
    tsv_file_path = "/home/ubuntu/Desktop/Thesis/overlap/tpl-MNI152NLin2009cAsym_atlas-Schaefer2018_desc-100Parcels7Networks_dseg.tsv"
    atlas_img=nib.load(atlas)
    atlas_data = atlas_img.get_fdata()
    # Create an empty image with the same shape as the atlas
    metric_img_data = np.zeros(atlas_img.shape)

    # Assign metric values to the atlas regions
    for i, region in enumerate(np.unique(atlas_img.get_fdata())):
        if region != 0:  # Ignore the background (region 0)
            metric_img_data[atlas_img.get_fdata() == region] = metric_values[i-1]  # i-1 to skip background

    # Create a new NIfTI image with metric values
    metric_img = nib.Nifti1Image(metric_img_data, atlas_img.affine)
    return metric_img


In [ ]:
from nilearn import plotting, datasets
import matplotlib.pyplot as plt
import numpy as np
import textwrap


# Assuming you have two different metrics as images
metric_img1 = Return_Metric_Img(min_T1)
metric_img2 = Return_Metric_Img(min_bold)
# vmin = min(np.min(min_T1), np.min(min_bold))
vmin=0.85
vmax = max(np.max(min_T1), np.max(min_bold))

print(np.min(min_T1), np.min(min_bold), np.max(min_T1), np.max(min_bold))
# Create a subplot with two figures (1 row, 2 columns)
fig, axes = plt.subplots(2, 1, figsize=(20, 12))

# Set color range and colormap
vmax = 1
cmap = "Reds_r"

# Plot the first metric with an individual colorbar
plotting.plot_stat_map(
    metric_img1,
    bg_img=datasets.load_mni152_template(),
    display_mode='ortho',
    cut_coords=(10, 0, 0),
    title='average dice scores of Atlas_InT1W ',
    cmap=cmap,
    axes=axes[0],
    vmin=0.85, #np.min(min_T1),
    vmax=vmax,
    colorbar=False
)

# Plot the second metric with an individual colorbar
plotting.plot_stat_map(
    metric_img2,
    bg_img=datasets.load_mni152_template(),
    display_mode='ortho',
    cut_coords=(10, 0, 0),
    title='average dice scores of Atlas_InBold',
    cmap=cmap,
    axes=axes[1],
    vmin=0.85, #np.min(min_bold),
    vmax=vmax,
    colorbar=False
)
# Add a common colorbar for both plots with extended tick marks
cbar_ax = fig.add_axes([1, 0.25, 0.02, 0.5])
norm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin=vmin, vmax=vmax))

# Generate custom tick marks across the color range
ticks = np.linspace(vmin, vmax, 15)  # Adjust the number of ticks here for finer granularity
cbar = fig.colorbar(norm, cax=cbar_ax, label='Dice scores', ticks=ticks)
# Show the plots
plt.tight_layout()

# Define the text and wrap it to a specific width
text = ("Figure 1: Dice scores were computed for each region across each pair of MCA runs. The minimum Dice scores were identified and then the averages were calculated across subjects for each region. The figure shows the average Dice scores for the registered atlas in subject space (top) and BOLD space (bottom).")
wrapped_text = "\n".join(textwrap.wrap(text, width=180))  # Adjust the width as needed

# Add the wrapped text to the figure
plt.figtext(0.5, -0.05, wrapped_text, ha="center", fontsize=15,  wrap=True)
plt.show()


In [ ]:
from nilearn import plotting, datasets
import matplotlib.pyplot as plt
import numpy as np
import textwrap


# Assuming you have two different metrics as images
metric_img1_filtered = Return_Metric_Img(min_T1_filtered)
metric_img2_filtered = Return_Metric_Img(min_bold_filtered)
# vmin = min(np.min(min_T1_filtered), np.min(min_bold_filtered))
# vmax = max(np.max(min_T1_filtered), np.max(min_bold_filtered))
# vmin = min(np.min(min_T1), np.min(min_bold))
vmin=0.89
vmax = max(np.max(min_T1), np.max(min_bold))
print(np.min(min_T1_filtered), np.min(min_bold_filtered), np.max(min_T1_filtered), np.max(min_bold_filtered))
# Create a subplot with two figures (1 row, 2 columns)
fig, axes = plt.subplots(2, 1, figsize=(20, 12))

# Set color range and colormap
vmax = 1
cmap = "Reds_r"

# Plot the first metric with an individual colorbar
plotting.plot_stat_map(
    metric_img1_filtered,
    bg_img=datasets.load_mni152_template(),
    display_mode='ortho',
    cut_coords=(10, 0, 0),
    title='average dice scores of Atlas_InT1W ',
    cmap=cmap,
    axes=axes[0],
    vmin=0.89, #np.min(min_T1),
    vmax=vmax,
    colorbar=False
)

# Plot the second metric with an individual colorbar
plotting.plot_stat_map(
    metric_img2_filtered,
    bg_img=datasets.load_mni152_template(),
    display_mode='ortho',
    cut_coords=(10, 0, 0),
    title='average dice scores of Atlas_InBold',
    cmap=cmap,
    axes=axes[1],
    vmin=0.89, #np.min(min_bold),
    vmax=vmax,
    colorbar=False
)
# Add a common colorbar for both plots with extended tick marks
cbar_ax = fig.add_axes([1, 0.25, 0.02, 0.5])
norm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin=vmin, vmax=vmax))

# Generate custom tick marks across the color range
ticks = np.linspace(vmin, vmax, 15)  # Adjust the number of ticks here for finer granularity
cbar = fig.colorbar(norm, cax=cbar_ax, label='Dice scores', ticks=ticks)
# Show the plots
plt.tight_layout()

# Define the text and wrap it to a specific width
text = ("Figure 1: Dice scores were computed for each region across each pair of MCA runs. The minimum Dice scores were identified and then the averages were calculated across subjects for each region. The figure shows the average Dice scores for the registered atlas in subject space (top) and BOLD space (bottom).")
wrapped_text = "\n".join(textwrap.wrap(text, width=180))  # Adjust the width as needed

# Add the wrapped text to the figure
plt.figtext(0.5, -0.05, wrapped_text, ha="center", fontsize=15,  wrap=True)
plt.show()


$$
Numerical Variability:  \forall r \in \left[ 1 \hspace{0.1cm}, R \right],  \hspace{0.5cm} 
\Delta_{r} = \text{mean}_{s \in \left[ 1 \hspace{0.1cm}, S \right]} \left( \min_{(p,p^{'}),(p^{''},p^{'''}) \in \left[ 1 \hspace{0.1cm}, P \right]} 
\hspace{0.2cm} D \left( V_{r,s,(p,p^{'})}, V_{r,s,(p^{''},p^{'''})} \right) \right)
\newline

\hspace{0.5cm}R:\text{Number of Regions}, \hspace{0.1cm} S: \text{Number of Subjects}, \hspace{0.2cm} \text{and} \hspace{0.2cm} P: \text{Number of MCA pairs}\,(p,p^{'})


$$

In [ ]:
#Sorting based on region size
avrofMCApairs_T1=np.mean(stack_matrix_T1,axis=2)
avrofMCApairs_bold=np.mean(stack_matrix_bold,axis=2)
print(avrofMCApairs_T1.shape)

# Step 1: Get sorted region indices based on size
sorted_indices = region_size_df_sorted['Label'].index.values  # Get indices after sorting
sorted_region=region_size_df_sorted['Region'].values
print((sorted_region[0]))
# Step 2: Apply sorting to the columns of the matrices
sorted_avrofMCApairs_T1 = avrofMCApairs_T1[:, sorted_indices]  # Reorder columns
sorted_avrofMCApairs_bold = avrofMCApairs_bold[:, sorted_indices]  # Reorder columns



In [ ]:
import plotly.subplots as sp
import plotly.graph_objects as go
import numpy as np

# Determine shared color map limits
vmin = min(sorted_avrofMCApairs_T1.min(),sorted_avrofMCApairs_bold.min())
vmax = max(sorted_avrofMCApairs_T1.max(), sorted_avrofMCApairs_bold.max())
# Convert index dictionaries to ordered lists of subject IDs
indexT_labels = [indexT[i] for i in range(len(indexT))]  # Extract subject IDs from indexT dictionary
indexB_labels = [indexB[i] for i in range(len(indexB))]  # Extract subject IDs from indexB dictionary
sorted_region=region_size_df_sorted['Region'].values

# Create subplots
fig = sp.make_subplots(rows=1, cols=2, subplot_titles=["T1", "Bold"])

# Add first heatmap
fig.add_trace(
    go.Heatmap(
        z=sorted_avrofMCApairs_T1,
        y=indexT_labels,  # Use extracted subject IDs for BOLD
        x=sorted_region,
        colorscale='rainbow_r',
        zmin=vmin,
        zmax=vmax,
        colorbar=dict(title="Average", len=0.8),
    ),
    row=1,
    col=1
)

# Add second heatmap (share the same color scale)
fig.add_trace(
    go.Heatmap(
        z=sorted_avrofMCApairs_bold,
        y=indexB_labels,  # Use extracted subject IDs for BOLD
        x=sorted_region,
        colorscale='rainbow_r',
        zmin=vmin,
        zmax=vmax,
        showscale=False,  # Hide extra colorbar
    ),
    row=1,
    col=2
)

# Update y-axis labels
fig.update_yaxes(title_text="Subject ID", row=1, col=1)
fig.update_yaxes(title_text="Subject ID", row=1, col=2)

# Update layout
fig.update_layout(
    height=1000, width=2000,
    title_text="Heatmaps of average of MCApairs_T1 and average of MCApairs_bold (Shared Color Map)",
    coloraxis_colorbar=dict(len=0.8)
)

fig.show()


In [ ]:
##Remove 5 worse subjects

In [ ]:
#Sorting after removing worse subjects based on region size
avrofMCApairs_T1_filtered=np.mean(stack_matrix_T1_filtered,axis=2)
avrofMCApairs_bold_filtered=np.mean(stack_matrix_bold_filtered,axis=2)

# Step 1: Get sorted region indices based on size
sorted_indices = region_size_df_sorted['Label'].index.values  # Get indices after sorting
sorted_region=region_size_df_sorted['Region'].values
print((sorted_region[1]))
# Step 2: Apply sorting to the columns of the matrices

sorted_avrofMCApairs_T1_filtered = avrofMCApairs_T1_filtered[:, sorted_indices]  # Reorder columns
sorted_avrofMCApairs_bold_filtered= avrofMCApairs_bold_filtered[:, sorted_indices]  # Reorder columns

# Now sorted_avrofMCApairs_T1 and sorted_avrofMCApairs_bold have regions sorted by increasing size


In [ ]:
import plotly.subplots as sp
import plotly.graph_objects as go
import numpy as np

# Determine shared color map limits
vmin = min(sorted_avrofMCApairs_T1_filtered.min(), sorted_avrofMCApairs_bold_filtered.min())
vmax = max(sorted_avrofMCApairs_T1_filtered.max(), sorted_avrofMCApairs_bold_filtered.max())
# Convert index dictionaries to ordered lists of subject IDs
indexT_labels_filtered = [filtered_indexT[i] for i in filtered_indexT.keys()]  # Extract subject IDs from indexT dictionary
indexB_labels_filtered = [filtered_indexB[i] for i in filtered_indexB.keys()]  # Extract subject IDs from indexB dictionary
sorted_region=region_size_df_sorted['Region'].values

# Create subplots
fig = sp.make_subplots(rows=1, cols=2, subplot_titles=["T1", "Bold"])

# Add first heatmap
fig.add_trace(
    go.Heatmap(
        z=sorted_avrofMCApairs_T1_filtered,
        y=indexT_labels_filtered,
        x=sorted_region,
        colorscale='rainbow_r',
        zmin=vmin,
        zmax=vmax,
        colorbar=dict(title="rainbow_r", len=0.8),
    ),
    row=1,
    col=1
)

# Add second heatmap (share the same color scale)
fig.add_trace(
    go.Heatmap(
        z=sorted_avrofMCApairs_bold_filtered,
        y=indexB_labels_filtered,
        x=sorted_region,
        colorscale='rainbow_r',
        zmin=vmin,
        zmax=vmax,
        showscale=False,  # Hide extra colorbar
    ),
    row=1,
    col=2
)

# Update y-axis labels
fig.update_yaxes(title_text="Subject ID", row=1, col=1)
fig.update_yaxes(title_text="Subject ID", row=1, col=2)

# Update layout
fig.update_layout(
    height=1000, width=2000,
    title_text="Heatmaps of average dice scores of MCApairs_T1 and average of MCApairs_bold across sorted region by size after removing 10 worse subjects from both T1 and Bold",
    coloraxis_colorbar=dict(len=0.8)
)

fig.show()



In [ ]:
#Sorting based on region size and average across subjects values
import numpy as np

# Compute mean across subjects (axis=0)
mean_values = np.mean(sorted_avrofMCApairs_T1, axis=0)  
mean_valuesb = np.mean(sorted_avrofMCApairs_bold, axis=0)  

# Get sorted indices based on mean values
sortedavr_indices = np.argsort(mean_values).astype(int)  
sortedavr_indicesb = np.argsort(mean_valuesb).astype(int)  

# Convert region labels to NumPy array and reorder
sorted_region = np.array(region_size_df_sorted['Region'].values)  
sortedavr_region = sorted_region[sortedavr_indices]  
sortedavr_regionb = sorted_region[sortedavr_indices]  

# Reorder matrix columns (regions)
sortedavr_avrofMCApairs_T1= sorted_avrofMCApairs_T1[:, sortedavr_indices]
sortedavr_avrofMCApairs_bold_T1= sorted_avrofMCApairs_bold[:, sortedavr_indices]
sortedavr_avrofMCApairs_bold= sorted_avrofMCApairs_bold[:, sortedavr_indicesb]

# Compute mean across regions for each subject (axis=1)
Smean_values = np.mean(sortedavr_avrofMCApairs_T1, axis=1)  
Smean_valuesb = np.mean(sortedavr_avrofMCApairs_bold, axis=1)  

# Get sorted indices for subjects (convert to integer type)
Ssortedavr_indices = np.argsort(Smean_values).astype(int)  
Ssortedavr_indicesb = np.argsort(Smean_valuesb).astype(int)  

# Convert subject labels to NumPy array
indexT_labels= np.array([indexT[i] for i in indexT.keys()])  
indexB_labels= np.array([indexB[i] for i in indexB.keys()])  

# Reorder subject labels **(For reference, NOT for indexing)**
sorted_indexT_labels = indexT_labels[Ssortedavr_indices]  
sorted_indexB_labels= indexB_labels[Ssortedavr_indicesb]  

# sortedavr_stdofMCApairs_T1 = sortedavr_stdofMCApairs_T1[Ssortedavr_indices, :]  
# sortedavr_stdofMCApairs_bold= sortedavr_stdofMCApairs_bold[Ssortedavr_indices, :]  
sortedavr_avrofMCApairs_T1= sortedavr_avrofMCApairs_T1[Ssortedavr_indices, :]  
sortedavr_avrofMCApairs_bold_T1= sortedavr_avrofMCApairs_bold_T1[Ssortedavr_indices, :]  
sortedavr_avrofMCApairs_bold= sortedavr_avrofMCApairs_bold[Ssortedavr_indicesb, :]  



In [ ]:
import plotly.subplots as sp
import plotly.graph_objects as go
import numpy as np

# Determine shared color map limits
vmin = 0.85 # min(sortedavr_avrofMCApairs_T1.min(), sortedavr_avrofMCApairs_bold.min())
vmax = max(sortedavr_avrofMCApairs_T1.max(), sortedavr_avrofMCApairs_bold_T1.max())


# Create subplots
fig = sp.make_subplots(rows=1, cols=2, subplot_titles=["T1", "Bold"])

# Add first heatmap
fig.add_trace(
    go.Heatmap(
        z=sortedavr_avrofMCApairs_T1,
        y=sorted_indexT_labels,
        x=sortedavr_region,
        colorscale='rainbow_r',
        zmin=vmin,
        zmax=vmax,
        colorbar=dict(title="rainbow_r", len=0.8),
    ),
    row=1,
    col=1
)

# Add second heatmap (share the same color scale)
fig.add_trace(
    go.Heatmap(
        z=sortedavr_avrofMCApairs_bold_T1,
        y=sorted_indexT_labels,
        x=sortedavr_region,
        colorscale='rainbow_r',
        zmin=vmin,
        zmax=vmax,
        showscale=False,  # Hide extra colorbar
    ),
    row=1,
    col=2
)

# Update y-axis labels
fig.update_yaxes(title_text="Subject ID", row=1, col=1)
fig.update_yaxes(title_text="Subject ID", row=1, col=2)

# Update layout
fig.update_layout(
    height=1000, width=2000,
    title_text="Heatmaps of the AVerage of MCA pairs_T1 and Average of MCApairs_bold sorted based on the average dice scores across subject and across regions of T1 ",
    coloraxis_colorbar=dict(len=0.8)
)

fig.show()



In [ ]:
import plotly.subplots as sp
import plotly.graph_objects as go
import numpy as np

# Determine shared color map limits
vmin = 0.85 # min(sortedavr_avrofMCApairs_T1.min(), sortedavr_avrofMCApairs_bold.min())
vmax = max(sortedavr_avrofMCApairs_T1.max(), sortedavr_avrofMCApairs_bold.max())


# Create subplots
fig = sp.make_subplots(rows=1, cols=2, subplot_titles=["T1", "Bold"])

# Add first heatmap
fig.add_trace(
    go.Heatmap(
        z=sortedavr_avrofMCApairs_T1,
        y=sorted_indexT_labels,
        x=sortedavr_region,
        colorscale='rainbow_r',
        zmin=vmin,
        zmax=vmax,
        colorbar=dict(title="rainbow_r", len=0.8),
    ),
    row=1,
    col=1
)

# Add second heatmap (share the same color scale)
fig.add_trace(
    go.Heatmap(
        z=sortedavr_avrofMCApairs_bold,
        y=sorted_indexB_labels,
        x=sortedavr_regionb,
        colorscale='rainbow_r',
        zmin=vmin,
        zmax=vmax,
        showscale=False,  # Hide extra colorbar
    ),
    row=1,
    col=2
)

# Update y-axis labels
fig.update_yaxes(title_text="Subject ID", row=1, col=1)
fig.update_yaxes(title_text="Subject ID", row=1, col=2)

# Update layout
fig.update_layout(
    height=1000, width=2000,
    title_text="Heatmaps of the AVerage of MCA pairs_T1 and Average of MCApairs_bold sorted based on the average dice scores across subject and across regions of T1 and bold separatedly ",
    coloraxis_colorbar=dict(len=0.8)
)

fig.show()



In [ ]:
import numpy as np

# Compute mean across subjects (axis=0)
mean_values = np.mean(sorted_avrofMCApairs_T1_filtered, axis=0)  
mean_valuesb = np.mean(sorted_avrofMCApairs_bold_filtered, axis=0)  

# Get sorted indices based on mean values
sortedavr_indices = np.argsort(mean_values).astype(int)  
sortedavr_indicesb = np.argsort(mean_valuesb).astype(int)  

# Convert region labels to NumPy array and reorder
sorted_region = np.array(region_size_df_sorted['Region'].values)  
sortedavr_region = sorted_region[sortedavr_indices]  
sortedavr_regionb = sorted_region[sortedavr_indicesb]  

# Reorder matrix columns (regions)

sortedavr_avrofMCApairs_T1_filtered = sorted_avrofMCApairs_T1_filtered[:, sortedavr_indices]
sortedavr_avrofMCApairs_bold_T1_filtered = sorted_avrofMCApairs_bold_filtered[:, sortedavr_indices]
sortedavr_avrofMCApairs_bold_filtered = sorted_avrofMCApairs_bold_filtered[:, sortedavr_indicesb]

# Compute mean across regions for each subject (axis=1)
Smean_values = np.mean(sortedavr_avrofMCApairs_T1_filtered, axis=1)  
Smean_valuesb = np.mean(sortedavr_avrofMCApairs_bold_filtered, axis=1)  

# Get sorted indices for subjects (convert to integer type)
Ssortedavr_indices = np.argsort(Smean_values).astype(int)  
Ssortedavr_indicesb = np.argsort(Smean_valuesb).astype(int)  

# Convert subject labels to NumPy array
indexT_labels_filtered = np.array([filtered_indexT[i] for i in filtered_indexT.keys()])  
indexB_labels_filtered = np.array([filtered_indexB[i] for i in filtered_indexB.keys()])  

# Reorder subject labels **(For reference, NOT for indexing)**
sorted_indexT_labels_filtered = indexT_labels_filtered[Ssortedavr_indices]  
sorted_indexB_labels_filtered = indexB_labels_filtered[Ssortedavr_indicesb]  


sortedavr_avrofMCApairs_T1_filtered = sortedavr_avrofMCApairs_T1_filtered[Ssortedavr_indices, :]  
sortedavr_avrofMCApairs_bold_T1_filtered = sortedavr_avrofMCApairs_bold_T1_filtered[Ssortedavr_indices, :]  
sortedavr_avrofMCApairs_bold_filtered = sortedavr_avrofMCApairs_bold_filtered[Ssortedavr_indicesb, :]  




In [ ]:
import plotly.subplots as sp
import plotly.graph_objects as go
import numpy as np

# Determine shared color map limits
vmin = 0.85 # min(sortedavr_avrofMCApairs_T1.min(), sortedavr_avrofMCApairs_bold.min())
vmax = max(sortedavr_avrofMCApairs_T1_filtered.max(), sortedavr_avrofMCApairs_bold_T1_filtered.max())


# Create subplots
fig = sp.make_subplots(rows=1, cols=2, subplot_titles=["T1", "Bold"])

# Add first heatmap
fig.add_trace(
    go.Heatmap(
        z=sortedavr_avrofMCApairs_T1_filtered,
        y=sorted_indexT_labels_filtered,
        x=sortedavr_region,
        colorscale='rainbow_r',
        zmin=vmin,
        zmax=vmax,
        colorbar=dict(title="rainbow_r", len=0.8),
    ),
    row=1,
    col=1
)

# Add second heatmap (share the same color scale)
fig.add_trace(
    go.Heatmap(
        z=sortedavr_avrofMCApairs_bold_T1_filtered,
        y=sorted_indexT_labels_filtered,
        x=sortedavr_region,
        colorscale='rainbow_r',
        zmin=vmin,
        zmax=vmax,
        showscale=False,  # Hide extra colorbar
    ),
    row=1,
    col=2
)

# Update y-axis labels
fig.update_yaxes(title_text="Subject ID", row=1, col=1)
fig.update_yaxes(title_text="Subject ID", row=1, col=2)

# Update layout
fig.update_layout(
    height=1000, width=2000,
    title_text="Heatmaps of the AVerage of MCA pairs_T1 and Average of MCApairs_bold sorted based on the average dice scores across subject and across regions of T1 ",
    coloraxis_colorbar=dict(len=0.8)
)

fig.show()



In [ ]:
import plotly.subplots as sp
import plotly.graph_objects as go
import numpy as np

# Determine shared color map limits
vmin = 0.85 #min(sortedavr_avrofMCApairs_T1_filtered.min(), sortedavr_avrofMCApairs_bold_filtered.min())
vmax = max(sortedavr_avrofMCApairs_T1_filtered.max(), sortedavr_avrofMCApairs_bold_filtered.max())


# Create subplots
fig = sp.make_subplots(rows=1, cols=2, subplot_titles=["T1", "Bold"])

# Add first heatmap
fig.add_trace(
    go.Heatmap(
        z=sortedavr_avrofMCApairs_T1_filtered,
        y=sorted_indexT_labels_filtered,
        x=sortedavr_region,
        colorscale='rainbow_r',
        zmin=vmin,
        zmax=vmax,
        colorbar=dict(title="Average", len=0.8),
    ),
    row=1,
    col=1
)

# Add second heatmap (share the same color scale)
fig.add_trace(
    go.Heatmap(
        z=sortedavr_avrofMCApairs_bold_filtered,
        y=sorted_indexB_labels_filtered,
        x=sortedavr_regionb,
        colorscale='rainbow_r',
        zmin=vmin,
        zmax=vmax,
        showscale=False,  # Hide extra colorbar
    ),
    row=1,
    col=2
)

# Update y-axis labels
fig.update_yaxes(title_text="Subject ID", row=1, col=1)
fig.update_yaxes(title_text="Subject ID", row=1, col=2)

# Update layout
fig.update_layout(
    height=1000, width=2000,
    title_text="Heatmaps of the AVerage of MCA pairs_T1 and Average of MCApairs_bold sorted based on the average dice scores across subject and across regions of T1 and bold separatedly",
    coloraxis_colorbar=dict(len=0.8)
)

fig.show()



In [ ]:
stack_matrix_T11=create_stackmatrix_OfDice(result_df_atlasinT1W1)
stack_matrix_bold1=create_stackmatrix_OfDice(result_df_ROI_atlasinT1W2Bold1)

Numerical_min_T11=  np.min(stack_matrix_T11,axis=2)
numerical_min_bold1=np.min(stack_matrix_bold1,axis=2)
min_T11=np.mean(Numerical_min_T11,axis=0)
min_bold1=np.mean(numerical_min_bold1,axis=0)

<!-- $$
Numerical Variability:  \forall r \in \left[ 1 \hspace{0.1cm}, R \right],  \hspace{0.5cm} 
\Delta_{r} = \text{mean}_{s \in \left[ 1 \hspace{0.1cm}, S \right]} \left( \min_{(p,p^{'}),(p^{''},p^{'''}) \in \left[ 1 \hspace{0.1cm}, P \right]} 
\hspace{0.2cm} D \left( V_{r,s,(p,p^{'})}, V_{r,s,(p^{''},p^{'''})} \right) \right)
\newline
Anatomical Variability:  \forall r \in \left[ 1 \hspace{0.1cm}, R \right], \hspace{0.5cm} \Delta^{'}_{r} = \text{mean}_{(p,p^{'}) \in \left[ 1 \hspace{0.1cm}, P \right]} \left( \min_{(s,s^{'}) \in \left[ 1 \hspace{0.1cm}, S \right]} 
\hspace{0.2cm} D \left( V_{r,s,(p,p^{'})}, V_{r,s^{'},(p,p^{'})} \right) \right)\\
\newline
\hspace{0.5cm}R:\text{Number of Regions}, \hspace{0.1cm} S: \text{Number of Subjects}, \hspace{0.2cm} \text{and} \hspace{0.2cm} P: \text{Number of MCA pairs}\,(p,p^{'})


$$ -->